# KNN

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from fancyimpute import IterativeImputer  # pip install fancyimpute
import os
import math
import tqdm
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest # 

(CVXPY) Jun 22 05:24:03 PM: Encountered unexpected exception importing solver CVXOPT:
ImportError('DLL load failed while importing base: Le module spécifié est introuvable.')
(CVXPY) Jun 22 05:24:03 PM: Encountered unexpected exception importing solver GLPK:
ImportError('DLL load failed while importing base: Le module spécifié est introuvable.')
(CVXPY) Jun 22 05:24:03 PM: Encountered unexpected exception importing solver GLPK_MI:
ImportError('DLL load failed while importing base: Le module spécifié est introuvable.')


In [52]:
# pip install missingpy

In [53]:
# Move up 2 directories
data_directory = '../..' 

# Load the CSV files
asthma_df = pd.read_csv(os.path.join(data_directory, 'Data', 'astma.csv'))

In [54]:
# In the asthma dataset there seems to be a problem with HR00 to HR23
# The heart rate goes above a million sometimes
# It seems like the first couple digits correspond with the expected heart rate.
# 2 cases: HR below 100 and HR above 100

# Loop through each column in the dataframe
def preprocess(asthma_df):
    for col in asthma_df.loc[:, "HR00":"HR23"]:
        for i in range(len(asthma_df[col])):
            if (asthma_df.loc[i, col] > 1000):
                # Take first 2 digits of float
                first_2_digits = str(asthma_df.loc[i, col])[:2]
                temp_number = float(first_2_digits)
                # If heart rate lower than 20 it means it should be greater than 100 (assuming heart rates < 200)
                if temp_number < 20:
                    val = str(asthma_df.loc[i, col])[:3] + '.' + str(asthma_df.loc[i, col])[3]
                    asthma_df.loc[i, col] = float(val)
                else:
                    val = str(asthma_df.loc[i, col])[:2] + '.' + str(asthma_df.loc[i, col])[2]
                    asthma_df.loc[i, col] = float(val)
                pass
    return asthma_df


In [55]:
# get index of a random element in a dataframe
def get_random_index(df, columns):
    # select a random element that is not Nan
    # take a random column in columns
    random_index = np.random.randint(0, len(df))
    random_column = columns[np.random.randint(0, len(columns))]

    df.loc[random_index, random_column]

    while pd.isna(df.loc[random_index, random_column]):
        random_index = np.random.randint(0, len(df))
        random_column = columns[np.random.randint(0, len(columns))]

    return [random_index, random_column]

In [57]:
asthma_tmp = preprocess(pd.read_csv(os.path.join(data_directory, 'Data', 'astma.csv')))
asthma_tmp.to_csv(('asthma_tmp.csv'), index=False)

In [ ]:
# # Play with the number of neighbors
# # Look at the kids with the fewest missing data
# difference = []
# for i in tqdm.tqdm(range(1000)):
#     asthma_df = pd.read_csv('asthma_tmp.csv')
#     random_index = get_random_index(asthma_df, ["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
#                 "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])
#     tmp = asthma_df.loc[random_index[0], random_index[1]]
#     asthma_df.loc[random_index[0], random_index[1]] = np.nan
#     imputer = KNNImputer(n_neighbors = 5)          # n_neighbors=2
#     asthma_df.loc[:, "HR00":"HR23"] = imputer.fit_transform(asthma_df.loc[:, "HR00":"HR23"])
#     # get the absolute value of the difference between the original value and the imputed value
#     difference.append(abs(tmp - asthma_df.loc[random_index[0], random_index[1]]))



# print(difference)

In [ ]:
# print(difference)

In [ ]:
adress = []
for i in range(5000):
    adress.append(get_random_index(asthma_df, ["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                                               "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"]))

# TRY

In [ ]:
# Play with the number of neighbors
# Look at the kids with the fewest missing data

# TODO: read the csv all the time.

differenceKNN = []
differenceMICE = []
differenceMissForest = []
for random_index in tqdm.tqdm(adress):
    asthma_df = pd.read_csv('asthma_tmp.csv')
    tmp = asthma_df.loc[random_index[0], random_index[1]]
    asthma_df.loc[random_index[0], random_index[1]] = np.nan
    imputerKNN = KNNImputer(n_neighbors = 5)          # n_neighbors=2
    # asthma_df.loc[:, "HR00":"HR23"] = imputerKNN.fit_transform(asthma_df.loc[:, "HR00":"HR23"])
    asthma_KNN = (imputerKNN.fit_transform(asthma_df.loc[:, "HR00":"HR23"]))
    asthma_KNN = pd.DataFrame(asthma_KNN, columns=["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])
    # get the absolute value of the difference between the original value and the imputed value
    differenceKNN.append(abs(tmp - asthma_KNN.loc[random_index[0], random_index[1]]))

    asthma_df = pd.read_csv('asthma_tmp.csv')

    mice_imputer = IterativeImputer(verbose=0)
    imputed_data = mice_imputer.fit_transform(asthma_df.loc[:, "HR00":"HR23"])
    # Convert the imputed data array back to a DataFrame
    asthma_MICE = pd.DataFrame(imputed_data, columns=["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])
    differenceMICE.append(abs(tmp - asthma_MICE.loc[random_index[0], random_index[1]]))

    


c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
 99%|█████████▉| 4952/5000 [1:46:36<01:13,  1.53s/it]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
 99%|█████████▉| 4953/5000 [1:46:38<01:09,  1.48s/it]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
 99%|█████████▉| 4954/5000 [1:46:39<01:05,  1.43s/it]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
 99%|█████████▉| 4955/5000 [1:46:40<01:03,  1.41s/it]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [Iterat

In [ ]:
print(differenceKNN)
print(differenceMICE)

print(sum(differenceKNN)/len(differenceKNN))
print(sum(differenceMICE)/len(differenceMICE))

[8.620000000000005, 14.780000000000001, 1.0800000000000125, 1.279999999999987, 12.279999999999987, 16.900000000000006, 3.8800000000000097, 5.299999999999997, 9.490000000000009, 10.799999999999997, 18.099999999999994, 22.059999999999988, 11.199999999999989, 18.820000000000007, 20.580000000000013, 18.08, 3.559999999999988, 0.519999999999996, 20.950000000000003, 16.519999999999996, 9.170000000000016, 26.890000000000015, 5.8799999999999955, 9.699999999999989, 0.6799999999999926, 6.739999999999995, 4.159999999999982, 14.239999999999995, 2.839999999999989, 11.330000000000013, 2.1200000000000045, 28.97, 11.849999999999994, 2.8699999999999903, 20.230000000000004, 2.0999999999999943, 0.9599999999999937, 0.9799999999999969, 1.0, 9.090000000000003, 8.469999999999999, 7.6299999999999955, 1.8200000000000074, 2.780000000000001, 5.180000000000007, 18.47999999999999, 0.6200000000000045, 6.019999999999982, 7.640000000000015, 5.950000000000003, 6.400000000000006, 11.659999999999997, 1.7800000000000011, 

In [ ]:
KNNResult = [8.620000000000005, 14.780000000000001, 1.0800000000000125, 1.279999999999987, 12.279999999999987, 16.900000000000006, 3.8800000000000097, 5.299999999999997, 9.490000000000009, 10.799999999999997, 18.099999999999994, 22.059999999999988, 11.199999999999989, 18.820000000000007, 20.580000000000013, 18.08, 3.559999999999988, 0.519999999999996, 20.950000000000003, 16.519999999999996, 9.170000000000016, 26.890000000000015, 5.8799999999999955, 9.699999999999989, 0.6799999999999926, 6.739999999999995, 4.159999999999982, 14.239999999999995, 2.839999999999989, 11.330000000000013, 2.1200000000000045, 28.97, 11.849999999999994, 2.8699999999999903, 20.230000000000004, 2.0999999999999943, 0.9599999999999937, 0.9799999999999969, 1.0, 9.090000000000003, 8.469999999999999, 7.6299999999999955, 1.8200000000000074, 2.780000000000001, 5.180000000000007, 18.47999999999999, 0.6200000000000045, 6.019999999999982, 7.640000000000015, 5.950000000000003, 6.400000000000006, 11.659999999999997, 1.7800000000000011, 2.2900000000000063, 12.809999999999988, 5.700000000000003, 1.0799999999999983, 12.060000000000002, 15.260000000000005, 1.8799999999999955, 5.11999999999999, 1.7000000000000028, 8.829999999999998, 13.920000000000002, 3.3400000000000034, 3.3400000000000034, 6.440000000000012, 2.180000000000007, 11.23999999999998, 5.049999999999997, 15.039999999999992, 23.029999999999987, 1.1799999999999926, 7.530000000000001, 5.950000000000003, 1.0799999999999983, 5.659999999999997, 2.0400000000000063, 1.8600000000000136, 1.5300000000000011, 6.179999999999993, 6.640000000000015, 17.64, 3.5999999999999943, 2.280000000000001, 2.0999999999999943, 6.6299999999999955, 26.330000000000013, 0.29000000000000625, 12.52000000000001, 15.190000000000012, 6.449999999999989, 1.2399999999999949, 18.870000000000005, 6.440000000000012, 3.1400000000000006, 16.690000000000012, 3.279999999999987, 2.240000000000009, 5.1200000000000045, 3.5400000000000063, 24.180000000000007, 8.980000000000004, 14.600000000000009, 23.08, 2.7299999999999898, 21.439999999999998, 3.0799999999999983, 6.52000000000001, 1.8700000000000045, 13.0, 0.28000000000000114, 8.460000000000008, 3.4000000000000057, 8.670000000000016, 3.8599999999999994, 4.780000000000001, 3.940000000000012, 5.640000000000001, 19.419999999999987, 4.159999999999997, 6.260000000000005, 8.939999999999998, 16.459999999999994, 7.059999999999988, 6.700000000000003, 13.809999999999988, 2.2099999999999937, 3.180000000000007, 5.920000000000002, 5.199999999999989, 0.5, 7.030000000000001, 26.849999999999994, 3.4200000000000017, 3.8200000000000074, 2.0400000000000063, 2.579999999999984, 2.2900000000000063, 10.900000000000006, 13.460000000000022, 4.400000000000006, 1.1999999999999886, 7.039999999999992, 3.9399999999999977, 12.600000000000009, 5.420000000000002, 9.5, 20.08999999999999, 3.440000000000012, 1.579999999999984, 5.890000000000015, 3.8299999999999983, 7.1000000000000085, 5.959999999999994, 13.679999999999993, 15.120000000000005, 3.4099999999999824, 10.320000000000007, 12.420000000000002, 5.200000000000003, 2.2399999999999807, 3.8400000000000034, 1.4899999999999949, 4.9399999999999835, 5.019999999999996, 27.68000000000002, 15.659999999999982, 0.07000000000000739, 3.0200000000000102, 8.219999999999999, 9.549999999999997, 14.86999999999999, 3.5800000000000125, 6.259999999999991, 5.219999999999999, 7.1699999999999875, 4.439999999999998, 12.760000000000005, 10.829999999999998, 6.25, 9.47999999999999, 7.540000000000013, 17.4, 4.939999999999998, 5.670000000000002, 5.820000000000007, 34.91999999999999, 12.959999999999994, 19.02000000000001, 0.6399999999999864, 16.560000000000002, 8.480000000000004, 0.5100000000000051, 27.97999999999999, 0.9200000000000017, 1.539999999999992, 0.09999999999999432, 6.040000000000006, 4.579999999999984, 8.320000000000007, 10.27000000000001, 5.900000000000006, 1.6200000000000045, 13.219999999999985, 5.52000000000001, 13.920000000000002, 4.820000000000007, 5.519999999999996, 8.820000000000007, 2.8499999999999943, 4.280000000000001, 7.599999999999994, 1.740000000000009, 6.0, 7.010000000000005, 1.0600000000000023, 5.720000000000013, 4.700000000000003, 7.420000000000002, 4.34999999999998, 4.239999999999995, 12.620000000000005, 15.25, 0.12999999999999545, 7.340000000000003, 2.6599999999999966, 26.799999999999997, 12.120000000000005, 5.780000000000001, 19.440000000000012, 12.379999999999995, 3.6899999999999835, 5.530000000000001, 3.5700000000000074, 9.439999999999998, 8.719999999999999, 9.120000000000005, 0.23999999999999488, 11.990000000000009, 4.739999999999995, 3.719999999999999, 0.2599999999999909, 20.67, 2.4399999999999977, 19.0, 10.299999999999997, 0.35999999999999943, 0.18000000000000682, 16.22999999999999, 8.590000000000003, 32.140000000000015, 9.430000000000007, 3.4099999999999966, 4.319999999999993, 0.6999999999999886, 12.519999999999996, 7.300000000000011, 5.159999999999997, 19.650000000000006, 8.919999999999987, 9.700000000000003, 6.519999999999996, 9.540000000000006, 0.9599999999999937, 8.870000000000005, 3.6200000000000045, 1.920000000000016, 8.33999999999999, 1.4299999999999926, 16.08, 8.180000000000007, 12.239999999999995, 1.5499999999999972, 6.02000000000001, 1.7000000000000028, 2.8100000000000023, 8.959999999999994, 12.019999999999996, 13.690000000000012, 6.540000000000006, 4.8700000000000045, 5.640000000000015, 2.509999999999991, 19.83999999999999, 2.839999999999989, 5.159999999999997, 6.1200000000000045, 1.3199999999999932, 19.200000000000003, 5.240000000000009, 10.240000000000009, 7.560000000000002, 8.219999999999999, 21.74000000000001, 1.8400000000000034, 25.28, 12.099999999999994, 3.180000000000007, 6.659999999999982, 1.680000000000021, 2.5200000000000102, 15.189999999999998, 1.0799999999999983, 2.6200000000000045, 5.900000000000006, 5.150000000000006, 6.6299999999999955, 0.44000000000001194, 2.819999999999993, 10.25, 2.950000000000003, 9.759999999999991, 5.079999999999991, 8.970000000000013, 0.2799999999999869, 18.36, 6.599999999999994, 0.28000000000000114, 5.27000000000001, 2.239999999999995, 8.480000000000004, 1.7000000000000028, 1.3400000000000034, 12.61999999999999, 9.709999999999994, 8.010000000000005, 8.14, 13.399999999999991, 11.420000000000002, 4.799999999999997, 2.6899999999999977, 0.3199999999999932, 3.6400000000000006, 0.4900000000000091, 5.6200000000000045, 9.329999999999998, 11.060000000000002, 3.6099999999999994, 12.760000000000005, 15.700000000000003, 3.6500000000000057, 0.7400000000000091, 1.920000000000016, 0.6900000000000119, 3.1799999999999926, 0.6800000000000068, 4.6200000000000045, 0.8299999999999983, 27.409999999999997, 5.759999999999991, 8.680000000000007, 12.110000000000014, 3.759999999999991, 12.959999999999994, 17.939999999999998, 2.1200000000000045, 8.099999999999994, 9.779999999999987, 9.060000000000002, 1.5799999999999983, 8.539999999999992, 27.039999999999992, 9.120000000000005, 11.900000000000006, 12.720000000000013, 5.599999999999994, 5.820000000000007, 6.640000000000001, 9.36, 0.4399999999999977, 1.539999999999992, 1.3400000000000034, 14.669999999999987, 32.760000000000005, 3.5799999999999983, 5.209999999999994, 0.7299999999999898, 10.019999999999996, 4.739999999999995, 8.420000000000002, 3.1599999999999966, 9.100000000000009, 0.5, 3.6799999999999926, 0.30999999999998806, 5.739999999999995, 22.97999999999999, 7.489999999999995, 1.4300000000000068, 1.1400000000000148, 12.019999999999996, 10.819999999999993, 13.219999999999999, 2.490000000000009, 8.179999999999993, 7.02000000000001, 12.83999999999999, 5.450000000000003, 5.329999999999998, 4.400000000000006, 12.22999999999999, 18.159999999999982, 4.599999999999994, 1.6400000000000006, 9.899999999999999, 6.820000000000007, 3.9000000000000057, 0.28999999999999204, 4.780000000000015, 18.760000000000005, 2.6699999999999875, 8.799999999999997, 14.939999999999998, 1.9099999999999966, 8.659999999999997, 11.760000000000005, 2.1400000000000006, 2.4599999999999937, 4.6000000000000085, 5.279999999999987, 4.579999999999998, 1.519999999999996, 10.370000000000005, 0.5600000000000023, 3.960000000000008, 1.6000000000000014, 7.660000000000011, 2.180000000000007, 6.5400000000000205, 10.740000000000009, 2.729999999999997, 4.900000000000006, 1.75, 2.3700000000000045, 2.0999999999999943, 31.200000000000003, 7.920000000000002, 0.45999999999997954, 2.9599999999999795, 12.079999999999998, 3.740000000000009, 7.340000000000003, 19.019999999999982, 9.950000000000003, 7.590000000000003, 1.960000000000008, 11.060000000000002, 1.0899999999999892, 27.75, 7.010000000000005, 5.780000000000001, 3.1599999999999966, 9.789999999999992, 7.8700000000000045, 14.969999999999999, 5.660000000000011, 0.010000000000005116, 8.649999999999991, 3.740000000000002, 11.079999999999984, 22.370000000000005, 5.799999999999997, 4.419999999999995, 0.8599999999999994, 9.969999999999999, 7.439999999999998, 11.219999999999999, 1.9000000000000057, 6.639999999999986, 2.6500000000000057, 2.5600000000000023, 0.4399999999999977, 10.260000000000005, 3.6899999999999977, 14.969999999999999, 18.930000000000007, 3.0800000000000125, 4.359999999999985, 5.239999999999995, 5.609999999999999, 28.120000000000005, 31.28, 7.280000000000001, 2.1599999999999966, 14.120000000000005, 7.579999999999998, 2.8200000000000074, 5.02000000000001, 8.870000000000005, 8.780000000000001, 4.939999999999998, 4.179999999999993, 0.9000000000000057, 9.460000000000008, 5.8799999999999955, 0.47999999999998977, 2.200000000000003, 4.679999999999993, 6.059999999999995, 13.530000000000001, 10.619999999999976, 0.020000000000010232, 7.819999999999993, 6.079999999999984, 0.1599999999999966, 17.049999999999997, 6.910000000000025, 17.030000000000015, 5.859999999999999, 5.719999999999999, 20.75, 3.3400000000000034, 7.780000000000001, 20.739999999999995, 8.560000000000002, 1.0200000000000102, 2.219999999999999, 7.799999999999997, 7.720000000000013, 2.0799999999999983, 2.739999999999995, 6.400000000000006, 9.140000000000015, 6.52000000000001, 19.159999999999982, 1.960000000000008, 13.379999999999995, 18.940000000000012, 12.25, 1.9699999999999989, 5.159999999999997, 16.30000000000001, 1.5800000000000125, 2.0900000000000034, 9.760000000000005, 4.840000000000003, 27.02000000000001, 33.67999999999999, 7.599999999999994, 1.660000000000025, 1.7000000000000028, 8.47999999999999, 6.180000000000007, 8.010000000000005, 3.0600000000000023, 5.739999999999995, 8.010000000000005, 5.579999999999998, 1.940000000000012, 0.9599999999999937, 54.170000000000016, 9.480000000000004, 1.210000000000008, 11.219999999999985, 0.28000000000000114, 0.9899999999999949, 3.049999999999997, 8.040000000000006, 10.879999999999995, 1.1799999999999926, 4.739999999999995, 5.070000000000007, 1.8100000000000023, 11.419999999999987, 23.070000000000007, 13.159999999999997, 2.969999999999999, 15.120000000000005, 0.0, 5.159999999999997, 7.799999999999997, 14.879999999999995, 49.78, 7.159999999999982, 6.799999999999997, 4.740000000000009, 3.1999999999999886, 2.440000000000012, 3.200000000000003, 3.3600000000000136, 2.489999999999995, 1.1200000000000045, 5.560000000000002, 3.5200000000000102, 0.37999999999999545, 17.650000000000006, 7.52000000000001, 8.600000000000009, 0.28000000000000114, 1.3399999999999892, 13.900000000000006, 2.9000000000000057, 4.030000000000001, 0.9599999999999937, 0.5400000000000063, 33.099999999999994, 7.819999999999993, 16.560000000000016, 0.8100000000000023, 3.3700000000000045, 2.8200000000000074, 7.820000000000007, 6.660000000000011, 4.019999999999996, 13.299999999999997, 2.289999999999999, 0.9199999999999875, 32.8, 4.640000000000001, 26.33, 2.0, 11.079999999999998, 6.519999999999996, 2.339999999999989, 0.5200000000000102, 1.0400000000000063, 3.0, 6.460000000000008, 10.509999999999991, 0.10000000000000142, 23.159999999999997, 2.8100000000000023, 7.3999999999999915, 15.579999999999998, 19.599999999999994, 11.200000000000017, 13.320000000000007, 3.240000000000009, 1.3999999999999915, 3.6299999999999955, 18.92, 8.759999999999991, 14.429999999999993, 12.54000000000002, 3.1200000000000045, 7.390000000000001, 1.5, 0.7900000000000063, 3.6799999999999926, 7.680000000000007, 37.230000000000004, 14.019999999999996, 11.539999999999992, 4.019999999999996, 7.290000000000006, 1.539999999999992, 7.859999999999999, 7.820000000000007, 0.47999999999998977, 1.7000000000000028, 2.539999999999992, 6.02000000000001, 22.260000000000005, 3.5600000000000023, 5.52000000000001, 5.1200000000000045, 15.019999999999996, 4.859999999999999, 6.049999999999997, 6.459999999999994, 5.539999999999992, 16.620000000000005, 7.279999999999987, 2.4200000000000017, 17.179999999999993, 7.820000000000007, 0.9200000000000159, 3.8799999999999955, 1.1299999999999955, 7.900000000000006, 5.040000000000006, 4.059999999999988, 2.509999999999991, 10.27000000000001, 9.129999999999995, 8.420000000000009, 7.8999999999999915, 16.039999999999992, 4.6200000000000045, 2.280000000000001, 2.5, 11.450000000000003, 1.0200000000000102, 10.599999999999994, 7.900000000000006, 20.47999999999999, 5.099999999999994, 0.8100000000000023, 9.920000000000002, 4.6200000000000045, 0.1799999999999926, 7.0800000000000125, 0.5200000000000102, 14.509999999999991, 4.0, 16.74000000000001, 1.4199999999999875, 0.789999999999992, 8.420000000000002, 1.2800000000000011, 3.530000000000001, 0.539999999999992, 3.259999999999991, 11.460000000000008, 12.700000000000003, 2.180000000000007, 7.259999999999991, 3.799999999999997, 4.460000000000008, 7.6000000000000085, 13.900000000000006, 16.77000000000001, 1.6199999999999903, 0.7199999999999989, 5.039999999999992, 6.299999999999997, 3.280000000000001, 5.179999999999993, 0.04000000000000625, 2.8200000000000074, 17.339999999999975, 14.719999999999999, 0.6399999999999864, 7.820000000000007, 2.8200000000000074, 66.16, 9.77000000000001, 1.4000000000000057, 10.27000000000001, 5.9199999999999875, 19.200000000000003, 5.079999999999998, 12.440000000000005, 3.700000000000003, 17.72, 13.670000000000002, 3.019999999999996, 1.460000000000008, 3.3400000000000034, 8.0, 20.72, 8.679999999999993, 10.219999999999999, 7.890000000000015, 13.460000000000008, 4.11999999999999, 2.799999999999997, 4.289999999999992, 1.6200000000000045, 2.660000000000011, 9.540000000000006, 6.070000000000007, 4.61999999999999, 0.3999999999999915, 5.179999999999993, 15.900000000000006, 32.44000000000001, 3.6400000000000006, 7.700000000000003, 4.579999999999984, 0.4800000000000182, 1.7999999999999972, 2.5600000000000023, 3.8799999999999955, 0.36000000000001364, 1.1100000000000136, 1.019999999999996, 10.219999999999999, 12.040000000000006, 5.259999999999991, 10.97999999999999, 12.679999999999993, 10.319999999999993, 4.010000000000005, 14.400000000000006, 0.020000000000010232, 0.9200000000000159, 9.700000000000003, 6.8700000000000045, 13.059999999999988, 1.0800000000000125, 0.38999999999998636, 5.040000000000006, 8.400000000000006, 5.8799999999999955, 12.039999999999992, 1.0, 3.720000000000013, 0.5, 16.58, 7.670000000000002, 6.739999999999995, 1.3100000000000023, 3.8799999999999955, 1.7299999999999898, 35.34, 6.049999999999997, 2.8799999999999955, 3.0600000000000023, 10.570000000000007, 8.060000000000002, 11.86, 10.11, 1.5200000000000102, 1.5300000000000011, 1.2800000000000011, 1.3799999999999955, 2.019999999999996, 0.29000000000000625, 15.700000000000003, 13.700000000000003, 5.079999999999998, 3.5600000000000023, 2.5600000000000023, 6.800000000000011, 13.480000000000004, 6.540000000000006, 1.9800000000000182, 3.6000000000000085, 2.220000000000013, 6.939999999999998, 5.839999999999989, 2.5800000000000125, 5.959999999999994, 4.4199999999999875, 3.3400000000000034, 0.29999999999999716, 7.889999999999986, 5.310000000000002, 9.820000000000007, 6.799999999999997, 12.740000000000009, 8.579999999999998, 0.8599999999999994, 9.289999999999992, 5.039999999999992, 2.019999999999996, 3.0799999999999983, 5.02000000000001, 0.9000000000000057, 19.450000000000003, 3.0300000000000153, 0.18999999999999773, 19.099999999999994, 1.6600000000000108, 17.370000000000005, 2.8800000000000097, 1.7199999999999989, 2.5, 1.4899999999999949, 7.020000000000003, 1.1500000000000057, 1.2600000000000051, 3.799999999999997, 5.019999999999996, 9.400000000000006, 47.260000000000005, 0.6599999999999966, 14.249999999999986, 9.620000000000005, 8.580000000000013, 6.319999999999993, 12.189999999999998, 3.6700000000000017, 21.08, 17.02000000000001, 0.4199999999999875, 2.950000000000003, 11.579999999999998, 9.099999999999994, 4.390000000000001, 3.6200000000000045, 12.650000000000006, 0.6200000000000045, 8.459999999999994, 12.359999999999985, 9.399999999999991, 6.200000000000003, 13.5, 0.47999999999998977, 13.820000000000007, 6.6200000000000045, 4.88000000000001, 5.739999999999995, 6.929999999999993, 0.1799999999999926, 19.870000000000005, 0.09999999999999432, 4.040000000000006, 31.429999999999993, 31.179999999999993, 3.3400000000000034, 6.599999999999994, 5.6000000000000085, 10.189999999999998, 2.910000000000011, 2.279999999999994, 9.159999999999997, 3.4699999999999847, 11.299999999999997, 2.1200000000000045, 1.6199999999999903, 15.64, 8.119999999999997, 5.739999999999995, 2.589999999999989, 5.040000000000006, 28.200000000000003, 3.719999999999999, 5.920000000000002, 1.4599999999999937, 1.1200000000000045, 7.719999999999999, 5.769999999999996, 7.179999999999993, 6.549999999999997, 1.6799999999999926, 8.86999999999999, 0.6700000000000017, 4.959999999999994, 2.3599999999999994, 4.090000000000018, 1.0600000000000165, 5.719999999999999, 9.420000000000002, 7.0600000000000165, 11.64, 5.200000000000017, 17.620000000000005, 2.6999999999999886, 17.320000000000007, 5.320000000000007, 12.86, 1.2000000000000028, 12.64, 2.9200000000000017, 3.960000000000008, 10.620000000000005, 1.559999999999988, 1.9799999999999898, 9.579999999999998, 0.8599999999999994, 7.469999999999985, 5.820000000000007, 2.6799999999999926, 4.160000000000011, 4.22999999999999, 10.779999999999987, 0.980000000000004, 5.5, 6.940000000000012, 3.979999999999997, 4.25, 6.8799999999999955, 0.7200000000000131, 2.4000000000000057, 7.299999999999997, 8.940000000000012, 2.019999999999996, 7.47999999999999, 7.799999999999997, 1.6700000000000017, 9.919999999999987, 15.769999999999996, 2.1599999999999966, 6.47999999999999, 2.2099999999999937, 3.1300000000000097, 0.35999999999999943, 6.180000000000007, 6.180000000000007, 1.4199999999999875, 7.789999999999992, 0.3499999999999943, 1.8599999999999994, 7.140000000000001, 0.04999999999999716, 1.8799999999999955, 4.400000000000006, 1.4599999999999937, 3.2900000000000063, 17.560000000000002, 6.659999999999997, 8.409999999999997, 12.140000000000015, 1.9200000000000017, 8.650000000000006, 21.700000000000003, 2.6000000000000085, 24.28, 9.779999999999987, 4.060000000000002, 3.9099999999999966, 3.9599999999999937, 7.920000000000002, 17.11999999999999, 11.01000000000002, 8.700000000000003, 11.679999999999993, 7.639999999999986, 1.0600000000000023, 12.739999999999995, 13.959999999999994, 6.6200000000000045, 13.510000000000005, 4.38000000000001, 7.209999999999994, 2.2700000000000102, 2.6199999999999903, 11.539999999999992, 9.149999999999991, 14.780000000000001, 9.180000000000007, 14.789999999999992, 11.41000000000001, 12.25, 8.600000000000001, 5.13000000000001, 13.970000000000013, 9.840000000000003, 4.019999999999996, 12.359999999999985, 5.940000000000012, 4.030000000000001, 4.859999999999999, 0.10000000000000853, 1.7399999999999949, 13.710000000000008, 13.019999999999996, 4.659999999999997, 23.139999999999986, 4.150000000000006, 13.230000000000004, 0.8299999999999983, 3.5800000000000125, 4.200000000000003, 4.820000000000007, 6.579999999999998, 2.8200000000000074, 5.339999999999989, 5.240000000000009, 14.340000000000003, 5.8700000000000045, 9.070000000000007, 14.599999999999994, 1.5400000000000063, 8.47999999999999, 6.02000000000001, 9.139999999999986, 5.9199999999999875, 25.129999999999995, 6.25, 12.090000000000003, 0.980000000000004, 15.5, 6.740000000000009, 13.47999999999999, 3.4599999999999795, 1.8100000000000023, 4.519999999999996, 18.47, 3.589999999999989, 1.5700000000000074, 6.0, 3.8200000000000074, 3.109999999999985, 7.740000000000009, 4.460000000000008, 0.4099999999999966, 0.25, 6.730000000000004, 19.03, 8.52000000000001, 1.1099999999999852, 2.4099999999999966, 5.340000000000003, 21.599999999999994, 6.739999999999995, 5.590000000000003, 24.039999999999992, 11.719999999999999, 17.25999999999999, 2.3799999999999955, 15.090000000000003, 1.5200000000000102, 8.170000000000002, 8.390000000000015, 2.519999999999996, 14.120000000000005, 8.379999999999995, 3.530000000000001, 1.279999999999987, 7.440000000000012, 2.660000000000011, 7.760000000000005, 10.060000000000002, 32.510000000000005, 6.959999999999994, 0.5299999999999869, 16.539999999999992, 7.200000000000003, 12.299999999999997, 3.200000000000003, 5.640000000000001, 7.079999999999984, 5.6200000000000045, 16.710000000000008, 7.720000000000013, 12.799999999999997, 5.140000000000001, 7.929999999999993, 4.099999999999994, 5.079999999999998, 0.9500000000000028, 4.699999999999989, 2.660000000000011, 17.140000000000015, 18.809999999999988, 0.6500000000000057, 6.590000000000003, 13.319999999999993, 4.560000000000002, 10.219999999999999, 19.179999999999993, 4.359999999999999, 1.1599999999999966, 9.680000000000007, 30.129999999999995, 7.079999999999984, 11.11999999999999, 10.700000000000003, 6.329999999999998, 4.239999999999981, 37.44, 4.060000000000002, 1.6999999999999886, 14.240000000000009, 26.549999999999997, 0.1799999999999926, 4.679999999999993, 9.690000000000012, 4.359999999999999, 0.8200000000000003, 3.6599999999999966, 11.219999999999999, 4.240000000000009, 8.61999999999999, 16.779999999999987, 2.5799999999999983, 3.339999999999989, 3.219999999999999, 1.2099999999999937, 3.9799999999999898, 13.759999999999991, 1.7399999999999949, 28.200000000000003, 8.849999999999994, 1.2299999999999898, 6.3500000000000085, 8.019999999999996, 4.5, 3.4599999999999937, 8.260000000000005, 13.679999999999993, 3.299999999999997, 10.079999999999998, 11.02000000000001, 0.030000000000001137, 18.940000000000012, 4.189999999999998, 11.780000000000001, 6.420000000000016, 0.23999999999999488, 4.5, 1.6599999999999966, 3.9000000000000057, 3.269999999999996, 13.120000000000005, 4.660000000000011, 4.050000000000011, 3.8899999999999864, 12.159999999999997, 6.719999999999999, 16.059999999999988, 6.359999999999999, 14.209999999999994, 13.399999999999991, 1.2000000000000028, 2.529999999999987, 9.0, 1.3199999999999932, 6.039999999999992, 5.439999999999998, 3.480000000000004, 5.340000000000003, 0.3199999999999932, 1.3599999999999852, 4.240000000000009, 6.420000000000016, 10.280000000000001, 0.7000000000000028, 0.3999999999999915, 13.870000000000005, 1.7800000000000011, 1.559999999999988, 3.499999999999986, 6.11999999999999, 15.700000000000003, 6.359999999999999, 10.86, 10.929999999999993, 14.5, 1.420000000000016, 1.7399999999999949, 9.309999999999988, 2.1799999999999926, 5.38000000000001, 23.929999999999993, 1.8400000000000034, 18.060000000000002, 5.439999999999998, 5.329999999999998, 0.21999999999999886, 1.5799999999999983, 0.2400000000000091, 4.429999999999993, 0.060000000000002274, 9.900000000000006, 23.03, 3.559999999999995, 7.339999999999989, 10.579999999999998, 2.1500000000000057, 5.929999999999993, 12.290000000000006, 4.739999999999995, 11.379999999999995, 7.319999999999993, 8.690000000000012, 6.5, 9.070000000000007, 5.8799999999999955, 0.6599999999999966, 6.599999999999994, 1.9399999999999977, 3.0700000000000074, 5.459999999999994, 12.319999999999993, 9.760000000000005, 6.759999999999984, 26.0, 8.929999999999993, 2.3400000000000034, 1.1200000000000045, 11.040000000000006, 0.8599999999999994, 4.760000000000005, 10.16000000000001, 25.00999999999999, 1.6099999999999994, 3.969999999999999, 4.3799999999999955, 20.299999999999997, 1.3799999999999955, 2.260000000000005, 2.5999999999999943, 2.950000000000003, 5.059999999999988, 1.2199999999999847, 1.8900000000000006, 3.0799999999999983, 3.8799999999999955, 1.0300000000000011, 2.359999999999985, 6.420000000000016, 5.640000000000001, 0.6600000000000108, 0.29999999999999716, 9.629999999999995, 7.049999999999997, 2.299999999999997, 5.769999999999996, 1.0400000000000063, 10.799999999999997, 15.120000000000005, 0.3200000000000074, 2.440000000000012, 2.200000000000003, 6.3500000000000085, 8.599999999999994, 19.099999999999994, 3.539999999999992, 1.6199999999999903, 1.0799999999999983, 0.28999999999999204, 2.719999999999999, 1.970000000000013, 7.340000000000003, 13.86, 0.7199999999999989, 3.6200000000000045, 25.840000000000003, 8.529999999999987, 0.0, 3.5, 2.230000000000004, 2.5200000000000102, 14.189999999999998, 2.0800000000000125, 3.299999999999997, 14.219999999999999, 3.279999999999987, 3.4599999999999937, 2.539999999999992, 0.4099999999999966, 0.6800000000000068, 1.4399999999999977, 7.660000000000011, 6.9399999999999835, 4.780000000000001, 10.419999999999987, 0.980000000000004, 9.890000000000015, 7.3700000000000045, 0.7000000000000028, 11.539999999999992, 10.360000000000014, 19.859999999999985, 8.61999999999999, 5.3700000000000045, 8.929999999999993, 6.740000000000009, 9.170000000000002, 3.239999999999995, 1.960000000000008, 7.010000000000005, 5.439999999999998, 4.490000000000009, 4.0, 2.5799999999999983, 0.6999999999999886, 5.740000000000009, 9.320000000000007, 3.2299999999999898, 1.1999999999999957, 7.340000000000003, 4.280000000000001, 1.079999999999984, 3.3200000000000074, 8.940000000000005, 12.179999999999993, 6.859999999999999, 1.0200000000000102, 11.529999999999994, 1.5399999999999991, 1.0999999999999943, 20.659999999999997, 4.840000000000003, 17.080000000000013, 11.86, 3.1799999999999926, 4.420000000000002, 1.2700000000000102, 1.7299999999999898, 7.519999999999996, 10.799999999999997, 2.299999999999997, 0.0799999999999983, 17.10000000000001, 2.8400000000000176, 0.8400000000000034, 5.660000000000011, 6.420000000000016, 30.74000000000001, 10.179999999999993, 7.320000000000007, 21.860000000000014, 2.680000000000007, 3.0600000000000023, 8.560000000000002, 27.440000000000012, 27.24000000000001, 3.019999999999996, 1.240000000000009, 3.239999999999995, 3.6500000000000057, 9.649999999999991, 24.679999999999993, 6.040000000000006, 4.079999999999998, 7.3799999999999955, 10.659999999999997, 1.3599999999999994, 5.5, 0.03999999999999915, 0.7099999999999937, 12.100000000000001, 16.559999999999988, 7.720000000000013, 11.700000000000003, 11.620000000000005, 16.64, 5.679999999999993, 12.120000000000005, 7.36999999999999, 16.72, 0.29999999999999716, 3.259999999999991, 3.640000000000015, 4.660000000000004, 8.299999999999997, 0.3100000000000023, 11.760000000000005, 0.539999999999992, 11.620000000000005, 2.7700000000000102, 2.700000000000003, 1.1200000000000045, 17.569999999999993, 4.799999999999997, 6.219999999999999, 20.379999999999995, 4.460000000000008, 1.9200000000000017, 2.640000000000015, 4.440000000000012, 4.099999999999994, 0.9999999999999858, 8.689999999999998, 1.059999999999988, 4.909999999999997, 15.909999999999997, 0.7199999999999989, 2.1199999999999903, 6.909999999999997, 21.66000000000001, 3.75, 4.400000000000006, 0.8900000000000006, 3.1599999999999824, 4.459999999999994, 3.519999999999996, 17.0, 4.739999999999995, 12.319999999999993, 3.299999999999997, 6.159999999999997, 1.8800000000000097, 2.3400000000000034, 6.420000000000002, 4.310000000000002, 1.8299999999999983, 1.2600000000000051, 0.21999999999999886, 7.359999999999999, 8.420000000000002, 12.140000000000015, 10.030000000000001, 6.1000000000000085, 6.4199999999999875, 1.2299999999999898, 10.230000000000004, 4.070000000000007, 3.75, 32.2, 6.799999999999997, 5.22999999999999, 20.60000000000001, 11.570000000000007, 5.890000000000015, 3.0, 7.640000000000001, 10.940000000000012, 33.19000000000001, 3.6000000000000085, 7.549999999999997, 13.61, 14.700000000000003, 5.060000000000002, 4.519999999999996, 0.4199999999999875, 8.419999999999987, 7.0, 8.139999999999986, 7.200000000000003, 25.049999999999997, 16.5, 0.37000000000000455, 0.45000000000000284, 13.0, 11.540000000000006, 7.259999999999991, 0.5600000000000023, 3.8900000000000006, 8.680000000000007, 4.040000000000006, 8.829999999999998, 1.8199999999999932, 13.11, 0.5300000000000011, 0.480000000000004, 4.859999999999999, 0.1799999999999926, 3.799999999999997, 1.5999999999999943, 0.6799999999999926, 3.7199999999999847, 10.860000000000014, 22.280000000000015, 14.120000000000005, 4.38000000000001, 10.560000000000002, 3.569999999999993, 24.120000000000005, 17.019999999999996, 2.039999999999992, 18.799999999999997, 7.700000000000003, 6.6200000000000045, 2.759999999999991, 0.6600000000000108, 5.140000000000001, 8.019999999999996, 4.780000000000001, 3.9200000000000017, 7.760000000000005, 15.670000000000002, 11.909999999999997, 0.6599999999999966, 4.680000000000007, 1.1300000000000097, 2.8400000000000034, 5.679999999999993, 45.92999999999999, 8.61999999999999, 10.819999999999993, 16.89999999999999, 2.9399999999999835, 4.700000000000003, 0.7400000000000091, 4.959999999999994, 11.840000000000003, 8.27000000000001, 1.4000000000000057, 4.540000000000006, 5.929999999999993, 6.339999999999989, 39.370000000000005, 9.989999999999995, 7.280000000000001, 0.10000000000000853, 25.89, 1.0999999999999943, 9.679999999999993, 5.579999999999998, 0.8799999999999955, 8.599999999999994, 2.5600000000000023, 5.060000000000002, 5.180000000000007, 19.099999999999994, 9.439999999999998, 18.92, 3.4799999999999898, 5.329999999999998, 1.6200000000000045, 3.1799999999999926, 0.480000000000004, 7.300000000000011, 0.15000000000000568, 10.780000000000015, 9.800000000000011, 5.0, 16.170000000000016, 5.659999999999982, 4.8799999999999955, 8.299999999999997, 2.4000000000000057, 3.039999999999992, 3.670000000000016, 7.140000000000015, 11.879999999999995, 0.18000000000000682, 6.010000000000005, 7.52000000000001, 2.0999999999999943, 7.360000000000014, 18.859999999999985, 0.3100000000000023, 29.36, 4.019999999999996, 3.0600000000000023, 0.5600000000000023, 29.140000000000015, 7.760000000000005, 1.8399999999999892, 6.059999999999988, 2.0799999999999983, 9.14, 2.680000000000007, 14.079999999999998, 7.200000000000003, 30.08, 11.290000000000006, 7.710000000000008, 4.420000000000002, 0.980000000000004, 3.980000000000004, 6.6000000000000085, 8.0, 10.36, 10.469999999999999, 7.5, 4.099999999999994, 7.109999999999999, 17.900000000000006, 0.25, 4.840000000000003, 2.1400000000000006, 8.710000000000008, 20.11, 2.819999999999993, 13.909999999999997, 9.740000000000009, 1.4399999999999977, 19.11999999999999, 2.780000000000001, 6.360000000000014, 2.0700000000000074, 9.600000000000009, 14.400000000000006, 7.159999999999997, 6.819999999999993, 11.920000000000016, 7.799999999999997, 2.8599999999999994, 18.679999999999993, 6.8799999999999955, 13.699999999999989, 4.539999999999978, 0.1599999999999966, 1.9799999999999898, 17.559999999999988, 0.2600000000000193, 2.009999999999991, 10.079999999999998, 12.02000000000001, 1.180000000000021, 4.529999999999987, 1.2299999999999898, 4.989999999999995, 10.0, 12.420000000000002, 9.640000000000015, 16.24000000000001, 18.460000000000008, 5.259999999999991, 3.6299999999999955, 14.420000000000002, 14.259999999999991, 9.299999999999997, 5.47999999999999, 3.5, 15.760000000000005, 1.9200000000000017, 3.6599999999999966, 1.1799999999999926, 22.090000000000018, 1.519999999999996, 5.079999999999984, 5.02000000000001, 7.079999999999998, 12.739999999999995, 4.3799999999999955, 4.640000000000001, 2.3799999999999955, 6.469999999999999, 0.03999999999997783, 9.060000000000002, 0.519999999999996, 8.360000000000014, 1.3599999999999994, 2.4399999999999977, 18.840000000000003, 0.0, 4.089999999999989, 5.349999999999994, 4.5, 14.700000000000003, 8.459999999999994, 18.879999999999995, 5.320000000000007, 12.64, 1.1600000000000108, 3.9199999999999875, 0.009999999999990905, 5.8799999999999955, 12.389999999999986, 9.219999999999999, 11.400000000000006, 4.0, 4.419999999999995, 6.3999999999999915, 23.019999999999996, 13.969999999999999, 0.5999999999999943, 9.099999999999994, 6.530000000000001, 3.8000000000000114, 8.039999999999992, 2.3400000000000034, 8.400000000000006, 6.6000000000000085, 4.5, 9.280000000000001, 16.02000000000001, 5.959999999999994, 17.11, 14.97999999999999, 5.549999999999997, 3.930000000000007, 2.200000000000003, 13.919999999999987, 4.0800000000000125, 4.740000000000009, 32.38000000000001, 1.75, 5.02000000000001, 3.5, 2.980000000000004, 2.769999999999996, 4.820000000000007, 1.6200000000000045, 0.960000000000008, 5.400000000000006, 2.220000000000013, 7.060000000000002, 6.6200000000000045, 0.20000000000000284, 1.4199999999999875, 6.700000000000003, 3.299999999999997, 1.4200000000000017, 12.920000000000002, 20.620000000000005, 3.0799999999999983, 10.759999999999991, 11.920000000000002, 7.420000000000002, 6.140000000000001, 8.810000000000002, 7.170000000000002, 0.03999999999999204, 13.400000000000006, 1.1799999999999926, 1.6400000000000006, 16.86, 6.6200000000000045, 11.570000000000007, 6.959999999999994, 21.72999999999999, 0.6000000000000085, 9.200000000000003, 0.8400000000000176, 4.689999999999998, 8.030000000000015, 6.429999999999993, 0.3900000000000148, 28.260000000000005, 11.859999999999985, 7.700000000000003, 3.8799999999999955, 9.820000000000007, 13.199999999999989, 11.030000000000001, 9.679999999999993, 2.740000000000009, 49.21000000000001, 21.14, 12.879999999999995, 6.460000000000008, 7.560000000000002, 0.8599999999999994, 34.109999999999985, 7.259999999999991, 4.359999999999999, 10.61, 7.299999999999997, 3.6000000000000085, 2.299999999999997, 12.330000000000013, 3.9200000000000017, 41.69000000000001, 8.879999999999995, 6.440000000000012, 0.7300000000000182, 13.36, 7.3799999999999955, 6.359999999999985, 4.4599999999999795, 4.959999999999994, 10.22999999999999, 6.859999999999999, 4.640000000000001, 19.239999999999995, 6.420000000000002, 2.0200000000000102, 19.26000000000002, 0.03999999999999204, 8.47999999999999, 11.340000000000003, 2.3400000000000034, 19.200000000000003, 3.5799999999999983, 1.6200000000000045, 9.110000000000014, 1.7399999999999949, 16.480000000000004, 10.120000000000005, 11.480000000000004, 3.660000000000011, 0.8999999999999915, 2.980000000000004, 11.550000000000011, 11.89, 13.340000000000003, 12.25, 10.220000000000006, 0.0799999999999983, 5.720000000000013, 0.4399999999999977, 4.68, 31.879999999999995, 22.42, 19.059999999999988, 1.559999999999988, 8.579999999999998, 4.52000000000001, 9.480000000000004, 1.3900000000000006, 7.060000000000002, 8.819999999999993, 4.140000000000001, 7.47999999999999, 9.399999999999991, 7.299999999999997, 0.4099999999999966, 11.490000000000009, 8.489999999999995, 7.299999999999997, 6.329999999999998, 7.420000000000002, 4.140000000000001, 13.570000000000007, 8.950000000000003, 7.240000000000009, 2.5400000000000063, 7.159999999999997, 2.1599999999999966, 19.599999999999994, 43.250000000000014, 2.039999999999999, 15.900000000000006, 0.28000000000000114, 1.5899999999999963, 0.6000000000000085, 12.39, 9.450000000000003, 8.700000000000003, 16.03, 0.46999999999999886, 0.9199999999999875, 11.090000000000003, 9.0, 9.920000000000002, 15.420000000000002, 9.920000000000016, 10.47999999999999, 0.6300000000000097, 5.8799999999999955, 5.969999999999999, 11.200000000000003, 4.670000000000002, 11.189999999999998, 38.65999999999998, 3.519999999999996, 0.4200000000000017, 40.3, 18.14, 15.970000000000013, 7.799999999999997, 1.519999999999996, 14.279999999999994, 2.5799999999999983, 9.349999999999994, 3.9200000000000017, 6.820000000000007, 15.960000000000015, 3.200000000000003, 4.079999999999998, 4.939999999999998, 3.0900000000000034, 5.279999999999987, 3.980000000000004, 10.239999999999995, 21.709999999999994, 7.840000000000003, 18.36, 0.20999999999997954, 10.179999999999993, 4.139999999999986, 0.7000000000000028, 17.89999999999999, 18.279999999999987, 12.060000000000002, 6.839999999999996, 1.1600000000000108, 42.0, 5.780000000000001, 3.1199999999999903, 2.0999999999999943, 13.990000000000009, 0.7199999999999847, 4.960000000000008, 21.819999999999993, 8.14, 17.260000000000005, 3.5700000000000074, 0.20000000000000284, 9.280000000000001, 1.9599999999999937, 14.100000000000009, 2.8700000000000045, 34.510000000000005, 5.200000000000003, 6.689999999999998, 7.240000000000009, 9.659999999999997, 0.05999999999998806, 5.240000000000009, 24.88000000000001, 1.1299999999999955, 1.8999999999999915, 0.8400000000000034, 0.6599999999999824, 4.160000000000011, 15.439999999999998, 1.1699999999999875, 10.820000000000007, 1.7000000000000028, 2.5200000000000102, 2.680000000000007, 2.1200000000000045, 0.18999999999999773, 9.299999999999997, 5.579999999999998, 12.740000000000009, 21.320000000000007, 8.5, 9.719999999999999, 4.3500000000000085, 7.939999999999998, 12.090000000000003, 15.209999999999994, 7.719999999999999, 7.829999999999998, 15.900000000000006, 21.039999999999992, 11.340000000000003, 17.03, 10.049999999999997, 6.140000000000001, 13.409999999999997, 26.470000000000013, 0.7000000000000028, 6.97999999999999, 14.330000000000013, 0.9200000000000017, 12.440000000000012, 0.25, 1.440000000000012, 8.11999999999999, 14.959999999999994, 9.370000000000005, 0.8599999999999994, 0.9200000000000017, 20.08, 9.800000000000011, 2.839999999999989, 7.320000000000007, 9.099999999999994, 2.180000000000007, 4.8799999999999955, 7.540000000000006, 14.120000000000005, 9.649999999999991, 15.509999999999991, 2.5, 4.140000000000001, 5.220000000000013, 3.3600000000000136, 0.4399999999999977, 0.9200000000000017, 15.33999999999999, 13.109999999999985, 2.1400000000000006, 1.5399999999999778, 5.639999999999986, 1.240000000000009, 11.139999999999986, 4.810000000000002, 1.4799999999999898, 1.4799999999999898, 10.5, 6.5, 1.6000000000000085, 11.980000000000004, 4.159999999999997, 1.1799999999999926, 10.400000000000006, 9.780000000000001, 7.140000000000001, 9.480000000000004, 1.2399999999999949, 1.5600000000000023, 7.079999999999998, 5.97999999999999, 7.099999999999994, 3.549999999999997, 2.480000000000004, 17.989999999999995, 7.679999999999993, 6.460000000000008, 5.579999999999998, 8.179999999999993, 2.2099999999999937, 13.040000000000006, 12.189999999999998, 12.780000000000001, 4.359999999999999, 8.780000000000001, 13.540000000000006, 0.4099999999999824, 1.2999999999999972, 5.440000000000012, 5.939999999999998, 6.52000000000001, 12.100000000000009, 0.5499999999999972, 7.4199999999999875, 3.059999999999988, 6.420000000000002, 7.640000000000001, 19.25999999999999, 14.599999999999994, 1.7600000000000051, 2.8599999999999994, 18.040000000000006, 4.840000000000003, 8.310000000000016, 4.159999999999997, 6.819999999999993, 2.589999999999989, 13.420000000000016, 0.3299999999999983, 4.8500000000000085, 4.349999999999994, 3.240000000000009, 11.959999999999994, 3.0600000000000023, 15.439999999999998, 36.22000000000001, 3.640000000000015, 5.27000000000001, 1.5799999999999983, 2.480000000000018, 10.629999999999995, 6.349999999999994, 3.980000000000004, 13.299999999999997, 22.30000000000001, 4.700000000000003, 6.280000000000001, 5.189999999999998, 18.61, 3.6000000000000014, 4.409999999999997, 13.280000000000001, 4.260000000000005, 5.839999999999989, 0.8999999999999915, 18.200000000000003, 29.200000000000003, 2.8299999999999983, 4.859999999999985, 11.02000000000001, 10.009999999999991, 6.540000000000006, 7.560000000000002, 4.040000000000006, 10.659999999999997, 0.35999999999999943, 13.439999999999998, 1.420000000000016, 2.3999999999999915, 3.9000000000000057, 4.299999999999997, 0.8100000000000023, 1.5, 7.77000000000001, 7.960000000000022, 0.5100000000000051, 8.289999999999992, 4.140000000000001, 9.600000000000009, 23.480000000000004, 7.670000000000002, 10.320000000000007, 13.32, 2.1599999999999966, 5.999999999999986, 10.360000000000014, 8.64, 1.5999999999999943, 1.2399999999999949, 3.8899999999999864, 11.47999999999999, 12.689999999999998, 7.799999999999997, 5.460000000000008, 12.120000000000005, 6.519999999999996, 5.299999999999997, 22.099999999999994, 17.220000000000013, 1.5799999999999983, 4.889999999999986, 5.400000000000006, 9.16000000000001, 11.5, 8.060000000000002, 3.25, 4.0, 1.960000000000008, 43.84000000000002, 9.58999999999999, 16.840000000000003, 3.6799999999999926, 6.799999999999997, 9.620000000000005, 2.8999999999999915, 6.6200000000000045, 8.920000000000002, 3.3299999999999983, 0.6200000000000045, 1.0799999999999983, 6.47999999999999, 16.540000000000006, 4.409999999999997, 4.710000000000008, 1.9200000000000017, 14.070000000000007, 19.5, 0.9399999999999977, 0.8400000000000034, 2.690000000000012, 1.5999999999999943, 1.1800000000000068, 0.38999999999999346, 10.439999999999998, 4.279999999999987, 9.539999999999992, 19.78000000000001, 19.64, 4.640000000000001, 6.640000000000015, 2.509999999999991, 4.82, 3.5400000000000063, 6.180000000000007, 1.2900000000000063, 6.739999999999995, 6.510000000000019, 11.359999999999992, 3.5, 0.12000000000000455, 9.790000000000006, 4.690000000000012, 4.640000000000001, 9.060000000000002, 16.180000000000007, 3.500000000000014, 0.5799999999999983, 21.17, 0.05999999999998806, 21.950000000000003, 2.950000000000003, 5.8700000000000045, 9.469999999999999, 6.420000000000002, 10.20000000000001, 1.7800000000000011, 8.36, 10.330000000000013, 5.4599999999999795, 9.280000000000001, 0.0, 1.519999999999996, 7.760000000000005, 20.019999999999996, 5.579999999999998, 4.150000000000006, 22.08, 27.03, 3.140000000000015, 16.52000000000001, 9.819999999999979, 15.0, 0.6799999999999926, 21.439999999999998, 7.820000000000007, 0.47999999999998977, 11.920000000000016, 6.280000000000008, 3.8400000000000034, 0.9000000000000057, 7.549999999999997, 0.1599999999999966, 15.179999999999993, 8.75, 1.0799999999999983, 0.23999999999998067, 15.340000000000003, 3.1500000000000057, 11.579999999999998, 1.8700000000000045, 16.320000000000007, 13.390000000000015, 5.6200000000000045, 4.049999999999997, 1.0999999999999943, 7.040000000000006, 5.269999999999996, 10.819999999999993, 11.0, 0.13999999999998636, 5.5, 14.560000000000002, 2.8200000000000074, 3.4000000000000057, 0.35999999999999943, 0.7399999999999949, 7.220000000000013, 8.379999999999995, 28.08, 3.8400000000000034, 0.2400000000000091, 10.320000000000007, 7.920000000000002, 5.799999999999997, 2.3299999999999983, 5.3799999999999955, 1.3599999999999994, 0.5, 23.47, 10.009999999999991, 18.72, 3.510000000000005, 13.5, 0.020000000000010232, 0.7000000000000028, 0.37999999999999545, 0.47999999999998977, 9.789999999999992, 6.02000000000001, 0.7799999999999869, 2.0600000000000023, 3.0799999999999983, 0.1599999999999966, 21.150000000000006, 0.11999999999999034, 4.780000000000001, 0.1799999999999926, 7.240000000000009, 1.8599999999999852, 40.620000000000005, 14.190000000000012, 17.760000000000005, 13.719999999999999, 0.47999999999998977, 8.789999999999992, 7.329999999999998, 0.7200000000000131, 6.560000000000002, 8.990000000000009, 5.219999999999999, 1.7000000000000028, 7.769999999999996, 25.599999999999994, 5.820000000000007, 29.08, 0.2599999999999909, 24.42, 23.97999999999999, 15.479999999999997, 2.5600000000000023, 1.7999999999999972, 18.16000000000001, 11.399999999999991, 16.099999999999994, 2.799999999999997, 4.579999999999998, 0.25, 0.46999999999999886, 3.0200000000000102, 4.679999999999993, 11.189999999999998, 8.700000000000003, 5.670000000000002, 8.939999999999998, 5.079999999999998, 2.9099999999999966, 7.259999999999991, 1.5999999999999943, 8.519999999999996, 4.780000000000001, 7.349999999999994, 13.83999999999999, 5.950000000000003, 9.829999999999984, 38.040000000000006, 6.760000000000005, 29.040000000000006, 7.1000000000000085, 2.6200000000000045, 2.759999999999991, 6.890000000000001, 8.219999999999999, 2.4200000000000017, 7.459999999999994, 3.7700000000000102, 2.7299999999999898, 10.829999999999998, 27.36, 8.799999999999997, 9.039999999999992, 12.20999999999998, 9.52000000000001, 1.6000000000000014, 0.9499999999999886, 12.670000000000002, 12.360000000000014, 8.180000000000007, 13.259999999999991, 8.879999999999995, 11.439999999999998, 4.009999999999991, 8.61999999999999, 12.939999999999998, 0.7199999999999989, 11.940000000000012, 9.509999999999991, 5.490000000000009, 10.570000000000007, 6.960000000000008, 6.170000000000002, 4.469999999999999, 7.440000000000012, 6.77000000000001, 5.579999999999998, 16.340000000000003, 22.75999999999999, 5.579999999999998, 4.859999999999999, 6.200000000000003, 13.040000000000006, 4.700000000000003, 3.0800000000000125, 5.060000000000002, 3.039999999999992, 8.129999999999995, 19.42, 11.859999999999985, 2.4399999999999977, 14.170000000000002, 15.329999999999998, 4.459999999999994, 10.469999999999999, 4.900000000000006, 17.200000000000003, 5.639999999999986, 1.2999999999999972, 25.260000000000005, 3.200000000000003, 3.480000000000004, 10.159999999999997, 2.019999999999996, 3.280000000000001, 12.399999999999999, 10.079999999999998, 2.3200000000000074, 2.299999999999997, 0.6599999999999966, 1.5799999999999983, 6.859999999999985, 4.359999999999985, 13.83999999999999, 5.140000000000001, 8.340000000000018, 0.8999999999999915, 12.189999999999998, 18.16000000000001, 4.72999999999999, 4.27000000000001, 6.299999999999997, 8.459999999999994, 3.8599999999999994, 10.080000000000013, 4.269999999999996, 1.9100000000000108, 5.1000000000000085, 0.7399999999999949, 3.249999999999986, 4.579999999999998, 1.9300000000000068, 7.060000000000002, 3.450000000000003, 22.799999999999997, 2.5799999999999983, 2.0200000000000102, 4.02000000000001, 13.379999999999995, 0.7599999999999909, 0.46000000000000796, 0.28000000000000114, 4.420000000000002, 7.220000000000013, 6.019999999999996, 2.3799999999999955, 18.689999999999998, 1.8400000000000034, 0.07999999999998408, 9.649999999999991, 6.3999999999999915, 19.260000000000005, 0.4899999999999949, 3.200000000000003, 1.75, 21.120000000000005, 9.439999999999998, 1.2800000000000011, 16.960000000000008, 6.060000000000002, 30.260000000000005, 23.39999999999999, 11.629999999999995, 1.5399999999999778, 3.4399999999999977, 11.749999999999986, 4.4199999999999875, 14.299999999999997, 7.180000000000007, 2.760000000000005, 6.269999999999996, 3.1700000000000017, 6.0, 8.330000000000013, 3.9799999999999898, 2.4599999999999937, 0.9000000000000057, 5.280000000000001, 7.070000000000007, 2.8999999999999915, 9.300000000000011, 3.1000000000000085, 0.9399999999999977, 22.760000000000005, 11.279999999999987, 1.8799999999999955, 20.019999999999996, 1.4100000000000108, 1.2600000000000051, 4.3799999999999955, 14.689999999999998, 1.519999999999996, 11.01000000000002, 3.549999999999997, 21.320000000000007, 11.309999999999988, 17.920000000000016, 14.259999999999991, 5.329999999999984, 7.549999999999997, 7.099999999999994, 9.859999999999985, 6.47999999999999, 7.159999999999997, 8.539999999999992, 3.1599999999999966, 6.179999999999993, 13.97999999999999, 0.700000000000017, 10.179999999999993, 1.1400000000000006, 32.45999999999998, 1.079999999999984, 5.52000000000001, 29.24000000000001, 0.44999999999998863, 14.920000000000002, 6.670000000000016, 5.759999999999991, 0.37999999999999545, 2.5, 7.359999999999999, 5.980000000000004, 29.28, 2.240000000000009, 8.879999999999995, 9.280000000000001, 0.6200000000000045, 7.57, 1.3200000000000074, 2.5900000000000176, 7.310000000000002, 10.97999999999999, 10.259999999999991, 10.259999999999991, 2.329999999999984, 8.390000000000015, 7.379999999999988, 18.150000000000006, 7.559999999999988, 5.11999999999999, 9.960000000000008, 0.7800000000000011, 0.3999999999999915, 25.929999999999993, 23.760000000000005, 11.33999999999999, 0.28000000000000114, 4.72999999999999, 4.259999999999991, 8.11, 13.5, 3.3400000000000034, 3.289999999999992, 30.83, 5.800000000000011, 2.299999999999997, 21.000000000000014, 11.009999999999991, 4.820000000000007, 8.579999999999998, 1.220000000000013, 41.959999999999994, 13.049999999999997, 5.420000000000002, 4.420000000000002, 3.0200000000000102, 8.73999999999998, 2.8700000000000045, 19.02000000000001, 4.059999999999988, 2.8700000000000045, 5.710000000000008, 3.799999999999997, 2.8999999999999915, 1.1400000000000006, 9.909999999999997, 3.259999999999991, 13.579999999999998, 2.280000000000001, 3.4299999999999926, 1.420000000000016, 6.5, 1.3799999999999955, 6.289999999999992, 13.100000000000009, 10.0, 13.319999999999993, 8.02000000000001, 9.259999999999991, 3.8200000000000074, 3.239999999999995, 15.140000000000015, 1.7000000000000028, 6.6499999999999915, 8.179999999999993, 6.239999999999995, 0.30000000000001137, 3.4200000000000017, 0.20000000000000284, 8.519999999999996, 12.709999999999994, 6.689999999999998, 1.210000000000008, 21.14, 11.799999999999997, 0.9200000000000017, 7.390000000000001, 5.3799999999999955, 21.319999999999993, 47.53999999999999, 6.640000000000015, 6.3799999999999955, 3.219999999999999, 4.3799999999999955, 4.0, 0.30000000000001137, 9.0, 2.690000000000012, 4.1299999999999955, 6.810000000000002, 6.72999999999999, 6.179999999999993, 6.540000000000006, 2.5700000000000074, 3.1999999999999886, 5.670000000000002, 29.47999999999999, 5.360000000000014, 0.29999999999999716, 33.02000000000001, 3.940000000000012, 3.5799999999999983, 2.6199999999999903, 21.299999999999997, 2.3799999999999955, 6.6200000000000045, 23.08, 8.0, 6.530000000000001, 32.620000000000005, 3.8600000000000136, 6.980000000000004, 2.1299999999999955, 2.2399999999999807, 0.7099999999999937, 2.980000000000004, 26.539999999999992, 9.439999999999998, 5.8799999999999955, 3.660000000000011, 3.8200000000000074, 4.079999999999991, 8.239999999999995, 2.0, 0.3499999999999943, 5.6200000000000045, 0.45000000000000284, 2.1000000000000014, 1.1500000000000057, 4.859999999999985, 0.7399999999999949, 3.1999999999999957, 4.459999999999994, 12.86, 6.349999999999994, 3.760000000000005, 1.1799999999999926, 13.899999999999991, 15.620000000000005, 5.039999999999992, 12.400000000000006, 20.939999999999998, 16.309999999999988, 0.8199999999999932, 1.5499999999999972, 21.86, 5.0400000000000205, 5.6200000000000045, 4.439999999999998, 22.39999999999999, 2.700000000000003, 13.090000000000003, 5.159999999999997, 9.820000000000007, 0.29999999999999716, 8.040000000000006, 3.1599999999999966, 14.91000000000001, 9.299999999999997, 2.260000000000005, 1.1599999999999824, 10.500000000000014, 12.97999999999999, 22.659999999999997, 8.0, 7.0, 3.1500000000000057, 11.419999999999987, 12.820000000000007, 0.6099999999999994, 20.349999999999994, 2.829999999999984, 5.990000000000009, 13.599999999999994, 4.989999999999981, 5.1899999999999835, 5.359999999999999, 6.090000000000003, 2.0200000000000102, 2.940000000000012, 14.27000000000001, 7.289999999999992, 4.030000000000001, 9.659999999999997, 5.6299999999999955, 54.7, 4.820000000000007, 21.50999999999999, 14.030000000000001, 3.8200000000000003, 4.579999999999998, 14.200000000000003, 7.170000000000002, 5.480000000000004, 7.799999999999983, 4.920000000000002, 1.2999999999999972, 0.20999999999999375, 9.179999999999993, 0.7900000000000063, 4.86999999999999, 7.0, 21.319999999999993, 11.159999999999982, 4.429999999999993, 13.109999999999985, 2.3900000000000006, 4.320000000000007, 8.75, 3.8200000000000074, 3.0400000000000205, 8.299999999999997, 0.9399999999999835, 0.5400000000000205, 9.400000000000006, 1.3200000000000074, 11.899999999999991, 1.9799999999999898, 1.3199999999999932, 0.8499999999999943, 6.849999999999994, 1.5, 2.6299999999999955, 10.27000000000001, 1.519999999999996, 0.8799999999999955, 12.659999999999997, 26.92, 8.720000000000013, 2.3799999999999955, 1.7800000000000011, 9.929999999999993, 10.459999999999994, 1.2000000000000028, 4.859999999999999, 6.340000000000003, 2.039999999999978, 12.819999999999993, 12.97999999999999, 6.1000000000000085, 12.39, 10.25, 6.139999999999986, 5.6200000000000045, 1.5400000000000063, 2.9200000000000017, 25.74000000000001, 3.539999999999992, 2.5999999999999943, 4.760000000000005, 14.019999999999996, 6.6899999999999835, 13.870000000000005, 0.04000000000000625, 0.9000000000000057, 5.590000000000003, 25.22, 9.36, 0.7700000000000102, 15.02000000000001, 20.54000000000002, 9.799999999999997, 4.540000000000006, 4.660000000000004, 2.8399999999999963, 5.960000000000008, 0.5, 14.939999999999998, 1.539999999999992, 2.339999999999989, 5.340000000000003, 0.7799999999999869, 5.159999999999982, 21.860000000000014, 5.920000000000002, 5.0400000000000205, 1.2600000000000051, 10.47999999999999, 3.220000000000013, 5.519999999999996, 16.669999999999987, 8.579999999999998, 0.8100000000000165, 7.6200000000000045, 13.759999999999991, 14.400000000000006, 2.4599999999999937, 10.319999999999993, 1.9399999999999835, 15.450000000000003, 4.540000000000006, 15.590000000000003, 44.17999999999998, 1.9200000000000017, 9.439999999999998, 1.7199999999999989, 3.6400000000000006, 9.549999999999997, 0.6399999999999864, 3.4799999999999898, 13.259999999999991, 0.29999999999999716, 14.070000000000007, 0.4700000000000131, 8.019999999999996, 1.5999999999999943, 1.3100000000000023, 1.0799999999999983, 2.1000000000000085, 12.820000000000007, 3.539999999999992, 8.519999999999996, 34.97, 7.8500000000000085, 4.429999999999993, 17.14, 0.9800000000000182, 5.569999999999993, 3.4599999999999937, 13.840000000000003, 2.950000000000003, 7.700000000000003, 4.409999999999997, 0.4099999999999966, 0.46000000000000796, 1.2700000000000102, 1.0999999999999943, 3.299999999999997, 5.640000000000001, 2.9200000000000017, 2.769999999999996, 46.0, 1.240000000000009, 2.9399999999999977, 26.890000000000015, 2.3599999999999994, 7.500000000000014, 0.8200000000000074, 2.6799999999999926, 2.180000000000007, 7.36999999999999, 3.680000000000007, 36.000000000000014, 0.5, 2.1899999999999835, 12.5, 1.980000000000004, 5.060000000000002, 6.459999999999994, 9.25, 12.779999999999987, 3.9799999999999898, 7.5, 9.779999999999987, 22.64, 3.519999999999996, 1.2700000000000102, 1.7800000000000011, 4.140000000000001, 11.479999999999997, 1.9500000000000028, 1.2800000000000153, 1.1200000000000045, 1.2399999999999949, 14.730000000000004, 0.7600000000000051, 4.079999999999998, 7.320000000000007, 13.459999999999994, 4.650000000000006, 2.3999999999999915, 6.6200000000000045, 4.939999999999998, 1.420000000000016, 2.3599999999999994, 7.189999999999998, 5.6200000000000045, 13.530000000000001, 3.319999999999993, 18.700000000000003, 0.2599999999999909, 5.1299999999999955, 6.769999999999996, 11.579999999999998, 0.18999999999999773, 5.340000000000003, 5.599999999999994, 2.1400000000000006, 9.320000000000007, 2.6500000000000057, 1.6599999999999966, 0.09999999999999432, 2.2299999999999898, 22.42, 5.160000000000011, 7.780000000000001, 12.670000000000002, 5.700000000000003, 0.14000000000001478, 7.780000000000001, 12.840000000000003, 2.4599999999999937, 6.290000000000006, 0.03999999999999204, 14.700000000000003, 7.280000000000001, 1.0, 4.859999999999999, 16.299999999999997, 12.670000000000002, 15.550000000000011, 4.759999999999991, 7.25, 9.36, 2.819999999999993, 2.780000000000001, 9.280000000000001, 0.5200000000000102, 9.019999999999996, 8.740000000000009, 9.219999999999999, 18.0, 3.359999999999985, 1.8599999999999994, 3.8400000000000176, 16.92, 12.189999999999998, 2.200000000000003, 2.8200000000000074, 13.060000000000016, 10.459999999999994, 9.91000000000001, 9.239999999999995, 2.6599999999999966, 2.3799999999999955, 1.2999999999999972, 1.4200000000000017, 2.799999999999997, 18.16000000000001, 13.519999999999982, 4.61999999999999, 5.300000000000004, 1.7699999999999818, 5.019999999999996, 3.549999999999997, 9.069999999999993, 1.8400000000000034, 8.700000000000003, 23.92, 4.359999999999985, 7.230000000000004, 0.29999999999999716, 1.3999999999999915, 1.3500000000000085, 8.679999999999993, 9.640000000000015, 27.11, 6.320000000000007, 15.5, 2.9899999999999807, 15.47999999999999, 4.820000000000007, 2.299999999999997, 9.939999999999998, 12.059999999999988, 2.460000000000008, 1.4399999999999977, 5.0, 12.879999999999995, 19.5, 3.3599999999999994, 0.25, 14.400000000000006, 0.36000000000001364, 5.319999999999993, 0.7000000000000028, 36.22000000000001, 9.049999999999997, 4.159999999999997, 10.419999999999987, 5.299999999999997, 3.9299999999999926, 4.700000000000003, 24.67, 0.6000000000000085, 7.439999999999998, 4.02000000000001, 6.5400000000000205, 2.3999999999999915, 0.9999999999999858, 6.179999999999993, 2.680000000000007, 8.320000000000007, 3.3799999999999955, 1.3100000000000023, 0.9400000000000119, 21.570000000000007, 4.440000000000012, 4.36999999999999, 3.2299999999999898, 6.540000000000006, 1.8400000000000034, 6.02000000000001, 5.97999999999999, 9.5, 12.699999999999989, 9.240000000000009, 4.150000000000006, 3.719999999999999, 8.88000000000001, 4.959999999999994, 1.3599999999999852, 19.919999999999987, 0.2599999999999909, 7.320000000000007, 2.75, 11.840000000000018, 5.5, 0.07999999999998408, 3.6500000000000057, 25.66000000000001, 10.52000000000001, 1.6400000000000006, 28.679999999999993, 4.100000000000001, 10.799999999999997, 2.460000000000008, 8.450000000000003, 14.629999999999995, 7.480000000000004, 1.75, 2.260000000000005, 10.219999999999985, 3.980000000000011, 15.650000000000006, 5.760000000000005, 3.1400000000000006, 4.530000000000001, 2.469999999999999, 23.260000000000005, 0.6200000000000045, 3.4200000000000017, 19.36, 2.8799999999999955, 4.3799999999999955, 6.779999999999994, 11.429999999999993, 2.9199999999999875, 3.059999999999988, 8.299999999999997, 10.47999999999999, 25.599999999999994, 12.120000000000005, 5.940000000000012, 8.5, 2.6000000000000014, 3.450000000000003, 1.1499999999999915, 8.0, 5.490000000000009, 16.64, 7.700000000000003, 4.200000000000003, 20.679999999999993, 5.8700000000000045, 4.099999999999994, 12.920000000000002, 7.9199999999999875, 0.6600000000000108, 37.31999999999999, 6.280000000000001, 2.0400000000000063, 4.359999999999999, 8.900000000000006, 6.819999999999993, 7.3999999999999915, 1.539999999999992, 0.11999999999999034, 4.310000000000002, 0.20000000000000284, 12.840000000000003, 1.779999999999987, 3.3400000000000034, 16.120000000000005, 19.14, 15.799999999999997, 10.779999999999987, 0.10000000000000853, 1.9100000000000108, 11.200000000000003, 22.5, 2.7900000000000063, 4.769999999999996, 1.8499999999999943, 5.839999999999989, 1.3100000000000023, 3.0, 8.070000000000007, 18.930000000000007, 4.47999999999999, 5.3999999999999915, 22.139999999999986, 6.97999999999999, 9.100000000000009, 10.120000000000005, 7.359999999999985, 3.7700000000000102, 1.2199999999999989, 5.960000000000008, 8.200000000000003, 20.27000000000001, 3.5999999999999943, 10.819999999999993, 3.039999999999992, 7.359999999999999, 2.4599999999999937, 1.5499999999999972, 4.600000000000001, 4.009999999999991, 1.7800000000000011, 5.799999999999997, 0.6899999999999977, 3.7800000000000082, 2.6400000000000006, 4.159999999999982, 5.900000000000006, 8.560000000000002, 23.210000000000008, 2.140000000000015, 4.019999999999996, 9.739999999999995, 1.2199999999999989, 6.220000000000013, 18.099999999999994, 2.6000000000000085, 23.559999999999988, 5.5, 21.28, 0.6399999999999864, 6.420000000000002, 0.0800000000000125, 5.6200000000000045, 1.710000000000008, 42.58, 0.2600000000000051, 4.0, 8.909999999999997, 2.6200000000000045, 7.02000000000001, 8.019999999999996, 2.200000000000003, 17.42, 2.25, 9.110000000000014, 11.989999999999995, 1.519999999999996, 1.2000000000000028, 15.120000000000005, 4.5800000000000125, 9.89, 10.920000000000002, 4.359999999999999, 2.6799999999999926, 19.549999999999997, 8.079999999999998, 4.219999999999999, 4.200000000000003, 2.1500000000000057, 1.4399999999999977, 6.140000000000015, 20.870000000000005, 2.469999999999999, 12.920000000000002, 9.359999999999985, 15.740000000000009, 6.189999999999998, 0.30999999999998806, 4.920000000000002, 9.960000000000008, 2.6200000000000045, 0.46000000000000796, 14.02000000000001, 6.599999999999994, 3.739999999999995, 2.0799999999999983, 1.6799999999999926, 10.819999999999993, 13.980000000000004, 5.420000000000002, 6.8700000000000045, 2.0, 3.1099999999999994, 0.5599999999999881, 3.1999999999999886, 0.20000000000000284, 8.479999999999997, 6.160000000000004, 3.8999999999999915, 3.3600000000000136, 1.0, 0.030000000000001137, 11.899999999999999, 4.159999999999997, 3.280000000000001, 20.230000000000004, 13.240000000000009, 0.3200000000000074, 3.930000000000007, 8.97999999999999, 33.959999999999994, 14.429999999999993, 0.20000000000000284, 5.659999999999997, 6.02000000000001, 1.1099999999999994, 4.659999999999997, 1.220000000000013, 29.08, 5.1000000000000085, 12.320000000000007, 6.059999999999988, 2.440000000000012, 2.019999999999996, 10.64, 12.519999999999996, 0.04000000000000625, 3.799999999999997, 0.8400000000000034, 1.1000000000000085, 9.560000000000002, 0.2799999999999869, 16.30000000000001, 7.679999999999993, 11.0, 9.899999999999999, 1.0799999999999983, 5.560000000000002, 4.579999999999998, 10.149999999999991, 7.640000000000015, 1.4699999999999989, 1.0799999999999983, 0.23999999999999488, 10.530000000000001, 1.8799999999999955, 8.160000000000025, 1.2600000000000051, 9.820000000000007, 7.02000000000001, 4.079999999999998, 1.539999999999992, 1.5300000000000011, 7.519999999999982, 4.209999999999994, 6.810000000000002, 2.780000000000001, 5.790000000000006, 14.150000000000006, 7.6000000000000085, 7.990000000000009, 2.559999999999988, 1.0800000000000267, 8.340000000000003, 7.239999999999995, 2.9000000000000057, 3.5, 4.159999999999997, 8.679999999999993, 14.540000000000006, 12.11999999999999, 10.36, 15.070000000000007, 8.879999999999995, 7.540000000000006, 9.490000000000009, 7.359999999999999, 7.160000000000011, 27.02000000000001, 2.660000000000025, 0.45999999999999375, 6.319999999999979, 7.470000000000013, 6.4199999999999875, 11.220000000000013, 0.7399999999999949, 3.3999999999999915, 5.430000000000007, 9.860000000000014, 4.280000000000015, 4.429999999999993, 22.22, 4.63000000000001, 4.720000000000013, 2.690000000000012, 14.790000000000006, 5.960000000000008, 0.060000000000002274, 0.12000000000000455, 14.36, 7.1200000000000045, 4.1299999999999955, 13.579999999999998, 0.9399999999999977, 19.42, 0.539999999999992, 13.599999999999994, 5.060000000000002, 4.940000000000012, 4.760000000000005, 1.8900000000000148, 10.86, 10.480000000000004, 5.579999999999998, 5.559999999999988, 7.3799999999999955, 0.28000000000000114, 9.27000000000001, 2.6200000000000045, 7.88000000000001, 1.7999999999999972, 0.4899999999999949, 16.080000000000013, 16.799999999999997, 4.570000000000007, 7.359999999999985, 5.61999999999999, 1.960000000000008, 2.1500000000000057, 1.3400000000000034, 10.199999999999989, 1.559999999999988, 0.5299999999999869, 9.179999999999993, 2.780000000000001, 27.650000000000006, 22.840000000000003, 5.779999999999994, 3.0200000000000102, 2.6400000000000006, 11.629999999999995, 0.5999999999999943, 3.1599999999999966, 2.4299999999999926, 10.400000000000006, 14.549999999999997, 0.8799999999999955, 13.740000000000009, 2.500000000000014, 3.9399999999999977, 10.0, 8.440000000000012, 5.3799999999999955, 20.480000000000004, 0.8799999999999955, 8.859999999999985, 6.200000000000003, 3.019999999999996, 17.22, 0.20000000000000284, 0.6400000000000006, 3.4399999999999977, 3.5200000000000102, 0.9200000000000159, 2.450000000000003, 6.060000000000002, 3.6399999999999935, 7.1000000000000085, 5.6000000000000085, 8.64, 6.780000000000001, 1.210000000000008, 7.859999999999999, 4.0, 0.7399999999999949, 27.42, 5.900000000000006, 3.4399999999999835, 2.719999999999999, 1.5, 9.820000000000007, 0.6399999999999864, 3.719999999999999, 8.909999999999997, 2.8200000000000074, 11.039999999999992, 3.2099999999999937, 3.779999999999987, 4.359999999999985, 8.409999999999997, 2.039999999999992, 16.930000000000007, 4.27000000000001, 3.5999999999999943, 9.659999999999997, 0.6199999999999903, 14.719999999999999, 9.77000000000001, 11.570000000000007, 4.759999999999991, 4.539999999999992, 22.840000000000003, 4.490000000000009, 5.060000000000002, 3.280000000000001, 3.9200000000000017, 3.210000000000008, 9.840000000000003, 9.849999999999994, 2.1400000000000006, 9.420000000000002, 3.240000000000009, 24.150000000000006, 6.189999999999998, 0.14000000000000057, 24.679999999999993, 6.290000000000006, 9.039999999999992, 10.420000000000002, 11.280000000000001, 9.320000000000007, 3.6200000000000045, 50.78, 8.86, 2.3999999999999915, 19.859999999999985, 6.409999999999997, 0.0799999999999983, 0.030000000000001137, 5.939999999999998, 4.140000000000001, 2.200000000000003, 3.039999999999992, 1.0799999999999983, 9.240000000000009, 3.530000000000001, 12.809999999999988, 26.760000000000005, 2.6599999999999824, 35.2, 22.200000000000017, 1.4599999999999937, 4.599999999999994, 2.9200000000000017, 2.0600000000000023, 6.340000000000003, 3.1399999999999935, 8.530000000000001, 6.600000000000001, 20.650000000000006, 10.129999999999995, 2.059999999999988, 9.879999999999995, 0.1799999999999926, 1.5200000000000102, 1.2600000000000051, 14.739999999999995, 1.1599999999999966, 12.280000000000001, 16.269999999999996, 2.759999999999991, 5.6299999999999955, 0.6400000000000006, 3.3800000000000097, 8.940000000000012, 9.279999999999994, 5.460000000000008, 29.409999999999997, 21.560000000000002, 16.840000000000003, 24.459999999999994, 5.069999999999993, 16.97, 11.620000000000005, 2.3400000000000176, 10.240000000000009, 8.120000000000005, 2.549999999999997, 1.0100000000000051, 10.0, 4.740000000000009, 6.280000000000001, 8.439999999999998, 0.19999999999998863, 3.1799999999999926, 9.659999999999997, 5.480000000000004, 1.480000000000004, 1.2199999999999989, 31.58, 2.4199999999999875, 11.090000000000003, 7.719999999999999, 0.0799999999999983, 2.799999999999997, 18.099999999999994, 4.679999999999993, 1.5799999999999983, 1.3299999999999983, 13.679999999999993, 5.779999999999987, 8.819999999999993, 2.0799999999999983, 4.259999999999991, 1.759999999999991, 9.999999999999993, 53.42, 4.489999999999981, 15.439999999999998, 9.560000000000002, 24.88000000000001, 11.280000000000001, 1.0200000000000102, 4.720000000000013, 12.740000000000009, 1.9000000000000057, 8.320000000000007, 14.280000000000001, 3.1300000000000097, 14.599999999999994, 9.100000000000009, 6.11999999999999, 1.8599999999999852, 13.060000000000002, 1.2600000000000051, 9.480000000000018, 1.4200000000000017, 1.779999999999987, 3.0500000000000043, 8.379999999999995, 14.620000000000005, 14.070000000000007, 6.1200000000000045, 0.2599999999999909, 4.760000000000005, 2.200000000000003, 3.9399999999999835, 5.320000000000007, 1.8900000000000006, 3.6400000000000006, 11.64, 3.980000000000004, 1.7600000000000051, 5.5, 0.04000000000000625, 1.8100000000000023, 6.340000000000003, 21.679999999999993, 5.660000000000004, 2.759999999999991, 11.920000000000002, 25.549999999999997, 0.1600000000000108, 15.440000000000012, 7.280000000000001, 34.84, 0.9399999999999977, 1.190000000000012, 19.830000000000013, 5.8400000000000105, 30.790000000000006, 2.7099999999999937, 28.060000000000016, 14.000000000000007, 4.219999999999999, 7.079999999999998, 4.13000000000001, 10.61, 5.780000000000001, 6.47999999999999, 0.20000000000000284, 7.459999999999994, 20.14999999999999, 0.3400000000000176, 11.86, 1.0200000000000102, 8.800000000000011, 8.260000000000005, 6.8799999999999955, 11.419999999999987, 2.680000000000007, 8.299999999999997, 14.88000000000001, 4.760000000000019, 15.680000000000007, 5.9599999999999795, 1.8799999999999955, 19.620000000000005, 1.4000000000000057, 4.739999999999995, 0.8799999999999955, 5.840000000000003, 2.990000000000009, 15.840000000000003, 9.620000000000005, 3.6400000000000006, 5.47999999999999, 16.970000000000013, 6.359999999999999, 42.90999999999998, 9.120000000000005, 0.8599999999999994, 6.479999999999997, 14.820000000000007, 4.939999999999998, 2.3200000000000003, 11.990000000000009, 7.0, 6.97999999999999, 1.6800000000000068, 4.0, 0.7399999999999949, 3.6400000000000006, 9.25, 6.079999999999998, 14.620000000000005, 2.6200000000000045, 6.190000000000012, 5.5800000000000125, 0.3399999999999892, 5.909999999999997, 2.839999999999989, 3.3000000000000185, 6.109999999999999, 5.489999999999995, 1.5, 6.079999999999984, 11.179999999999993, 13.659999999999982, 4.640000000000001, 6.3700000000000045, 3.059999999999988, 10.20999999999998, 1.8200000000000074, 3.3299999999999983, 7.88000000000001, 10.52000000000001, 1.3200000000000074, 0.36000000000001364, 12.019999999999996, 33.48999999999998, 9.260000000000005, 4.519999999999996, 1.75, 4.359999999999999, 4.97999999999999, 3.280000000000001, 19.0, 1.5600000000000023, 15.219999999999999, 3.6299999999999955, 3.6400000000000006, 9.080000000000013, 8.179999999999993, 5.22999999999999, 1.0799999999999983, 5.820000000000007, 6.5, 3.8999999999999915, 14.779999999999987, 2.299999999999997, 4.819999999999993, 0.20000000000000284, 0.4399999999999977, 11.600000000000009, 22.28, 17.159999999999997, 7.940000000000012, 0.03999999999999204, 11.920000000000002, 10.090000000000003, 4.640000000000001, 6.97999999999999, 10.5, 1.3700000000000045, 6.019999999999996, 1.539999999999992, 2.9000000000000057, 1.279999999999987, 3.1400000000000006, 0.18000000000000682, 4.0800000000000125, 4.059999999999988, 5.070000000000007, 38.000000000000014, 5.340000000000003, 2.319999999999993, 7.090000000000003, 2.9599999999999937, 1.3200000000000074, 4.200000000000003, 17.02000000000001, 7.659999999999997, 16.47, 8.0, 9.649999999999991, 4.820000000000007, 3.1799999999999926, 4.009999999999991, 13.599999999999994, 0.14000000000001478, 8.420000000000002, 1.2800000000000153, 1.2800000000000011, 15.909999999999997, 1.5100000000000051, 6.1000000000000085, 1.8999999999999915, 10.379999999999995, 0.0799999999999983, 6.349999999999994, 1.1000000000000085, 0.5600000000000023, 0.39000000000000057, 7.839999999999989, 0.6700000000000017, 12.670000000000016, 3.5400000000000063, 5.4199999999999875, 5.800000000000011, 10.5, 24.97, 2.660000000000011, 7.239999999999995, 11.329999999999998, 3.780000000000001, 12.959999999999994, 19.510000000000005, 1.8000000000000114, 20.24000000000001, 2.1799999999999926, 7.340000000000003, 8.0, 13.360000000000014, 9.539999999999992, 2.240000000000009, 7.979999999999997, 11.180000000000007, 4.5400000000000205, 7.639999999999986, 1.6899999999999977, 6.460000000000008, 4.859999999999999, 8.47999999999999, 5.094527027027013, 6.079999999999998, 0.9200000000000017, 9.490000000000009, 10.080000000000013, 5.530000000000001, 1.6199999999999903, 8.490000000000009, 16.779999999999987, 3.6799999999999926, 7.6299999999999955, 12.379999999999995, 8.280000000000001, 15.679999999999993, 2.6400000000000006, 13.599999999999994, 5.819999999999993, 8.219999999999999, 34.0, 15.14, 10.010000000000005, 14.340000000000003, 1.3199999999999932, 4.159999999999997, 5.660000000000011, 10.170000000000002, 1.3599999999999994, 0.0, 7.560000000000002, 1.5799999999999983, 0.7000000000000028, 9.879999999999995, 12.540000000000006, 1.7399999999999949, 49.24000000000001, 30.539999999999992, 3.6400000000000006, 15.920000000000002, 2.680000000000007, 8.320000000000007, 2.2900000000000063, 0.44000000000001194, 4.5600000000000165, 17.02000000000001, 1.9399999999999977, 5.099999999999994, 2.5400000000000063, 11.019999999999982, 24.140000000000015, 16.24000000000001, 0.1799999999999926, 11.639999999999986, 27.120000000000005, 11.090000000000003, 6.560000000000002, 17.019999999999996, 3.700000000000003, 13.100000000000009, 10.189999999999998, 8.620000000000005, 1.5, 8.519999999999996, 0.8900000000000006, 33.50999999999999, 8.290000000000006, 3.200000000000003, 12.220000000000013, 24.61999999999999, 25.799999999999997, 4.259999999999991, 5.859999999999999, 6.490000000000009, 8.400000000000006, 5.97999999999999, 1.2800000000000011, 0.8200000000000074, 17.75999999999999, 1.8300000000000125, 2.680000000000007, 9.260000000000005, 2.700000000000003, 5.579999999999998, 21.22, 15.159999999999997, 15.059999999999988, 4.320000000000007, 7.540000000000006, 12.75, 3.239999999999995, 6.5, 6.349999999999994, 2.1699999999999875, 12.960000000000008, 11.64, 12.759999999999991, 8.169999999999987, 2.3200000000000074, 18.889999999999986, 11.529999999999987, 5.5, 2.8100000000000023, 4.8799999999999955, 9.640000000000015, 14.579999999999998, 7.1200000000000045, 8.140000000000015, 5.82, 2.4599999999999937, 6.159999999999997, 15.420000000000002, 6.599999999999994, 8.820000000000007, 18.939999999999998, 12.350000000000009, 7.280000000000001, 0.9500000000000028, 2.759999999999991, 3.5, 7.700000000000003, 6.640000000000001, 0.6799999999999926, 16.24000000000001, 0.6100000000000136, 16.909999999999997, 14.100000000000001, 2.1099999999999994, 6.8799999999999955, 2.839999999999989, 0.9399999999999977, 0.8799999999999955, 3.740000000000009, 2.140000000000015, 1.1200000000000045, 3.640000000000015, 3.4000000000000057, 3.3599999999999994, 10.11999999999999, 4.679999999999993, 10.359999999999985, 19.070000000000007, 2.1000000000000085, 0.730000000000004, 23.57000000000002, 9.980000000000004, 0.04999999999999716, 8.840000000000003, 5.059999999999988, 3.5400000000000063, 3.1299999999999955, 15.659999999999982, 12.799999999999997, 13.060000000000002, 3.980000000000004, 10.319999999999993, 17.27000000000001, 2.1200000000000045, 1.1200000000000045, 3.719999999999999, 3.5900000000000034, 2.549999999999997, 7.200000000000003, 1.4099999999999966, 5.3500000000000085, 5.349999999999994, 7.359999999999999, 11.319999999999993, 4.88000000000001, 6.299999999999997, 2.480000000000004, 2.9200000000000017, 5.439999999999998, 0.0799999999999983, 4.280000000000001, 3.6599999999999966, 10.400000000000006, 9.939999999999984, 1.0, 28.47999999999999, 1.8400000000000034, 20.210000000000008, 1.8100000000000023, 0.759999999999998, 6.320000000000007, 1.4000000000000057, 0.6399999999999864, 2.8000000000000114, 5.420000000000002, 11.519999999999996, 2.7299999999999898, 11.260000000000005, 2.5, 6.959999999999994, 5.549999999999997, 3.280000000000001, 2.259999999999991, 2.4299999999999926, 2.2600000000000193, 8.329999999999998, 2.3599999999999994, 11.119999999999997, 8.560000000000002, 2.8599999999999994, 0.09999999999999432, 14.620000000000005, 19.980000000000004, 10.939999999999984, 9.980000000000004, 8.599999999999994, 8.680000000000021, 12.219999999999999, 4.440000000000012, 0.4300000000000068, 17.0, 4.8799999999999955, 9.700000000000003, 0.8199999999999932, 3.4599999999999937, 2.4200000000000017, 4.359999999999999, 29.900000000000006, 4.460000000000008, 4.519999999999996, 8.159999999999997, 0.7400000000000091, 0.29999999999999716, 1.1799999999999926, 0.07000000000000739, 9.060000000000002, 0.5800000000000125, 6.719999999999999, 12.36, 3.769999999999996, 4.6200000000000045, 1.3599999999999852, 12.52000000000001, 7.920000000000002, 0.8400000000000176, 0.29999999999999716, 0.04000000000000625, 7.759999999999991, 0.75, 3.799999999999997, 17.76000000000002, 14.650000000000006, 0.45000000000000284, 26.26000000000002, 13.75, 7.299999999999997, 4.349999999999994, 32.28, 13.279999999999987, 0.1799999999999926, 22.140000000000015, 0.1799999999999926, 2.640000000000015, 2.1200000000000045, 2.759999999999991, 6.52000000000001, 12.899999999999991, 6.400000000000006, 5.679999999999993, 16.53, 2.210000000000008, 11.539999999999992, 0.37999999999999545, 4.6200000000000045, 11.939999999999998, 0.37999999999999545, 2.969999999999999, 6.689999999999998, 5.949999999999989, 2.740000000000009, 19.0, 3.3599999999999994, 2.239999999999995, 11.439999999999998, 2.6000000000000085, 17.89999999999999, 1.1199999999999903, 7.359999999999999, 12.33999999999999, 7.699999999999989, 0.09999999999999432, 4.579999999999998, 5.920000000000002, 9.679999999999993, 3.480000000000004, 7.299999999999997, 3.700000000000017, 8.219999999999999, 6.239999999999995, 4.260000000000005, 6.040000000000006, 3.259999999999991, 6.400000000000006, 0.3299999999999983, 11.989999999999995, 2.3599999999999994, 10.220000000000013, 6.6200000000000045, 21.900000000000006, 7.200000000000003, 14.279999999999987, 2.740000000000009, 7.780000000000001, 6.939999999999998, 9.170000000000002, 0.4200000000000017, 6.8700000000000045, 2.3000000000000114, 1.6400000000000148, 2.319999999999993, 3.4200000000000017, 22.60000000000001, 2.3500000000000085, 2.8900000000000006, 3.9000000000000057, 27.75999999999999, 21.74000000000001, 13.379999999999995, 10.709999999999994, 1.9000000000000057, 3.9000000000000057, 11.799999999999997, 4.559999999999988, 11.800000000000011, 16.560000000000002, 1.0700000000000074, 9.260000000000005, 13.960000000000008, 3.779999999999987, 15.590000000000003, 11.039999999999992, 3.0200000000000102, 6.560000000000002, 6.420000000000009, 7.219999999999999, 20.400000000000006, 6.200000000000003, 10.409999999999997, 9.25, 15.780000000000001, 2.1799999999999926, 3.1299999999999955, 6.240000000000009, 1.2600000000000051, 5.359999999999985, 13.679999999999993, 1.2000000000000028, 18.370000000000005, 5.959999999999994, 5.200000000000003, 7.02000000000001, 6.980000000000004, 18.159999999999982, 17.599999999999994, 3.1200000000000045, 16.459999999999994, 4.260000000000005, 1.789999999999992, 15.260000000000005, 7.5, 5.25, 9.079999999999998, 3.239999999999995, 9.360000000000014, 18.840000000000003, 9.379999999999995, 8.689999999999998, 10.780000000000001, 4.740000000000009, 11.879999999999995, 5.469999999999999, 8.140000000000015, 3.4799999999999898, 4.340000000000003, 0.1799999999999926, 2.5400000000000063, 4.139999999999986, 22.43000000000002, 9.620000000000005, 18.679999999999993, 4.599999999999994, 6.459999999999994, 15.810000000000002, 4.319999999999993, 5.359999999999999, 1.5699999999999932, 1.7399999999999949, 18.439999999999998, 17.31999999999998, 14.699999999999989, 7.52000000000001, 4.640000000000001, 8.700000000000003, 2.25, 28.340000000000018, 0.18000000000000682, 20.809999999999988, 16.47, 21.25, 9.700000000000003, 1.720000000000006, 3.720000000000013, 6.6000000000000085, 19.669999999999987, 1.1400000000000006, 0.7199999999999989, 1.3900000000000148, 17.86, 2.1400000000000006, 7.8700000000000045, 1.259999999999991, 4.200000000000003, 24.140000000000015, 0.37999999999999545, 3.239999999999995, 15.219999999999999, 5.200000000000003, 1.9000000000000057, 4.689999999999998, 0.07000000000000739, 20.64, 21.179999999999993, 3.019999999999996, 0.9300000000000068, 6.789999999999992, 0.7800000000000011, 7.359999999999985, 12.140000000000015, 3.719999999999999, 7.900000000000006, 10.430000000000007, 4.109999999999999, 2.1399999999999864, 16.30000000000001, 18.950000000000003, 14.350000000000009, 7.980000000000004, 9.079999999999998, 4.880000000000003, 20.939999999999998, 2.5800000000000125, 2.1400000000000006, 0.47999999999998977, 30.040000000000006, 2.8999999999999915, 10.400000000000006, 4.040000000000006, 19.02000000000001, 7.3799999999999955, 3.8400000000000034, 0.6799999999999926, 5.189999999999998, 4.340000000000003, 7.659999999999997, 0.04000000000000625, 1.0100000000000051, 2.259999999999991, 2.5600000000000023, 8.539999999999992, 9.120000000000005, 3.819999999999993, 49.500000000000014, 0.1799999999999926, 4.060000000000002, 1.8600000000000136, 9.680000000000007, 1.1800000000000068, 21.159999999999997, 3.780000000000001, 9.900000000000006, 3.6000000000000085, 8.680000000000007, 13.799999999999997, 5.400000000000006, 9.629999999999995, 18.280000000000015, 6.219999999999985, 8.409999999999997, 0.4200000000000017, 9.579999999999998, 1.9599999999999937, 0.3499999999999943, 20.22, 16.859999999999985, 22.17, 0.29000000000002046, 2.239999999999995, 4.340000000000003, 15.189999999999998, 3.1400000000000006, 9.439999999999998, 0.4999999999999858, 0.5800000000000125, 0.009999999999990905, 0.12999999999999545, 7.559999999999988, 8.799999999999997, 5.890000000000015, 0.4399999999999977, 7.339999999999989, 19.879999999999995, 10.97999999999999, 11.669999999999987, 10.739999999999995, 1.0200000000000102, 5.740000000000009, 12.759999999999991, 0.9199999999999875, 0.6400000000000148, 5.440000000000012, 10.420000000000002, 5.090000000000003, 24.64, 4.780000000000001, 0.8799999999999955, 7.639999999999986, 5.539999999999992, 5.189999999999998, 9.829999999999998, 5.640000000000015, 12.5, 0.2400000000000091, 8.730000000000004, 17.080000000000013, 9.14, 6.480000000000004, 6.3799999999999955, 24.929999999999993, 3.0999999999999943, 1.6400000000000006, 15.280000000000001, 1.1599999999999966, 4.929999999999993, 9.439999999999998, 1.0600000000000023, 11.659999999999982, 1.4100000000000108, 19.440000000000012, 11.72999999999999, 12.420000000000002, 8.340000000000003, 2.200000000000003, 0.0799999999999983, 17.50999999999999, 5.1200000000000045, 1.7000000000000028, 3.960000000000008, 10.320000000000007, 0.8200000000000074, 4.440000000000012, 3.539999999999992, 0.45999999999999375, 17.669999999999987, 4.819999999999993, 0.8599999999999994, 13.620000000000005, 13.799999999999997, 1.7199999999999989, 8.599999999999994, 17.200000000000003, 3.200000000000003, 3.780000000000001, 11.45999999999998, 11.760000000000005, 5.560000000000002, 0.9399999999999977, 8.659999999999997, 61.08000000000001, 1.8599999999999994, 5.150000000000006, 0.45999999999999375, 0.6400000000000148, 2.3599999999999994, 0.7599999999999909, 8.989999999999995, 6.749999999999986, 1.8799999999999955, 2.950000000000003, 3.1500000000000057, 4.340000000000003, 10.47999999999999, 14.510000000000005, 10.200000000000003, 0.7400000000000091, 19.179999999999993, 11.439999999999998, 5.989999999999995, 7.02000000000001, 1.5399999999999778, 7.949999999999989, 5.840000000000003, 3.0, 1.960000000000008, 5.719999999999999, 1.559999999999988, 7.220000000000013, 2.280000000000001, 11.700000000000003, 11.540000000000006, 6.700000000000003, 9.100000000000009, 10.560000000000002, 17.109999999999985, 4.9199999999999875, 21.760000000000005, 1.5, 12.579999999999998, 17.470000000000013, 8.77000000000001, 10.460000000000022, 1.3100000000000023, 3.4200000000000017, 1.8200000000000074, 2.980000000000004, 6.340000000000003, 13.579999999999998, 6.199999999999989, 9.730000000000004, 3.6799999999999926, 3.780000000000001, 3.5700000000000074, 3.6599999999999966, 4.090000000000018, 15.820000000000007, 10.190000000000012, 0.7599999999999909, 1.4899999999999949, 11.519999999999996, 9.33999999999999, 1.5, 3.989999999999995, 3.140000000000015, 1.2199999999999989, 4.599999999999994, 7.579999999999998, 0.35999999999999943, 1.9000000000000057, 13.359999999999985, 11.539999999999992, 0.519999999999996, 16.25, 8.420000000000002, 9.230000000000004, 7.0, 10.260000000000005, 11.759999999999991, 0.7799999999999869, 9.11, 12.409999999999997, 9.420000000000002, 10.620000000000005, 12.740000000000009, 2.0200000000000102, 39.72999999999999, 6.259999999999991, 3.299999999999997, 3.210000000000008, 3.260000000000005, 4.359999999999999, 1.2199999999999989, 5.190000000000012, 6.219999999999999, 2.719999999999999, 9.699999999999989, 6.680000000000007, 2.0799999999999983, 11.280000000000001, 5.060000000000002, 3.3799999999999955, 7.820000000000007, 0.5, 4.140000000000015, 19.64, 15.420000000000016, 9.340000000000003, 6.800000000000004, 2.859999999999985, 11.059999999999988, 7.199999999999996, 18.590000000000018, 4.599999999999994, 0.8700000000000045, 5.890000000000015, 12.940000000000012, 4.5400000000000205, 2.359999999999985, 11.320000000000007, 13.66000000000001, 9.16000000000001, 6.3799999999999955, 0.8200000000000074, 32.040000000000006, 4.09999999999998, 8.620000000000005, 1.519999999999996, 3.4200000000000017, 0.9500000000000028, 1.0400000000000063, 2.739999999999995, 47.05, 3.859999999999985, 13.620000000000005, 11.029999999999987, 3.700000000000003, 17.120000000000005, 1.3599999999999994, 3.200000000000003, 1.2099999999999937, 21.409999999999997, 4.8799999999999955, 0.7900000000000063, 3.8799999999999955, 8.559999999999988, 0.9400000000000119, 24.939999999999998, 5.260000000000005, 0.44000000000001194, 3.3200000000000074, 8.259999999999991, 5.960000000000008, 13.600000000000009, 7.699999999999989, 3.380000000000024, 12.220000000000013, 4.260000000000005, 2.4200000000000017, 3.359999999999985, 15.16000000000001, 3.339999999999989, 14.290000000000006, 1.2399999999999949, 0.01999999999998181, 16.279999999999987, 4.0, 8.200000000000003, 4.240000000000009, 24.61999999999999, 14.220000000000013, 13.120000000000005, 20.14999999999999, 1.9399999999999977, 1.2600000000000051, 4.020000000000003, 1.1999999999999886, 11.200000000000017, 9.879999999999995, 0.7000000000000028, 3.8400000000000034, 20.14999999999999, 29.72999999999999, 6.420000000000002, 2.8799999999999955, 4.960000000000008, 3.4599999999999937, 11.579999999999998, 2.5800000000000125, 4.299999999999997, 0.5900000000000034, 2.8700000000000045, 2.6599999999999966, 1.2600000000000051, 0.14000000000001478, 18.939999999999998, 29.42, 4.939999999999998, 32.67999999999999, 7.1000000000000085, 5.970000000000006, 2.8900000000000006, 0.6700000000000017, 24.54000000000002, 13.960000000000008, 6.740000000000009, 7.219999999999999, 1.0, 0.7800000000000011, 0.5, 6.180000000000007, 0.5600000000000023, 3.8200000000000074, 0.29999999999999716, 4.180000000000007, 9.740000000000009, 2.940000000000012, 9.699999999999989, 2.4599999999999937, 7.939999999999998, 13.299999999999997, 0.8200000000000074, 1.5900000000000034, 5.860000000000014, 7.820000000000007, 6.599999999999994, 19.039999999999992, 0.3499999999999943, 7.299999999999997, 9.61999999999999, 17.680000000000007, 1.6899999999999977, 6.779999999999987, 22.789999999999992, 4.97999999999999, 9.879999999999995, 2.5600000000000023, 19.359999999999985, 11.900000000000006, 3.299999999999997, 2.1400000000000006, 2.9200000000000017, 4.820000000000007, 10.97999999999999, 5.5800000000000125, 3.799999999999997, 2.0, 4.219999999999999, 5.1200000000000045, 3.1200000000000045, 7.290000000000006, 4.47999999999999, 39.72999999999999, 0.5, 8.569999999999993, 10.659999999999982, 1.5799999999999983, 14.560000000000002, 3.200000000000003, 11.819999999999993, 11.319999999999993, 5.079999999999998, 3.680000000000007, 0.45000000000000284, 19.679999999999993, 0.9200000000000017, 6.779999999999987, 7.180000000000021, 10.240000000000009, 6.979999999999997, 2.3799999999999955, 11.740000000000009, 11.549999999999997, 4.159999999999997, 7.040000000000006, 6.170000000000002, 6.529999999999987, 17.190000000000012, 0.35999999999999943, 1.3100000000000023, 1.480000000000004, 9.02000000000001, 6.0, 0.5800000000000125, 5.299999999999997, 1.960000000000008, 0.21999999999999886, 13.640000000000015, 6.239999999999995, 2.0600000000000023, 12.689999999999984, 4.570000000000007, 16.25999999999999, 20.909999999999997, 4.159999999999997, 4.920000000000002, 9.86, 5.4199999999999875, 7.920000000000002, 6.02000000000001, 7.3799999999999955, 5.150000000000006, 10.950000000000003, 10.420000000000002, 7.780000000000001, 16.159999999999997, 0.519999999999996, 14.579999999999998, 7.720000000000013, 12.38000000000001, 3.9000000000000057, 1.0, 0.980000000000004, 15.070000000000007, 20.680000000000007, 0.9399999999999977, 7.3799999999999955, 0.5899999999999892, 13.759999999999991, 9.47999999999999, 5.22999999999999, 4.779999999999987, 1.4000000000000057, 13.559999999999988, 14.299999999999997, 4.740000000000009, 3.039999999999992, 11.519999999999996, 7.1200000000000045, 1.3799999999999955, 2.200000000000003, 2.960000000000008, 7.400000000000006, 5.269999999999996, 2.0, 3.6200000000000045, 11.200000000000003, 12.340000000000003, 0.10999999999999943, 9.700000000000003, 8.52000000000001, 12.280000000000001, 4.3799999999999955, 1.9200000000000017, 2.460000000000008, 15.599999999999994, 10.149999999999991, 10.799999999999997, 5.1200000000000045, 6.88000000000001, 34.97, 10.179999999999993, 2.1799999999999926, 2.4599999999999937, 13.579999999999998, 11.719999999999999, 6.519999999999996, 5.840000000000003, 7.460000000000008, 6.460000000000008, 10.350000000000009, 1.9199999999999733, 20.75, 2.6200000000000045, 3.440000000000012, 4.0, 14.929999999999993, 3.839999999999989, 5.920000000000002, 5.660000000000011, 5.1000000000000085, 5.640000000000015, 0.7700000000000102, 1.0, 7.159999999999997, 10.140000000000015, 8.999999999999986, 6.079999999999998, 0.28000000000000824, 21.019999999999996, 6.909999999999997, 23.659999999999997, 4.840000000000018, 1.5799999999999983, 4.670000000000002, 4.640000000000001, 6.150000000000006, 1.0399999999999991, 5.390000000000015, 14.860000000000014, 7.359999999999999, 15.769999999999982, 1.3599999999999994, 10.280000000000001, 4.099999999999994, 10.61999999999999, 8.939999999999998, 5.420000000000002, 2.719999999999999, 15.280000000000001, 3.760000000000005, 11.659999999999997, 7.989999999999995, 6.799999999999997, 28.049999999999997, 0.9000000000000057, 7.300000000000011, 1.980000000000004, 0.8200000000000074, 6.420000000000002, 6.680000000000007, 0.5000000000000142, 4.47999999999999, 0.8599999999999994, 17.340000000000003, 4.950000000000003, 7.360000000000014, 2.8599999999999994, 3.920000000000016, 2.6399999999999864, 3.8000000000000114, 0.45999999999999375, 14.009999999999991, 1.3199999999999932, 6.289999999999992, 4.340000000000003, 4.219999999999999, 4.1200000000000045, 28.059999999999988, 9.260000000000005, 5.799999999999997, 15.52000000000001, 3.299999999999997, 14.920000000000002, 17.53, 5.560000000000002, 1.7199999999999847, 11.780000000000001, 2.1799999999999926, 11.399999999999991, 7.099999999999994, 4.709999999999994, 21.92, 1.0600000000000023, 0.4299999999999926, 10.179999999999993, 7.060000000000002, 2.4000000000000057, 10.620000000000005, 25.590000000000003, 10.960000000000008, 7.060000000000002, 4.659999999999997, 6.519999999999982, 9.629999999999995, 0.12000000000000455, 4.079999999999998, 2.280000000000001, 5.11999999999999, 25.89999999999999, 9.86, 10.940000000000012, 1.1700000000000017, 13.769999999999996, 13.179999999999993, 8.5, 0.539999999999992, 5.159999999999997, 1.8400000000000034, 6.75, 22.36999999999999, 14.939999999999998, 1.8200000000000074, 2.0999999999999943, 17.939999999999998, 9.469999999999999, 0.6599999999999966, 11.079999999999998, 4.920000000000002, 2.1299999999999955, 18.980000000000018, 10.800000000000011, 15.159999999999997, 1.7600000000000051, 3.4200000000000017, 8.500000000000007, 8.820000000000007, 14.049999999999997, 3.5, 14.58999999999999, 1.7199999999999989, 11.439999999999998, 2.6199999999999903, 0.7399999999999949, 2.9599999999999937, 1.1300000000000097, 3.6399999999999864, 9.719999999999999, 19.22, 16.099999999999994, 3.019999999999996, 1.289999999999992, 1.3999999999999915, 6.480000000000004, 18.16000000000001, 10.870000000000005, 2.719999999999999, 0.46999999999999886, 7.159999999999997, 4.799999999999997, 0.29999999999999716, 2.6200000000000045, 11.439999999999998, 0.12000000000000455, 10.310000000000002, 10.019999999999996, 3.019999999999996, 18.189999999999998, 0.5600000000000023, 9.120000000000005, 0.7399999999999949, 0.9000000000000057, 6.609999999999999, 4.199999999999989, 17.42, 17.159999999999997, 12.129999999999995, 14.099999999999994, 12.560000000000016, 4.599999999999994, 0.6799999999999926, 11.280000000000001, 0.9000000000000057, 10.299999999999997, 6.980000000000004, 5.539999999999992, 9.420000000000002, 0.6599999999999966, 4.480000000000004, 5.200000000000003, 9.219999999999999, 4.860000000000014, 7.800000000000011, 4.1000000000000085, 10.650000000000006, 8.599999999999994, 0.7399999999999949, 11.58999999999999, 2.759999999999991, 12.420000000000002, 3.460000000000001, 7.560000000000002, 9.920000000000002, 22.88000000000001, 19.05000000000001, 2.480000000000004, 15.120000000000005, 10.820000000000007, 3.339999999999989, 3.1500000000000057, 4.1200000000000045, 6.400000000000006, 8.719999999999985, 8.429999999999993, 4.5, 0.12000000000000455, 16.480000000000004, 3.760000000000005, 7.9199999999999875, 18.960000000000008, 4.8799999999999955, 6.180000000000007, 3.8200000000000074, 3.789999999999992, 0.020000000000010232, 1.019999999999996, 13.86, 5.319999999999993, 8.900000000000006, 10.049999999999997, 0.2400000000000091, 14.440000000000012, 5.320000000000007, 2.0799999999999983, 0.4899999999999949, 2.760000000000005, 24.099999999999994, 11.579999999999998, 9.559999999999988, 13.530000000000001, 7.559999999999988, 6.799999999999997, 15.459999999999994, 5.800000000000004, 0.9599999999999937, 6.299999999999997, 7.240000000000009, 3.9799999999999898, 1.4399999999999977, 2.1199999999999903, 1.759999999999991, 0.6000000000000014, 3.839999999999989, 10.900000000000006, 14.52000000000001, 11.079999999999998, 6.200000000000003, 10.83999999999999, 0.5100000000000193, 13.849999999999994, 11.900000000000006, 6.099999999999994, 3.75, 10.159999999999997, 1.1199999999999903, 0.8799999999999955, 9.040000000000006, 3.0600000000000023, 2.0999999999999943, 3.740000000000009, 1.1599999999999966, 13.019999999999996, 11.960000000000008, 19.679999999999993, 10.259999999999991, 0.5600000000000023, 11.129999999999995, 1.0999999999999943, 3.6599999999999966, 8.079999999999998, 5.610000000000014, 14.599999999999994, 1.8799999999999955, 7.3799999999999955, 4.639999999999986, 1.4399999999999977, 9.919999999999987, 10.52000000000001, 5.150000000000006, 3.8800000000000097, 5.38000000000001, 10.780000000000001, 1.6400000000000006, 11.5, 9.200000000000003, 1.5799999999999983, 4.47999999999999, 7.119999999999997, 3.9599999999999937, 1.7000000000000028, 54.640000000000015, 4.940000000000012, 15.659999999999997, 3.6599999999999966, 12.960000000000008, 8.859999999999985, 4.47999999999999, 5.9599999999999795, 1.9399999999999977, 4.840000000000018, 3.8200000000000074, 0.09999999999999432, 7.790000000000006]

print(np.var(KNNResult))


51.35599425782624


In [ ]:
MICEResults = [12.766766250659572, 5.04808166008776, 3.845820192375456, 3.863054597446066, 12.146229583514256, 17.798823288046208, 0.41313324499304827, 6.576792290197076, 6.838067606819422, 1.5194554299059746, 11.871015312941879, 15.344276352854436, 10.970314006507834, 22.070928837210133, 23.47698430867004, 12.322651288325545, 2.8420750827701227, 1.6219617829865456, 10.025747037559, 8.996826882483802, 5.017077519371853, 24.81598490322567, 9.814793777309532, 5.437895402904033, 2.5098759760477947, 4.292602170067269, 4.7759867823597375, 15.49989663872617, 3.031565503122593, 4.096835670222603, 8.935727347233126, 21.79629621540502, 1.9746460793096077, 2.8436979656126056, 14.429617312664078, 4.859526412046421, 1.381676254136778, 1.2534132496843853, 3.0208960437918506, 7.353952920735011, 3.262709514080669, 7.162602904185732, 3.2595680343872715, 0.28407657157120525, 1.9517012155953068, 19.457014999362514, 0.7411264266279716, 1.927576437976441, 8.64013002660164, 4.983321945395943, 0.94978440589054, 10.567880462054916, 0.05811476378887903, 4.637059190886646, 12.436870484039304, 7.919806295330773, 1.334722471625092, 10.230895884014089, 12.866033143439566, 10.387416095643786, 0.07976995551514676, 11.814111642789044, 5.686586639549375, 3.2354180447160843, 0.20659762411384008, 7.375994539420688, 5.899253382446261, 0.5319228912096605, 8.71171554793007, 6.902614185497214, 12.507855716594548, 15.524329719952775, 5.728749531368891, 8.268683750106078, 7.725239467157692, 1.294686414776919, 8.108622776070888, 0.03805927741427695, 8.086867023138424, 1.0727613040440644, 12.279014272381701, 5.277853302209579, 10.09542595950522, 0.7324418133653836, 9.308968986293664, 6.251988722737963, 1.7418226241400845, 18.714685572837155, 2.6244675763602885, 16.821685749087507, 6.6354725336607885, 6.413766913747224, 2.367037015202797, 5.786804681392226, 7.09260496003715, 0.197124851911731, 6.406131552664675, 3.2178325915804464, 3.7049946613952898, 15.05328369312744, 1.7690585643289154, 13.65466304092405, 9.782848356969751, 4.231222895465223, 11.974213052855603, 0.3450652822572806, 14.299531107534179, 3.565287168831645, 5.434034920141713, 2.329623251611025, 6.150301030622657, 2.9356532450601236, 3.5397663956282486, 0.7101577029736603, 3.0060230128269865, 3.7970562131136774, 7.184075389002956, 3.7908911778389154, 0.32408730383244233, 13.7403124609326, 2.6695651129170415, 4.590493647203218, 0.5561765159841201, 6.728208358631775, 0.9680008848400519, 4.783688511130521, 8.2044562290411, 4.816962744378856, 2.794562046141678, 2.2145526788746253, 4.909052168095002, 0.1596048563914536, 7.414045420363863, 28.898892588375347, 3.7929161136479053, 1.5397087795131625, 16.605306149221434, 3.492784903040274, 4.637059190886646, 12.683362187043528, 10.244330948610866, 2.5326604690029484, 0.05650671995681478, 1.0171674112898614, 0.08288801757881004, 6.651129211359574, 4.744316179177417, 7.302338955512852, 11.950793190397675, 6.169376899256548, 6.936821374569874, 5.497723297408015, 3.9549197014653856, 0.8086140452194854, 0.6245390186044801, 14.370265550085563, 22.09913078037296, 1.5085045818642584, 8.156642199464017, 11.03072210201934, 4.6490086238133586, 4.1240300318323335, 5.5539531972550265, 2.4691166435953704, 5.367496268566541, 1.8234101480995122, 22.098371626706438, 10.951711074173303, 3.443052738019162, 3.6512025336116096, 1.8328353529570478, 6.03412671569555, 8.89076392911521, 7.11289405108711, 6.208472214984312, 4.249685478644622, 6.487925035435637, 1.9408300039791015, 3.3140751683289125, 6.914392738163357, 17.13526254187245, 7.6151125516966545, 3.608670387598913, 18.041007825897758, 1.6852684908322715, 3.6965383219566235, 1.5533316575605909, 31.67732529155215, 13.79192305743851, 14.62253160359191, 3.194127792027359, 17.954244602547476, 11.661673293069398, 6.577380030078501, 24.848727765389143, 4.020963639797188, 0.835866147397212, 4.421415877463332, 3.2663964981744016, 1.1588095878871343, 6.099996859607472, 4.2169310349536175, 1.6050072686749957, 1.7499937053712102, 0.4312541425815226, 3.4877528889757485, 16.377450259614236, 0.33016936604185787, 3.9030596902874635, 5.881844312206809, 0.02189646292745806, 0.4990093882055504, 8.875301522579662, 0.9906466501968936, 0.6405834095298246, 4.2478517121970185, 1.649258118498608, 5.291911978914342, 0.4512776274376762, 6.5045229253764205, 3.360229200013478, 12.183084222125743, 8.511646053311424, 7.820617577108408, 4.029730262823108, 4.857589674203197, 6.599847210521162, 13.055152767726966, 12.029341219748503, 12.573027856736118, 4.336819805550817, 9.511303189559627, 3.428381997092913, 12.74610805954596, 3.1354631537270876, 5.0642138440000934, 7.407220807937549, 2.693211605590051, 6.361393006428813, 7.946524989181398, 2.7846717506068472, 5.46455238061607, 10.132365961455633, 18.08985492326609, 1.4992953188300788, 21.588418463628102, 2.8879312318318853, 2.942188182944335, 6.094584259167249, 13.014502290595999, 0.4969186394043703, 14.408235422846474, 5.226603136132184, 2.8988072123222537, 0.7852930188344942, 0.6442238000917939, 1.6188149378999555, 3.224216039414813, 0.4075620792213641, 25.078477462171136, 3.808158835302123, 8.07061852640797, 2.4070514299927055, 6.955758966248595, 16.584205424039794, 1.4537360759332216, 1.5419808878572638, 1.868828327456157, 5.189109919471434, 1.9328012043127387, 12.997279798127934, 4.087054826503348, 13.653900563390636, 0.10959127388177592, 5.253261972943534, 0.307748188675248, 3.9870816586636266, 9.315524086718867, 5.903621136616692, 11.27238686310217, 8.328366844079682, 6.538985622928337, 8.386971784873936, 12.34516047347853, 8.930116795427224, 2.8542003202363873, 3.7578090173466734, 6.915858984451319, 8.489686986180914, 5.570482419722197, 3.920099555227523, 10.500880689803438, 9.773890750235878, 1.8602547056894991, 16.600029131474727, 4.442538365879599, 27.750026454367017, 5.97732924774121, 5.363514251342465, 2.381590657309218, 1.992852415826647, 7.208079627979245, 7.018506642920784, 4.55846696831243, 9.87150590112968, 3.388239039413392, 6.056833049830729, 1.3058130583694236, 0.9244531596615104, 0.4835798211807827, 8.209356286684368, 2.413067963134239, 5.324132905866236, 0.18632596072861674, 3.0070412232036006, 3.4433336003848467, 7.876952879734461, 3.3536901836848187, 1.7939515292012942, 0.5294566475242135, 4.719420677060896, 11.661673293069398, 1.7877249132209698, 2.367076937502361, 4.187707461838457, 6.81969873796578, 6.156090089360973, 2.3137926148550747, 7.86052727007808, 6.704498181849829, 4.661376773279315, 2.097694755410288, 8.433292174372212, 3.550117936048423, 2.3691328090193053, 3.608895759522511, 9.258917506550205, 10.135824825836224, 5.1178086946577395, 3.3140751683289125, 9.727642741157027, 2.995856095270767, 1.8174889820430167, 1.8030232678991638, 2.8218861979483876, 4.861386815916873, 3.8773941208494023, 2.0089345251485184, 1.7525291258549487, 13.569857137052509, 5.672469801011829, 0.513604353871429, 12.504858380914015, 5.157352821706866, 1.3736025481948246, 18.672379018098738, 7.559618209602675, 0.6896750722872866, 10.489809634027111, 4.06349019973419, 1.8908798001556448, 2.4454508791958887, 12.155633899003774, 6.040320790087591, 10.735030153073012, 7.27326965986245, 0.7547878240172423, 0.2613002574885286, 0.18794856240823776, 6.818292886919082, 5.438963126247117, 3.8406157523129423, 1.8198623035200967, 6.400692621344433, 30.07295542898578, 1.9973928067864364, 8.806092892550268, 4.021393688065558, 5.452739038062447, 10.405747090739979, 6.397026894408555, 2.8034388886697172, 11.894927799922883, 2.563567593055353, 0.6883454687387598, 5.776575433212159, 6.453794040363917, 23.567579652347234, 6.0980004222646755, 3.4792574677744597, 2.8842353264403613, 7.645013755353858, 14.057936960743277, 13.192743214534815, 0.8954113577433276, 6.600196437904415, 1.532823270842428, 6.392840827730353, 2.2133410848702653, 3.508114186088463, 2.8307508398573873, 12.737067490346007, 3.500686800488438, 3.34192313380818, 5.091730088568539, 10.239933440482126, 8.673012366102895, 2.022138583463459, 3.7977013702661964, 2.4813611127248976, 15.042253072878793, 0.6496433759408546, 7.178492228674372, 7.888816143093521, 1.3452624795172312, 9.992143539217281, 7.254187135616505, 5.929123152282159, 0.5664842743827876, 0.05164170112996658, 5.151820754405136, 4.453367246488654, 4.5687657322011574, 9.000323617080142, 0.1762685775187265, 1.7106377843600598, 1.6113342145220173, 1.3906409080984332, 0.848153262391449, 2.6419458619926672, 7.166764175666017, 3.005917446310484, 3.67981754535883, 1.6465841461020716, 0.7507369537900104, 1.0239531871721113, 32.74015259771858, 8.731582828871936, 8.894981754319375, 1.672478341370038, 7.1632858065519684, 2.453407546094141, 1.4147738345827747, 19.460726027342844, 3.3820042893658098, 3.6239063194722263, 3.422205495271527, 3.723608908131041, 4.85852889763234, 30.10441415885218, 9.860675475010822, 1.6502388175801883, 4.775165724033428, 4.83465838352582, 9.981167273730918, 13.109465487110157, 7.365555105000794, 0.6869542185415867, 3.5524232331005976, 0.16685098527603515, 8.740349286808566, 20.230011600626227, 3.8363170845241115, 2.680613226836982, 0.37837683423157387, 7.708604632279261, 3.273155319537736, 11.640463619418867, 0.5282547474811707, 9.727267391943272, 1.1099222017789003, 4.416789878227519, 3.6054441743070242, 2.7710381326537714, 0.8554686427841744, 8.30797468612272, 11.812268964652887, 6.985884232394, 1.51211792733271, 0.6585777477557144, 8.157628186936847, 12.401762326787164, 32.753964359483746, 3.7454094080663367, 5.602789006394815, 10.509882573578395, 7.174393367101345, 2.8435121737537656, 4.13448212213072, 3.7984217916872183, 7.923683024041139, 7.869523148545866, 2.2436306340365775, 6.856326267396895, 7.423118720524286, 2.504989997702168, 4.081080157103202, 1.2019032328838293, 8.868408224075793, 10.862916522173727, 9.88814489157346, 0.9406524260268299, 9.456744496502168, 0.18704935594753636, 1.9809465449060184, 2.3832917483984204, 14.854240599764012, 3.93496539121503, 5.807081708907617, 6.090489783134771, 7.75127271349416, 21.052131274324523, 13.659994985737569, 7.520981366493103, 13.03941250356651, 8.065323488999155, 0.7259711915890534, 2.049558376508074, 2.0634296430853, 8.469604314836005, 0.6209776488605598, 2.0936426957021013, 3.678966274476565, 3.5947498955095654, 1.368474808986079, 18.89111301243082, 3.461699881145748, 1.6813529225042032, 10.20617672541141, 4.490540673462888, 3.245986960742826, 5.580652840476802, 11.627922854808077, 4.684677426495696, 0.10732129403453428, 0.7834417402303586, 2.083337239106669, 24.438544684009315, 29.60677426488226, 3.0397220322354457, 0.4917407181606279, 0.002205119283047452, 7.480598539489492, 6.8567305066767545, 6.862184131729464, 2.081430622476006, 4.223316546050498, 3.014292556327362, 0.9400290623142382, 2.4577937229091447, 0.7188555570717767, 51.38416390669143, 5.98539244228877, 8.662741045559372, 8.0743517930661, 1.0976697553476242, 4.888919799530356, 4.880185485422643, 7.6647232117251605, 4.704694590661333, 0.6480802902263747, 0.584302610600453, 4.8271403109831255, 1.9293026446992343, 6.455793785308373, 16.450861764307405, 8.194892511218626, 2.0909922859115255, 10.993292027003221, 1.9867661451751957, 6.007553737381926, 5.389068069909143, 4.684155091590895, 52.66158435020179, 6.813900226189574, 1.6322323545139028, 0.5525063299755004, 1.7138222630246958, 10.652259602753887, 2.79356167518101, 1.898009250387787, 4.3195653478748, 1.96398742424536, 2.91055073388155, 4.261048869304858, 4.406950749957971, 18.95756750467835, 5.740984598443603, 7.918582545936715, 7.337340651169484, 1.4838260518397703, 8.001540593659954, 8.83705377344657, 13.057987543205769, 4.751298109741768, 2.758508591006475, 47.937023370508086, 0.18704935594753636, 2.9266373188685577, 0.34446867053131314, 1.613585142563565, 0.19995327576624788, 2.4714971117517166, 4.732865376934484, 0.30150913796038026, 4.172610362248662, 6.030143398611912, 0.14657816251644817, 20.52298141499024, 9.101515129922035, 20.69409195953901, 1.2513885609198923, 6.302342162114485, 2.4070514299927055, 3.977088652388744, 3.0112080236523866, 5.90035654776905, 2.45918557214155, 11.184684936262684, 5.035276300180087, 1.5439195687094482, 10.947855469845237, 2.741848630668045, 9.786960151018889, 10.181331140163223, 10.837610806432778, 4.011918949389923, 6.218130019229903, 4.490746293293697, 5.003687598528245, 2.710734446376293, 7.006117198751497, 10.811454987432342, 12.620543445028531, 12.923483140426072, 0.20003397913244214, 0.4941081781559973, 1.9192378498702425, 10.002501439801563, 2.336417889313992, 6.897866822844406, 25.152875670699302, 11.951491391505513, 8.96546481525175, 3.8000507682769467, 1.3434962518917075, 4.977514976154808, 7.250763934406464, 3.029255182605624, 0.32527529463452254, 2.446703219238046, 2.4716278122525495, 5.832623375827708, 7.873876303797218, 4.753334149286758, 4.4416142363979105, 4.547781904225346, 8.816177049936243, 1.0521543887496847, 2.428964649167895, 13.338654651821756, 4.9538827792611215, 14.667073420045881, 8.548028547972578, 1.718691370202393, 11.190858522235928, 2.6253433923004508, 2.2970418851934937, 0.6516085195636663, 2.2049487547412383, 2.8886246172520202, 4.160791100391137, 3.213346656231849, 12.34516047347853, 9.435359194521595, 11.780973252660601, 10.240920045037726, 6.217196087910068, 11.021298490597033, 0.5449465135356775, 0.26907309914838606, 0.45505702073590726, 4.795886396080633, 2.505930124175606, 7.355594307893654, 10.54217080107361, 14.927664680326714, 0.36858779143400966, 2.9594641619663236, 11.556889418218518, 7.190555431748507, 0.010245303999582234, 6.6924267764707395, 2.8564103434336943, 9.477040660496044, 1.4194971357325983, 21.461737116833334, 3.15852729820746, 4.8106458806772, 11.413590445419146, 4.038088614465053, 1.2634860076947518, 5.99134642286586, 6.535630477621794, 9.995961440319107, 10.721147718584533, 5.328120785540037, 2.3395594168542857, 3.876223029978263, 1.436311475133877, 1.0353086552843251, 8.001540593659954, 12.33770567229439, 0.22824656355969353, 5.173550623744006, 0.9271190942887273, 9.434522504741025, 2.8595745588046384, 0.9231691589164654, 1.2463318431352377, 2.5756770737789196, 15.182482027120813, 8.329540463885806, 0.7736276767751065, 3.029255182605624, 2.189190603583981, 74.67911968779684, 9.789317211274962, 0.9767674840298355, 4.404002444072248, 6.601145281650929, 7.868664117868761, 3.924620130531835, 12.30815937581496, 2.8065004839445464, 8.319758225899008, 17.379265034378193, 7.320806455503913, 1.5338389752071606, 3.8352842566313257, 7.921091567509421, 21.43889438582606, 6.408218191617081, 7.216948501378354, 0.3155053467023521, 17.21501260520816, 8.118060273422202, 3.3007474147846807, 13.27169054084571, 0.15534563828464343, 3.16543419349793, 7.197921433527085, 4.267673914318735, 4.014100723761899, 4.183697271236056, 12.414835361919785, 4.521919552462407, 15.089458358614621, 3.7815434517542315, 8.609333816268446, 7.096497269249994, 1.477198646004723, 1.2636048171751924, 2.819576532910702, 1.8885604756476937, 0.7698807607248312, 4.343490400787829, 7.813535233031018, 4.694291580795422, 11.925821438236554, 1.9950488899194738, 9.958967420776204, 12.443470260204478, 11.901649962599151, 5.121291097566413, 2.4636041055438795, 1.0635415091828975, 0.277261930739499, 7.647421231742996, 2.4035154290660756, 6.3805795223861, 2.314537083888581, 0.700674995190596, 0.8700436508005112, 7.305609155180846, 4.853776908967944, 13.250862423673937, 7.333635589279268, 0.5895163451125143, 0.8395487262122288, 21.899803135324234, 7.083181331223102, 1.445647304606922, 6.167754304691272, 4.49341431279457, 5.594645953005667, 26.685243889224196, 9.770618578060024, 2.409117767632992, 0.49030301057676695, 9.176462128657448, 5.619791218873004, 4.004173151603581, 10.677603704298548, 2.585266222420259, 1.9854368181098891, 0.4166863143766051, 1.419663251264467, 1.9270006750958686, 2.4751403355308383, 12.37990102146614, 14.098616727887645, 8.841281642214625, 2.067355419501041, 1.3187172147399906, 4.644494494085308, 9.554465298759126, 5.137287532254831, 1.8804191974152502, 0.47886945255167745, 2.166617653133727, 7.465731321780339, 3.0397701694416526, 1.090857571265417, 1.4715552759944757, 2.1275596089936357, 1.658420228137615, 8.55121099150422, 8.350809388991465, 3.03916530484328, 6.903284318152657, 10.646614346855543, 0.8385262458211855, 8.137043887898031, 6.616314374461908, 11.362719765852361, 8.05561941862537, 4.10017969635939, 2.8686671054788064, 2.43160388758551, 4.186849154204609, 23.4732609320464, 6.938840848759071, 1.6583848318875312, 4.792659738109762, 0.14231622921154496, 7.043499902408186, 0.713526009934597, 4.097238584852221, 3.6334281430806357, 7.5732473371412965, 2.67728150856869, 1.7925242336421263, 0.624563808362268, 5.478485939921924, 3.9148070226472527, 2.111228765223075, 45.079623767680516, 2.2806616853901573, 10.062557335517894, 9.547298337544717, 1.6791417691853923, 15.285874729740826, 0.934883045115015, 1.4392699835519949, 21.55350058272022, 2.2342998219621535, 0.25880338341002584, 4.808593945749152, 11.502483666993484, 3.1796443387011664, 7.084931396317913, 11.5892200775872, 4.127016533141372, 0.878364697748168, 8.150186276472837, 6.60014779796272, 10.450358325209649, 5.7322523827832015, 15.303046263812817, 1.0577067931376547, 11.967048864807197, 3.364081529605315, 2.5881304375600678, 0.9546947524079599, 5.807223254761425, 2.0407021790423556, 8.371162683098007, 2.207852951626137, 1.8816026986593357, 31.485115735328876, 35.99676185835102, 0.7536345196793377, 1.6111407388544023, 2.4642154438252106, 11.83471598590053, 0.703319718975763, 4.632289114491307, 3.2870515904756985, 9.149913455007038, 11.071574254072885, 5.916039099485502, 5.937412237518473, 4.159201546316481, 5.867580560624198, 1.6174926456072143, 3.5786083114121965, 10.918357664871749, 23.245341958584604, 2.395799192524649, 5.616655515834751, 0.12675159044408701, 0.9514010138397708, 7.72547780162688, 10.286927482454274, 3.80900888559718, 1.211601756333394, 4.0166273942810164, 9.281418544669393, 2.4059998181372606, 1.5731594548287688, 5.346021218284221, 0.8652400567120964, 1.9180640155821607, 12.212890677844896, 9.601879509774818, 3.617175353120956, 9.410684630667731, 10.394226229803422, 9.592202671619162, 5.1055604236102, 17.065018003466363, 6.317631741054186, 14.40221064401959, 0.4053982387772521, 15.829973754886737, 5.016028519787881, 1.7221272307391757, 4.247688299101441, 2.906220774239415, 12.380939592091678, 8.186211365349578, 7.089171923486546, 2.188744163671089, 8.640894012922374, 1.5111718748180607, 2.489193527905215, 3.3623721131199176, 5.69828621416346, 0.5253263949349503, 1.5999106456125816, 5.900411564148115, 3.536278354270813, 5.612577437238372, 4.355774806446959, 7.272219872377931, 1.609223241852547, 1.1359090271933212, 6.770588945863381, 3.83187571108175, 9.252005561182457, 4.78564112742248, 3.508041251908054, 10.200000515204891, 13.978527930863748, 4.881413150188493, 3.2022586616361224, 3.0667621755368373, 5.622797464601774, 2.172410255375624, 6.047645355614918, 2.007817083571055, 5.1708309624381315, 6.3683003988394375, 8.734408021602974, 5.041409017674269, 8.795361640744844, 6.638037045770147, 3.8032089369801696, 3.483148437621125, 0.7530843617652039, 6.557443971241497, 17.22385863152887, 12.406325914721677, 6.102816532988854, 16.092235227617806, 0.5565055901024607, 2.5080785824078617, 0.4764096438723868, 2.782359239601746, 28.535497456617165, 2.701819344824429, 7.337205252631222, 10.688397613818225, 2.912270421465749, 12.038331208482845, 9.755105259685564, 5.439431519725588, 2.4300448212215855, 9.439111723462759, 5.109547608646309, 1.3384874005056844, 14.576543107029892, 7.883069841712967, 4.081582644293746, 14.498585529827963, 2.856177946828609, 11.364046892192803, 1.2551342807755503, 1.4386932597454205, 6.21141416567373, 3.4730110698654784, 16.951909038602807, 8.267802448382099, 12.005429705769572, 13.035681241298477, 15.317470696415128, 5.324066301326049, 2.1055316563767974, 13.591152042166456, 13.968829605342506, 5.536366235092018, 9.130512891094284, 6.6310474636715355, 2.2686013262361513, 5.635751034841164, 2.4881076272406872, 7.155713748363738, 4.143087787717079, 10.643545759304757, 1.2602096501631337, 23.874111198604623, 9.749127310864537, 8.648429714382516, 1.7525291258549487, 6.2845867573431065, 0.7396155550125769, 2.2876035120544174, 0.4476755557007692, 2.7036119728670513, 8.288222775961614, 1.971694270830497, 10.860807150094558, 6.45302113954186, 7.047666700452979, 8.175642463544747, 0.30902493414923526, 8.67465378380787, 2.0713025801243674, 9.631842499643398, 9.855828812384345, 25.665358205589584, 3.2603228162540745, 6.363903585911942, 1.1079785104497546, 0.5196923674346579, 3.7750741513154935, 11.388468838098106, 5.50793627439279, 1.9293026446992343, 1.5343158330255164, 18.412329664339765, 2.2112885294650937, 6.330607046438786, 4.4273448968189655, 7.420929867709859, 3.776727980856208, 3.879911156383244, 10.455126861026486, 4.3136366909212995, 5.97501089226823, 1.3722848982998812, 8.692194767085397, 1.4158367718899, 3.4186345942366216, 4.929828428959368, 4.35805427906412, 20.868036635538658, 4.2261355288150355, 3.435195449104569, 30.134767696439653, 7.182995734630495, 17.80712641787028, 0.13029415792507848, 9.18974123936357, 4.180553874352441, 4.194128294036773, 7.554246094360053, 15.536027640703182, 19.78488156143527, 12.793157562193741, 3.6758947030831592, 1.3359018723614042, 1.6797584360589894, 3.784713552820307, 6.503957751822014, 9.041381653584963, 22.666841879959932, 8.700980732994637, 4.840848702600184, 17.17206886364049, 3.538378844539821, 7.605895812841879, 7.064195679075809, 2.2378904816894334, 8.022555147577918, 0.2343535261165357, 20.908202471927865, 9.595021818415887, 6.484018460526784, 14.013110365716045, 0.986769957682796, 4.4528156657711975, 3.749526816869775, 7.56372715299311, 6.268921008000575, 1.882105634868708, 22.782014161568, 6.169617888171345, 0.5180134462407722, 0.9426506494141336, 5.523000179335114, 2.98636469736023, 9.064711280829187, 19.27306899923157, 3.396584580473913, 1.129248322200283, 2.363014763872897, 15.15096413076266, 11.769238715770015, 6.115517194711714, 7.065445710768444, 2.7324947682695893, 3.044636597800192, 40.85963080783033, 3.9524085244892717, 4.156390367569827, 18.49292873109509, 5.830716994378605, 0.4650969094075208, 7.178616231875296, 13.816366788979394, 4.296277676705941, 10.15666856018754, 8.128894413215278, 3.451987198758985, 2.021636737422739, 5.7610493051980995, 8.659187769668307, 1.4469922555104802, 4.924183245976977, 3.959425997822322, 2.9317981940664737, 6.223566731826153, 10.450719623107545, 0.5405236564797988, 19.529311706230914, 4.183592167627268, 3.93102238565163, 2.93510922335291, 11.030363083796956, 2.096241473579198, 5.03284464518174, 5.990998019262804, 6.424476939558161, 0.9674745840404597, 13.576411006493714, 6.757598854337388, 0.9647379627383401, 12.856328361998763, 0.07514861875354484, 11.929184087411599, 3.574282321214241, 1.87899838330857, 7.924606316816721, 7.748181562406131, 2.967207758373789, 1.1271034613057367, 0.8055705493219563, 1.3218304957072036, 2.3080730451095093, 1.5962590676169555, 4.524015291074406, 0.8703060798220577, 25.909185790246056, 2.127417712561936, 9.149711860378503, 19.72562446745161, 4.326006096349587, 0.776326048541975, 7.056950057826924, 0.3444054453746759, 3.930904704245279, 1.4001071573503197, 7.328402220542785, 1.0506797187686772, 4.827082838562291, 1.6695997293886364, 10.41066618945591, 3.574282321214241, 7.505336650764761, 1.645904719258752, 0.22345173762998627, 2.8287151769753365, 2.4330103940935857, 0.2996212986713118, 3.103446053683868, 7.129101792740087, 12.37990102146614, 2.0527837535904467, 3.318711067944591, 9.392231305985817, 13.073781147692529, 1.0768326374003863, 0.8774734133597661, 8.555560225945712, 0.34836095848427817, 5.238377751851274, 21.493527713548247, 0.34287883707250444, 11.528747547541329, 2.123699027810673, 7.751391122857953, 2.661630892972042, 9.24534754246325, 5.792132696281158, 3.5060561187068657, 6.90697128943809, 11.97613416388198, 19.41751386608911, 4.538814806366794, 8.380775095816816, 10.8969251351741, 2.791300087963606, 3.2399830908908314, 11.73993585605703, 0.5397218756741182, 3.13964670544199, 4.133186212108953, 6.229012235670567, 9.940588230773741, 7.047666700452979, 10.838007168205408, 1.6457714361170872, 1.3768614965613324, 5.960960704503947, 0.5866772473866604, 7.499375325683744, 3.2645696073315946, 0.7834417402303586, 6.597248268134727, 14.733350806376649, 3.2737074438549882, 4.258100524423142, 1.8399776129152343, 10.528693860077425, 0.2623125136257016, 8.036013898177444, 3.012612240002767, 2.164245053750264, 3.5522509310111587, 7.547616567652611, 7.521491459216932, 10.963747054418121, 4.841295323438359, 0.8081227628605063, 1.0825191971507877, 2.426249639962691, 4.534486525863997, 3.549438698886078, 2.709896700149713, 1.3001655317374485, 3.245336832587739, 5.601319295515125, 2.394322095927663, 4.1039831126150546, 2.0880024830624393, 3.028685158188935, 18.93926596219555, 7.862207063877875, 9.947203318959723, 1.8704569430082643, 3.919117452175513, 1.788437848600097, 8.783975325941668, 11.422793042383049, 2.997820786883537, 4.071144512033456, 3.564793068716213, 5.42682360572303, 3.128725207257034, 12.734259058855756, 0.7663156604129924, 6.623245937713392, 3.429545139443988, 2.6726113022823625, 2.4513740682178167, 1.7591568452242683, 4.616043715138559, 21.43289730614552, 5.173550623744006, 1.048062519552417, 24.237301980445196, 6.2497318665869415, 1.0326680353695536, 7.385504953191642, 0.9936921223774533, 8.220196889093145, 12.192816115474287, 5.319848627202958, 1.6520991334974013, 26.216086904090616, 4.09717185021421, 3.5726942164869655, 4.980676565892836, 0.7814195130056163, 4.996402345215472, 3.1349642103093913, 1.3906409080984332, 5.127802351181799, 2.0603972737278298, 4.543014296332956, 0.7297692167113752, 1.9295621838960528, 5.896171933895431, 2.1902052614071863, 12.343637054939052, 3.080407866287217, 23.038163841178118, 3.3433962765622454, 4.97213318971157, 1.4488795840191528, 3.7750741513154935, 5.420321049493026, 3.48575160408285, 0.7565225471449395, 4.885359494620232, 6.74157358380387, 12.823345811711647, 3.205827214090718, 1.4469922555104802, 0.6484894077881194, 7.0601412737486555, 10.846686584839787, 2.415931981070429, 6.127757418808521, 1.1565913264701067, 0.5272606246234943, 0.029528486809908827, 1.0966216393792223, 6.831820309558516, 1.3198208059359189, 1.2191573816958154, 2.071763792093847, 7.610985567605134, 2.2736433236004956, 2.7580214478896607, 0.8493053138645195, 5.186183953335487, 0.8223219910007629, 5.395460830737633, 0.3267587741754596, 8.184843453345522, 2.528962737469513, 3.172076056961231, 5.74248091093942, 10.091878402914176, 4.185628571048241, 0.20862027419276785, 10.632144370553547, 6.9343645721731235, 1.6084418750898521, 7.938217728048684, 4.1039831126150546, 27.553449159756383, 1.58815763069191, 2.167026127263071, 25.217702296669913, 4.739818816152422, 0.6154001354981773, 1.1206151737212622, 34.02195929316416, 21.625935007074645, 2.673862739089273, 2.384077430443938, 2.3483988039197428, 4.401200230674718, 0.16302486691981244, 16.10501302411184, 6.875711413746899, 5.049368038870043, 5.573737309311852, 8.303516114744824, 1.0948794864581686, 0.3002944571990014, 2.75677101961616, 1.8855012344730824, 7.610562415366346, 14.923771031810887, 0.006509279874734375, 9.05736661177329, 12.31548794122078, 16.957437497634487, 4.788280972473018, 11.253712207345345, 0.13766753216320637, 18.84568070152936, 4.2281006403721335, 4.531634896514788, 1.6555862765036693, 3.0553392789308873, 10.134061053674415, 1.7539090842216893, 8.421161010020796, 2.2966775108071573, 4.707957597788777, 2.7452320157581767, 5.522306462404842, 0.6024753870815687, 16.94871168910693, 3.5319052302668865, 2.7866981681870016, 15.425305165892865, 3.5004563395363277, 0.47903002514557613, 2.325881966192142, 4.719250719505695, 10.87218511926038, 3.8985622989291784, 0.3656655687139221, 0.011283254996200753, 3.573739747506437, 15.064081612633572, 4.021141815940766, 9.78273043325683, 8.159562073829832, 13.823108213737612, 1.8319938590582439, 2.1376451429238443, 3.0923971218670374, 3.3224360659551877, 5.923658585744349, 0.1523039505360657, 18.46538219088724, 3.313287518396649, 11.811638159406499, 5.135629595507055, 9.711435297855402, 3.3044643334446704, 1.1352596616977308, 6.660981513396877, 5.05397745478102, 3.387131662244542, 0.2894406265791929, 1.1467649756319247, 5.822239304200465, 5.488539191724996, 3.719365570879205, 6.095539069585328, 1.8856634886986825, 7.810618652118265, 0.5889002932513279, 7.089786562226109, 12.984442996317384, 1.4957963946085613, 32.34892516472665, 10.646614346855543, 5.9622881848490294, 22.744359580596466, 1.8458925540308542, 5.497723297408015, 7.980287867912324, 5.17650449172416, 10.610115678972932, 27.617132761382095, 2.3777240354965983, 4.744674123948002, 7.098913366204613, 11.349201679226184, 5.407567174899285, 1.6601339000325623, 1.4022481622660763, 3.7981306375185255, 6.472854727425755, 3.3361205804576173, 4.2142919839731405, 26.161880772778332, 14.140666163682084, 4.027015704855671, 0.6382115009101739, 14.889072953619035, 10.27632357439289, 6.0527600417038485, 2.411109684430386, 4.053786417345577, 8.06418381030602, 5.388960974798422, 6.23466185815559, 0.9935073256782658, 6.286759906680587, 2.297997648647609, 2.872256762668414, 7.234576416279424, 0.28131634320821064, 4.03300002288843, 2.7132044978227867, 2.6744124838317163, 1.2695744139431326, 12.981859963931868, 17.228876389377348, 9.736479159457943, 7.533742185209647, 2.2442638030885007, 1.8615621816616539, 18.49740578476353, 12.96494121018253, 0.3150442244817384, 21.239339931824986, 7.546984319862034, 8.941185733252908, 2.37474040919669, 1.069613344173618, 1.0377946988683675, 14.380244842412111, 0.01770746668998413, 2.6627046393221576, 2.0698215401681352, 14.627744899190503, 13.79523909626677, 8.211507430540578, 0.9218115448160944, 4.62998102162814, 9.42765460089116, 1.3323686560015204, 39.56244777038451, 9.475032827684572, 15.929488737209553, 22.11456642171737, 3.80519033597848, 5.50854837247401, 3.3361288268636144, 10.217644007468309, 6.909375069495155, 15.746116488025095, 1.8315495993407325, 6.851715761502746, 11.216446427050826, 3.970849632182336, 58.42979160508435, 10.913508145133363, 12.968097059841497, 2.43455854321698, 13.459940427643986, 0.28305169248444884, 6.824479559127781, 7.230104317959203, 1.0431855218746335, 6.484917829677983, 5.248986838508927, 2.8862509979587117, 3.1428888325470155, 14.343529366193337, 0.34964818351649285, 0.6933360893285538, 5.423704565553265, 2.0384338499540746, 0.8699209299191466, 7.147930028147627, 1.7097008752284353, 9.837859442171876, 4.2918406918515615, 7.647143986241076, 16.90307158325936, 5.227913132288123, 15.345983919390903, 0.026253814496811856, 5.35227599589075, 0.8353823667232518, 4.660144125337183, 7.20846927220272, 6.716080268738352, 12.625143976837393, 20.003817027665548, 0.1560572052112974, 5.827592989978541, 14.25158350616357, 4.859526412046421, 2.576185127726376, 11.068197252234128, 5.6992611135750195, 29.19623282955996, 5.042600288225799, 0.24665876471856052, 2.0909730415672527, 26.675021438072392, 4.911139062081261, 4.218516308018451, 4.0170942264101654, 1.8576893814987727, 3.9593225516259025, 2.9626287517126144, 11.664604976052416, 1.0446218258124134, 33.929234836226726, 5.7012016093748485, 7.536852811406646, 2.7810894809327067, 7.534479755720113, 2.159600395434012, 6.8188931170046345, 6.52024273665846, 5.86370229837739, 4.871311803484417, 5.656007691035498, 3.8185703600886853, 8.142749627971384, 17.679921575014575, 5.97501089226823, 1.4570826957093885, 1.74739163938861, 9.037111844719703, 19.1714491260377, 2.396530000323935, 11.8540018443572, 13.24152273062181, 2.0893260706768757, 9.454605402537808, 2.668805199225943, 1.4326060410596284, 1.4747827339670465, 3.505076232847344, 7.389761657569096, 6.564542871150408, 2.7601445950080574, 12.081481349197489, 7.5143281335058845, 1.0326036523582331, 6.614583731751324, 2.5876280093798982, 18.10160451129437, 4.2128693657122795, 9.184791585387785, 4.01911067451627, 13.178040028452983, 7.566719037049282, 9.8359700092156, 13.243572031678994, 12.830583084023004, 11.589126424389946, 8.446577236687801, 3.298012985461071, 5.078820510019767, 8.935139618872455, 6.0683864604955176, 1.438033118046178, 9.517158036746252, 12.311651302808855, 4.882183286042064, 6.486368492955471, 12.5316727078892, 12.2660585227962, 0.014505057818809064, 3.4318824954205525, 7.28588695908887, 12.764855887436923, 0.47239405569041537, 8.128894413215278, 7.500861222915276, 16.545953706233945, 1.354739872165851, 1.1978318335085874, 1.7950371445661375, 7.425726230077601, 3.473369308383859, 4.966865872398827, 5.310518178071419, 0.37185601317786166, 3.7892880192302982, 2.9245631409749677, 6.882723272077328, 2.529314658041841, 0.7120297622785898, 1.5667621990827172, 4.267783439920379, 7.425046019896499, 1.0326680353695536, 2.780104687011999, 3.7283447766610323, 6.339889183439155, 22.818120174769135, 9.414720880055029, 13.317232748815641, 4.379819733623648, 2.0860067145711128, 0.0431607503219027, 3.4633044736140732, 2.9764708131034325, 1.8296428302349454, 11.875829642072716, 6.400059940712566, 5.380176990608092, 1.8719525455233992, 2.680613226836982, 8.072219596059668, 18.109939159574566, 11.452906945579315, 0.34451720650432094, 3.1796443387011664, 6.626046436555086, 0.29205910856749995, 5.695602612499599, 2.9139341549923117, 6.538566829472586, 4.122596814094692, 8.39096374523416, 7.935505846990338, 14.532212098318894, 5.351241680069791, 8.5541142201443, 11.461067938616779, 11.288992884442507, 5.835052391873674, 1.366088058607417, 7.582317869783793, 12.404909769262588, 4.490189210371, 24.006288923466812, 1.6465841461020716, 5.3271401146797075, 4.114203512410086, 0.9549127328978813, 2.78312958636117, 4.808305507121744, 2.877490706911587, 6.666332153606049, 1.1004353823244912, 4.580369247293078, 3.849954246970327, 9.7498643464221, 0.25973719087993175, 4.876845416174476, 10.176344570165654, 5.320506070947857, 2.2696079169358114, 0.2080626854501304, 13.494657364769324, 1.2750238434920647, 8.522083347919164, 7.155261190469005, 8.742031446259702, 3.435045455739413, 12.115858897324017, 3.42942387179572, 2.8289605733609875, 9.83602564839137, 1.0627482457971382, 5.397866803501714, 14.518343131347649, 5.59044300759831, 14.301514875166106, 2.8159111664206478, 10.275392753818494, 0.4946791992040147, 9.666373847773016, 0.10255311135645684, 5.7349395410888775, 5.1011829500914985, 8.983993152095096, 1.001817388875864, 8.693162618472215, 11.276825725267955, 0.8838994325349603, 1.6438015436936553, 6.3973436233559084, 5.306298323864411, 20.236701650894688, 6.824479559127781, 1.6985488124313832, 29.220770066995385, 14.384849840278562, 12.56184207312009, 3.3845599109244944, 8.328696647245692, 3.5404541292712253, 38.08613304675275, 7.493554643489986, 0.23407873213294295, 8.692259435143242, 10.02504909011212, 2.310522253419421, 0.25741568968355466, 7.065830584224685, 7.4779337571683016, 39.957544771375495, 15.851348071115027, 7.09260496003715, 3.9605970890273454, 7.738789905585321, 5.147616771093411, 3.56576115636247, 4.69740508746365, 0.5101244888406455, 8.787580164172311, 1.2191573816958154, 2.39398426684059, 3.9879864190554315, 2.035727688291743, 4.062476297569958, 7.792351622441842, 2.5599373300959485, 18.323254604010558, 8.767222161349721, 9.091745842718424, 8.49140164962806, 1.6948612972790045, 0.3686743570049771, 6.7398094058689395, 2.2076668830872705, 7.5636881110652325, 5.87369773270467, 12.29366120043116, 4.208592048896804, 0.19891687851456652, 3.035069428656797, 0.8585511773043777, 3.4215984546666647, 15.733347941995376, 15.317470696415128, 8.914128887669207, 0.8050992190267152, 3.945231263857508, 2.568081923549883, 3.33863006021641, 30.0819046447931, 14.200365325682426, 11.878647806795541, 1.8844948850389471, 3.1013931305094076, 6.182597731768567, 7.799253209872688, 7.310696750799423, 4.729289517077518, 2.324583519824955, 7.027782541508657, 8.297881512436135, 4.872078257303997, 8.876450821796382, 1.1597431551407027, 14.201036426275408, 4.246952048298638, 3.403053599923325, 2.6380655464392504, 7.461454373507252, 4.692455423635408, 17.569048613384197, 3.166892220281312, 2.3424454277316897, 2.762879627834252, 0.4836206758861721, 7.644023477550917, 14.648156365078961, 44.47229841806487, 0.04593712176589548, 19.09763907120275, 3.2750381941621356, 0.3303776294196581, 4.052665148244188, 10.26195883226056, 10.539161864679869, 1.5349072897504499, 7.847428017456792, 2.082893929694592, 2.9532492946753024, 8.256570469399094, 9.879316343255638, 6.805353297617245, 7.939410476778249, 10.302212800578133, 5.442685941011604, 2.502234591844271, 4.680615274516995, 3.6747184220385947, 10.828236765219742, 3.9943111654501564, 4.618288754689729, 33.438444328726774, 5.385803420399654, 0.9527371327397134, 28.706649836949282, 18.63876119268535, 22.989753982162412, 7.667779762116126, 2.6193972275723922, 14.144473705447247, 3.075607153979206, 3.779474374123552, 4.290112021822068, 4.186452661714725, 8.289511716517033, 2.780188248383496, 5.6858434627869485, 1.2134546230887224, 0.4693070836562896, 6.450480871260709, 2.2828049952745033, 9.66084520389586, 10.782393704894673, 6.484250169645193, 15.484312011093252, 5.048795281405134, 21.83866624297906, 3.590343525322183, 0.7089151495828361, 4.195478201509317, 28.96789747484216, 15.841199861422098, 5.17404510508954, 1.9367125870147106, 38.81677245459544, 2.4181197921482465, 3.67064825989263, 1.1432707261695327, 7.478605132109195, 4.302084150441345, 0.8573857416521804, 19.83510804636977, 2.7404515188683973, 16.7375027406938, 3.1354631537270876, 4.627325839071091, 4.868589946184159, 6.426406933634354, 12.796062135914553, 8.519841084256583, 15.177212869024899, 1.0433180036534964, 4.037799333552329, 10.604940103726861, 2.1057263192902838, 10.09547197161882, 4.667399165950528, 21.86617248482058, 2.2049487547412383, 3.561391070071309, 0.11573949963030827, 0.9254248243374974, 4.598809593448777, 11.722320818666532, 9.074282076895472, 14.385706062769302, 0.7804302710171527, 0.28500958288061895, 2.3567291148074645, 3.4567651248050453, 7.766985878835996, 5.100642232846241, 8.299943941644457, 16.88612206246765, 8.6764211417632, 11.456447139157746, 4.204479439799556, 4.923487821648919, 15.045933956444586, 8.643477616234101, 13.799688621299069, 2.848983960821414, 0.7139078936336745, 19.09763907120275, 23.153281321983627, 12.840052519420027, 12.530678994820946, 2.575823852552986, 3.435045455739413, 23.356718720810676, 14.076018821001753, 6.295834251561885, 3.8687598978935256, 5.329979699138022, 0.18563620726474994, 3.234853329193939, 0.15928347020803812, 0.05928370401375105, 9.129936361942484, 16.980520829507682, 8.196544793376916, 0.3115304378639365, 8.95558687644106, 7.843461016851833, 6.178108798844548, 2.1926205947725066, 1.4577619836707072, 8.812652833436466, 3.2132265961027286, 0.3218775448025042, 8.238135464344523, 16.41717735543797, 9.229360138540173, 3.096585112697227, 5.5749923597008575, 7.027782541508657, 6.315496766614601, 5.882333119129868, 1.6228913165433596, 1.1018845885815836, 11.656094869614549, 12.79714663925455, 0.10072878244517369, 1.0129957264374099, 0.9524024097795944, 4.358809248621796, 16.214595579565824, 1.4442937085463257, 1.795088209591725, 3.637900693728568, 14.965666755558644, 4.459185431833106, 0.9015292559693222, 9.31354968666598, 0.7576249923885854, 7.473235447506866, 11.479239471886032, 4.207655284212635, 5.925767935437193, 6.330614147902011, 0.6858471545182567, 5.711388222452683, 16.84784866353594, 6.820442174978595, 5.141569973206856, 0.2751644904618189, 0.976096264025287, 4.697055812763665, 3.8334548881162362, 8.613773118789567, 2.0031662833613524, 8.476255499607788, 3.8105207063583038, 16.78142537483346, 6.921780804414723, 9.575247591582055, 1.4035868514653487, 5.727491400354168, 2.5739306992282707, 9.196890809231746, 5.784448302462572, 2.047989788600276, 5.42340346295569, 6.600699127570351, 6.908760436660899, 2.392389230181095, 6.295965323008758, 3.9007165301505324, 5.703702494433244, 10.785764030967016, 4.330865784878995, 5.3541297799761765, 0.39990121492665764, 5.9408782914183575, 20.6459881820077, 8.690766201076798, 4.345465522622121, 5.825682236347632, 9.333791900956214, 5.279175756628206, 2.8829479949754955, 0.8896985123023669, 3.9131241177797165, 7.247711684252394, 2.7670919899409654, 8.019406824856318, 5.666080456872521, 14.38973454747854, 34.13518058336409, 3.4928530063555883, 2.008692518308564, 0.27149809920109647, 4.1573023066307115, 8.186813375440451, 10.33241558280912, 2.5240945917274047, 15.637015834553523, 20.643295256424395, 2.6851093238992547, 12.499906536736347, 4.111874586284628, 5.099136042581108, 2.7728157845833437, 1.4317259284417219, 2.06157270337107, 4.069129234041114, 2.5300609225968174, 5.559781264063304, 4.551781464782536, 25.39536216212798, 5.540683885222819, 3.6358366414110037, 7.884237630332095, 12.039035044696519, 0.4366528882486165, 8.328696647245692, 2.8579650026373713, 8.57234722371011, 3.8991943657431705, 11.46214874923352, 2.6595951940732334, 4.03816053410921, 0.14864071369299836, 4.8673679186631205, 1.504067548812941, 0.875750472045425, 6.829800826028986, 0.04176208921010982, 0.7506292609549803, 0.41098878147747087, 5.230767396851405, 3.912550330038144, 15.078439268706248, 3.959679619754837, 7.865364415808315, 7.322670333021229, 6.500759778217557, 4.58301611946122, 10.406924496779553, 1.9554161974782573, 1.6407224207189017, 0.3007196395393237, 6.335850759730519, 6.164265781327018, 8.6533557588469, 9.867662878260695, 5.338169019289779, 8.763475062262785, 3.42031237549061, 0.8420506939570487, 18.660477028942978, 11.541523406403755, 0.1222541067180174, 0.6920348353165053, 3.7380270762367473, 7.684245856148536, 8.63683682434619, 4.761163382204799, 5.36271989036419, 6.159735994634644, 4.935027971664667, 42.30282127801355, 12.734074148296187, 14.979787085580213, 8.219940819478452, 7.273344177543265, 8.380605526904077, 4.624133179142746, 11.905654297689594, 6.127345689766969, 8.905243477768025, 3.403845295771788, 1.5294726224763622, 7.40570818356268, 13.90148906418662, 0.9021359767569379, 1.971189653050132, 14.533897656391161, 10.346967024945457, 17.240394348194314, 4.001890739402711, 0.7350450096636081, 0.9193934688774306, 0.8258905485639474, 1.590793772168766, 1.4954335061649147, 13.462542121865013, 0.7578511318308756, 6.978935403037141, 14.981224934444135, 6.815684163942194, 5.062241002937938, 5.054638132458763, 2.644966615153592, 1.4144416896804373, 0.1357139279561892, 4.327582732959399, 1.7064523607523938, 8.663715856265341, 4.583631531731115, 6.983701465900317, 1.8371732003504064, 4.143565744775472, 8.087906180135036, 1.813808749377955, 1.9274597978422037, 5.852762423200559, 15.019670896435343, 0.6247473165334725, 0.5841411643459935, 22.23519071235262, 3.6557973495234677, 23.20952619631538, 3.3559413350882465, 5.434599451299434, 20.120265437056517, 1.9440105013647724, 9.727855584880594, 0.2652264250363032, 2.4684446012334433, 7.927072476050526, 0.8184168020870857, 8.304562275206607, 2.257672737441908, 0.7767279484344556, 0.8354664797354587, 9.515036158824572, 9.087362326874342, 4.690060396964213, 12.221323535519815, 17.743515429301013, 1.9289145513224497, 19.733575889851537, 1.4772042571925965, 3.643605849039659, 2.1260307526997053, 14.299531107534179, 2.2568314247570953, 4.081080157103202, 4.8850077387152595, 0.2484297239667015, 9.517869579710649, 5.51658226274202, 15.63296769637364, 0.02127946795843627, 11.115471254811325, 15.856970088882775, 1.7873313536442907, 10.407231212943415, 18.476600439797195, 0.6333272786160933, 7.591413301403037, 4.041167602965871, 12.79930076106723, 5.065137043259995, 3.394768895274467, 0.8582990881805301, 1.7432142410236793, 6.847949519989996, 5.771866013913325, 14.731933043819069, 11.06455230996076, 1.1705921872870562, 6.057779720792325, 12.858997758838754, 8.614968692085853, 3.7953306642103968, 2.3753101390985876, 3.212210610723673, 3.390612385627861, 3.6140148823500198, 23.972442131584003, 3.366885787758548, 1.4085449808163588, 7.865364415808315, 8.480132185632726, 5.187423729554624, 4.1084343496954006, 5.042672096226397, 0.13581546070190598, 3.5509710019331067, 14.136436661863058, 9.412398268768769, 12.264283789554014, 1.0839237371135795, 14.513190080853931, 2.502290385295538, 7.96266849788239, 0.9994027935730116, 0.9100377746189423, 8.49861465593196, 6.50584961349864, 12.73539007546708, 0.09006287892989917, 1.6351064603869645, 0.8703324697232375, 8.050316511609196, 6.915834455148627, 1.0666833663865702, 8.650293477108221, 16.38847901483699, 0.133018060240488, 64.69401157274442, 10.156754640706808, 6.8809195602611055, 6.501042115801866, 4.081080157103202, 11.890596968371327, 9.524158552911928, 4.62057867103907, 7.484293481564052, 3.6304952352864603, 9.01175964227366, 2.7596289011995765, 7.7914381666663814, 20.212773948245655, 4.644845440384785, 19.023287932082198, 5.115021930449359, 13.005615011910464, 18.490432114891192, 7.6974953470647804, 8.332339350489988, 1.5374470466785795, 9.17778814095702, 8.485627076941896, 16.773687596052866, 1.0936473046724302, 0.2151332609805081, 2.4913857164094395, 7.163150134925033, 0.7632982123762133, 1.7014180650117936, 2.8071948942801157, 11.542610255584592, 5.407098086310114, 9.50750949162844, 2.8342486433740532, 0.861265768632947, 5.148485136859833, 0.9911735157947703, 7.4749306389163905, 4.306320109904519, 4.9646092918025175, 14.015652348786006, 10.532279369671713, 4.804544125056481, 37.22477135469498, 3.7463056966271324, 41.00001350971765, 5.374038804102838, 0.9522288288476091, 2.0204243380480307, 3.948788081155527, 11.345944056553208, 6.46954144822007, 7.66123848053779, 0.4543571103095303, 1.8370950221161877, 3.5293392363375773, 12.2767502031341, 5.190836022736207, 6.706093320952178, 2.4725809663095646, 0.952365879037842, 7.673439884258336, 3.1174846653316592, 7.923493972719697, 2.032913852739412, 4.087054826503348, 12.755625289667378, 5.841326775887495, 9.936387506514393, 6.061417915383942, 6.016229646188066, 4.819500231132352, 5.6215241753067176, 6.156665831023503, 8.476347040725457, 2.664154785272146, 12.299044359837723, 2.498852269896773, 0.5251226320808229, 6.715098863293662, 9.113765968500516, 1.1027522348120726, 11.339104969433322, 10.975556527938409, 25.16374190836231, 7.432974319048114, 1.0521543887496847, 2.1943684198619806, 5.713568154587875, 3.0026523251314075, 5.562577913834517, 5.074900277409277, 0.36912907555203844, 9.690082006763959, 15.889703221127888, 12.192003693098442, 3.9869526542415485, 6.95511464507814, 8.90853454858582, 5.923658585744349, 8.294028001604502, 1.3961909757063324, 16.230525511230013, 0.9524024097795944, 4.726772490437824, 21.930082954505693, 1.2973815014783838, 5.576274827308097, 17.811275250510604, 1.2432843620275662, 0.9159378488693903, 2.6190278915425296, 2.408557273583483, 7.774879283184347, 0.3217717085730527, 0.5455805415932815, 6.074520456522762, 5.754163940614845, 1.51211792733271, 14.015652348786006, 2.6646604196588015, 4.22100098547935, 4.833028422992015, 10.133051552015587, 16.624950489136907, 7.456453626720361, 3.295038447233779, 4.874401478896331, 10.831677859326149, 5.235712223594476, 6.779816233580846, 1.9548110507705445, 3.316405106713731, 4.790158132694245, 1.1660264060202792, 0.3945604779352152, 0.5116262696071487, 2.5473206146002525, 0.2705458500021152, 8.113391915656194, 16.49360726183687, 2.3758311942637107, 0.8220051741478471, 1.6556165098659932, 1.6813529225042032, 4.6976878723901905, 0.610276902459276, 1.0641908466934211, 2.593272652680156, 3.1782844630833864, 7.629366576524902, 0.37185601317786166, 21.782992911348472, 4.268847424675386, 3.0937700324449366, 2.5775097189171987, 6.478475041333965, 7.267526751452692, 4.889148677625215, 2.115425232982858, 5.963861612926351, 13.42108843260624, 5.609322970101502, 0.8879781697395828, 17.389041200747656, 6.179710282379816, 40.02135263274026, 15.550840117000917, 6.174109647164926, 1.8403387847000943, 1.197856044116861, 6.943493210116145, 3.1689904833895923, 14.99202519842396, 7.9101269787595925, 1.0443287542663882, 3.028073971794882, 4.241728098943241, 3.8764304843793695, 5.43810954453491, 1.6478110948113311, 4.002327578730629, 2.0135207054854902, 0.153102492963626, 3.7471756680909465, 3.3680908163469923, 1.277139499879965, 0.5508257646309715, 1.217459416090307, 23.095765420768032, 14.4091530850839, 2.5169601672605495, 19.095622024316512, 0.06508919019394455, 4.212904847737505, 1.9561519425527507, 15.39226443915338, 0.7767279484344556, 4.723381012873631, 11.138210393949905, 8.6764211417632, 3.0584033771098547, 15.497498010281518, 17.282972853017185, 17.458004523390713, 1.125567551867661, 5.810263814950474, 2.863182320259085, 4.85037380837025, 4.986808852586918, 2.4454508791958887, 2.1190166038515486, 3.3828887482224133, 10.417538503616655, 4.649009425368803, 14.908227552648171, 3.3859482509707846, 30.02849200859592, 3.8010068708464644, 0.7884782623794138, 28.922999770204157, 0.25725126861166814, 10.078030938372237, 3.4441890468235954, 6.450844127451916, 0.1560436433133532, 6.595309788438698, 23.621708458788333, 2.592701862130127, 33.92739053322143, 5.561056059790062, 9.978143345870876, 12.871271566752682, 2.0689066161987455, 5.997317530025342, 1.376663726570925, 3.479788285968141, 1.832887315280317, 2.2474809070328092, 4.129481365279801, 10.387828774513679, 3.6570055629514258, 7.554246094360053, 5.643886161722328, 6.399379267097771, 4.351846270083129, 1.425176516586177, 11.257564920805635, 1.0344934353365147, 0.6046990995670143, 14.427329409667507, 22.710690889730586, 7.0317503247214574, 0.8273905606235132, 6.725845759880244, 2.179040811971163, 5.989691237857286, 8.770737013536078, 2.2006465100462407, 6.704120427579568, 21.725726679853082, 3.452717766363108, 0.8767544730485497, 13.376841913867068, 3.133342232432611, 10.297234967890432, 12.546921868044066, 5.391065181830157, 26.860090200362166, 10.04414630538679, 6.007444070664889, 1.4147261140537069, 3.6512025336116096, 9.841463328644181, 3.479421897468498, 9.20406168267563, 0.5894812431320844, 1.901702928753366, 3.2158024927992557, 4.737667652225468, 4.1454275789287465, 3.0932054697223066, 2.251334851790716, 0.12244735429248976, 11.427828735738046, 1.41417472584466, 4.965195435111752, 2.375811627113748, 4.48532381353543, 2.901835110403667, 8.08174394603644, 5.988730448066661, 7.066661183828998, 11.250048533675965, 6.196119965755784, 9.529890528341795, 1.9153780111729048, 3.0494463508006504, 12.7747111158656, 0.2686781707245842, 1.0555924655236453, 1.95103545560967, 0.5413242073419013, 0.4879432171119049, 2.8983844180325633, 0.7216082088372673, 7.4749306389163905, 9.560125606653642, 2.536064365875859, 1.0472772036353462, 14.384849840278562, 9.600574647526045, 7.725658498478651, 1.547304760910734, 2.5055317564589927, 11.316244454929873, 39.38594637291649, 6.121898216442105, 6.793818799798672, 7.005220297772439, 0.6601230295561606, 0.416781296094598, 2.534746607094661, 11.665616784783865, 0.8409151754720199, 10.206751486779964, 8.163825647985632, 5.522008220667587, 9.191671126352716, 6.87488748912709, 3.384734881967219, 5.397302232254276, 7.299447386918089, 30.089811253074416, 7.596304684592496, 1.5254042948383102, 12.96910278738578, 0.7227437705756472, 3.6226670370721905, 0.6495694876585816, 15.910526534038837, 3.064939012647585, 9.198465919399212, 14.563711503271321, 1.5452109616603593, 5.026943467392911, 11.421669165481674, 3.122267944424621, 10.349672572235477, 8.593792975963922, 2.218355125493119, 0.049393656554372, 3.2062078870243766, 24.317877694832745, 7.086212516337497, 4.680615274516995, 4.841546487799889, 8.970794859090901, 6.047276176398249, 2.5935898264507102, 4.381820105266513, 0.22852335897898968, 2.8310760954785366, 0.170636858001032, 1.8853243913650815, 0.9235213067180155, 1.6417144066143692, 6.989856736361702, 3.0809992581382417, 5.207005759460415, 9.81034750097875, 2.3689354529481648, 4.133203719141974, 4.600985829395768, 10.824623770272169, 17.03465567973906, 1.2459761544343309, 1.9028380378508558, 21.134014596711083, 6.514227005490639, 9.250345195177616, 5.734058817015949, 8.394543162057388, 3.8151668265966663, 4.804380811489466, 2.770932834588507, 15.22840875007347, 0.7688951743955244, 12.352332840000798, 6.989374316817518, 10.059294666051471, 0.7631221023693087, 3.4539251929864747, 1.443412766908807, 8.906177833476178, 9.887540806396956, 1.5020262870572907, 4.775027541436145, 5.014591512234219, 2.3918121562249155, 17.309692096956383, 1.144014568577333, 2.751529227534405, 0.5641186365469508, 8.548646779244294, 11.455873161835925, 1.0032115171739093, 11.210609789846757, 6.419541930327981, 12.038923630082152, 11.608597672454508, 2.6775975135524703, 4.1817686455610215, 7.120714133566267, 3.857335650337717, 3.871803416993771, 0.3321553161303683, 18.97971528366186, 5.370392953750638, 0.16740105107800218, 3.7615520561346187, 1.9859506644625782, 41.20088156909941, 5.8344825196466275, 0.6568536070148809, 6.63219870566482, 2.0095107332873994, 6.699578537944859, 8.395654451403061, 3.5743227864289935, 6.8725425682673205, 7.23476897963738, 7.4380581749846755, 1.9190677628314745, 3.519575483391975, 16.551005625810006, 3.8411645701082193, 3.7640313474240514, 3.953831739755344, 11.316244454929873, 7.431980265128317, 7.367858544184628, 15.106323994588394, 2.582596581938745, 1.3648461522194992, 4.220259263105419, 5.0891555833370035, 10.568102604030727, 9.231907474762608, 3.3309112308358237, 3.1977590531803344, 11.456296208597365, 4.212950958087035, 8.987632249060752, 3.8618643456423882, 2.9586753790830613, 2.3233071156621747, 5.4298946247675985, 2.1818495637555557, 1.372645401146272, 12.51154616888256, 7.572234432483555, 6.27836372958248, 4.225093095865759, 18.2987014188259, 10.072557376398109, 0.13029415792507848, 2.4159022543446156, 8.224087491399104, 9.943650496852868, 1.123779719410578, 3.353508046411008, 7.099667954764016, 0.7020695805363317, 2.203170213256854, 6.826430507212535, 3.543758539187351, 7.6053085066260735, 9.591741476539568, 5.355106859947426, 2.312544443178524, 2.9513646029848672, 9.379431645427246, 21.099943247904108, 1.9719196164527375, 7.164455899104809, 1.8495188180059827, 17.178355822609547, 7.0676011518183515, 10.695746944687357, 2.0634374430300397, 4.186849154204609, 0.0636317734465024, 12.1901780803543, 6.051440686177202, 7.561000042922757, 12.585147086181692, 26.17212126202773, 6.109644775145611, 8.516453814546765, 0.495935691961229, 1.093836250525655, 2.555204347133653, 0.5857275698892721, 14.131308775680822, 1.0865463972903058, 0.7334360487924556, 4.485393318658879, 12.73539007546708, 7.1051678043371, 25.217702296669913, 5.0675268313088395, 10.17369581733658, 7.219598170580127, 8.747868103383865, 6.463383279096547, 3.2097891764976083, 7.0868931650703075, 8.34015453208167, 2.5596545225903498, 0.9136454486184675, 0.24579638098614964, 2.4636041055438795, 6.252374613163894, 11.336558418087293, 4.844341175307335, 10.301501207306686, 8.476749909654103, 7.610719189671244, 32.704315847231925, 5.278339836646978, 7.086212516337497, 7.198258749853778, 0.46260893931608393, 1.0975580713284216, 2.8840592750072176, 0.488261201545626, 14.247488938480075, 3.5415373308527904, 1.499994494347817, 7.3921859764851945, 6.561496087590413, 4.705507978289589, 3.232010363211046, 5.059447000553874, 2.7335541923210087, 11.455873161835925, 3.3006448994282636, 7.611921142888278, 31.22262454886524, 8.56061729098488, 5.87169491374739, 11.001383973971329, 2.78064140762649, 3.2377677959397317, 4.986467842638618, 16.25467171741505, 0.2002344585843332, 8.609333816268446, 10.511782340313275, 1.1597431551407027, 0.940404939005731, 3.562649733408861, 2.263279066874489, 4.106077763147567, 3.2421555242870994, 9.072287168265177, 2.446325278224066, 35.89302431704375, 2.655637329331725, 0.3697036407163239, 24.81598490322567, 1.796001828492976, 6.657965424834288, 0.30984593862878285, 1.4351657212712041, 4.595249157758872, 2.9159326934059777, 4.324189081046683, 33.94330632555685, 6.2358072826071265, 7.099734981152238, 10.614622727405902, 1.4572990356676314, 6.0367899366166, 1.8959999383573063, 7.831568176497669, 8.86320677272964, 3.352792982816439, 3.5222697897650477, 10.417349595068359, 9.55100464318447, 10.851067727941711, 1.607037746050878, 1.2141257835229453, 3.377005722413031, 8.54696694004243, 2.6840148803818806, 1.6562028390023187, 1.7761193610664776, 0.27680075098757584, 14.88635743514476, 2.4083224931164295, 6.012761654864505, 0.7798640574674778, 18.786765686737056, 6.550442472079695, 3.0032249987310706, 5.341199596121683, 3.304411919331727, 3.617929281194037, 3.2675836122435555, 11.735044266888067, 4.40038054850389, 13.23376128133799, 8.75374750788336, 17.227266324538718, 1.9160841322702424, 6.7615009798632855, 3.855650566130848, 4.737532103098658, 1.7520817355534675, 0.6007141185696128, 6.664512534984567, 2.4702381863914837, 1.0816365048418106, 3.787380321442839, 2.0522281506530504, 0.7031140554391158, 1.7289500349538969, 15.216234171238128, 0.8519748803320937, 8.53376933757555, 12.467984713062052, 1.5284724249622883, 5.505094433784109, 4.402327509557239, 14.387059539941887, 1.2348844679154496, 4.896863962225154, 11.61489021671234, 8.175538281490262, 9.291367602018795, 1.8009929637613453, 3.57988321686274, 15.277788757275559, 8.387744213576596, 9.3617664343341, 4.582932843551646, 7.692381102818615, 7.748464289267105, 7.948216198812176, 3.1717936398893727, 7.555732178412512, 1.8193679174224542, 4.117049392963551, 13.46256499306108, 2.552209198829047, 10.596899712788613, 0.3562359354830704, 2.6155765648503007, 3.6683268067253465, 14.390327455267993, 10.865532372163628, 1.4138346835922988, 6.755464964543563, 18.776683936349357, 0.5472723287831514, 6.254322221708293, 4.002194300517516, 15.987348159937653, 0.3887407855253855, 3.072519131318117, 8.286718739401152, 2.715149128878764, 13.82705387873942, 13.38678517369975, 4.014100723761899, 3.931530074547851, 0.7914152207355301, 1.8234101480995122, 2.3057850886409312, 3.978874038608353, 1.3768544507764346, 7.934205471787379, 14.083605745789015, 5.877289208329032, 2.3787716371014227, 0.8099313646335986, 3.3521827160837887, 1.0307186455254254, 4.478633684331172, 0.351551314826807, 33.21845452737011, 0.8977460496522696, 9.186205438444958, 1.8092701126108466, 11.958929606211584, 3.0682743960279595, 0.8724623629941846, 9.63702959374524, 11.189484269001369, 2.7213302913500286, 2.0893260706768757, 1.6427841456925165, 3.8437119091297944, 5.86446066032687, 6.864412515963437, 6.9599722428197595, 10.273490716345648, 0.10804221024208971, 4.296772540845538, 0.7204835097698918, 34.13518058336409, 1.7902432816776042, 7.636456710467442, 6.543723289565278, 5.174935115246086, 5.8323886992354375, 4.023229385361034, 15.211195210387274, 0.4946791992040147, 7.932971692992595, 0.8119862190610974, 0.6048898734192676, 3.2971695707038293, 3.8985622989291784, 3.290772503256278, 4.016742712378914, 4.054359321616914, 1.1292847265226555, 4.8725490646220635, 0.5899491022357637, 24.372258312197516, 2.113581761749529, 1.0951133807143663, 6.705033181077383, 6.87488748912709, 6.381752200067879, 4.934641902730448, 8.12665203367419, 1.2171634083177594, 16.250059107465646, 2.344199729753825, 12.20427315859186, 2.395799192524649, 6.486876221492281, 2.421818130076886, 4.299219721712447, 8.45977933491119, 13.09545373172952, 2.1137866364205706, 2.263507846116255, 7.086803934616469, 4.079788146803622, 0.44465863000914396, 1.574058093264739, 19.400527345706493, 11.52620964196943, 1.356897705424771, 26.142255419846023, 2.604375583281211, 9.337719504866058, 2.7213302913500286, 7.617468362338542, 7.765191497427182, 7.443975147532058, 1.6465841461020716, 1.2790387930078282, 9.776527204304571, 5.733601192124972, 6.437540215037359, 3.3717410902229403, 5.219567813260539, 0.5061666827147633, 0.3095995906937361, 25.739198267626463, 6.933700230412413, 6.291847035624585, 7.949746208257949, 1.9851787897112985, 3.1827696085419177, 3.873139574379387, 7.312995096353092, 0.9451905451221592, 7.1498549548874735, 10.07224024073058, 7.6879231823564425, 22.28869350926358, 11.045634909989445, 4.276128179219569, 3.860692718992425, 1.0443941453861214, 1.461104471987511, 1.0480903087089644, 1.5410374022952453, 4.790618206028, 7.601982450189681, 2.545116175717439, 2.184009394578908, 13.07129876554211, 1.177948012006155, 7.397315249235348, 7.327684305826082, 9.121408586817111, 3.221361431432314, 25.63361822098652, 0.6685058252457736, 3.887209125562322, 2.0886291580000744, 6.1806420756840055, 2.4868016267302124, 4.300682764097246, 3.3844518758695727, 0.7599867007224788, 3.86307952551266, 1.8057559887328267, 7.840808889496913, 1.0726801862525548, 1.4651387390991033, 13.716915720016203, 10.726409615729288, 13.073568655336032, 14.242863850471721, 4.954445264988976, 7.954903051743628, 3.147903639389753, 7.779410932446382, 4.591652892305277, 4.885641922938916, 4.069884677356669, 7.925380749662963, 1.4970732181507458, 8.258028411645085, 4.8721030534534435, 6.133794433168646, 5.6514434183857105, 2.648128871555315, 16.899527451818074, 3.7836268219303975, 8.494878766463785, 6.9072496734469695, 12.43570864321579, 3.8762542387029697, 1.7730738078701194, 1.2995459849395132, 3.8183603103672112, 11.972108889884694, 1.9728688825978793, 10.596177212700653, 3.3456319332387636, 0.4874100558044745, 0.0649445060866185, 1.223908862756872, 5.147286446823806, 4.123676338248572, 2.3251861584228948, 3.0175107462893322, 1.6613391063713294, 1.1360488420677868, 0.8418497131581972, 7.870917601239057, 5.036870367127079, 9.809014737228154, 19.761950434015816, 1.0514921449088916, 6.473408194161564, 6.859152370925486, 3.647459624864382, 5.463984997029243, 11.871015312941879, 0.6214669732833755, 6.6473476349530785, 6.057779720792325, 22.14712998066365, 6.275347998078104, 4.627974721978404, 3.0570773893180387, 4.34126544895139, 2.3778262655764024, 32.930489965964696, 3.5763805196603755, 3.379405430643672, 8.67136512329239, 2.2533059425735757, 0.5354084236225845, 8.722337824928672, 1.2019032328838293, 15.39634756133617, 3.7992942081863106, 4.1424819362289895, 10.592644932280166, 10.587974772034428, 4.956289854716601, 18.840226261199362, 2.5796027733974114, 6.666540068105746, 1.9235478539654025, 6.175510967233862, 0.6275641753425276, 12.138398755956786, 0.006430175493306933, 5.568656006352015, 6.610434780311351, 0.511479784104182, 0.7158614159733361, 0.04250780644102292, 14.435029540330717, 4.5093905723249605, 7.260280647916915, 2.1240685412703044, 10.37700012149729, 11.68198660099769, 1.514659258214735, 9.493874490766572, 8.33988955666429, 4.265017247572857, 1.6312739884895109, 16.423444825800374, 2.5752084294549036, 3.8703000658043294, 4.387677004367447, 0.14911388006277093, 7.892831623256981, 6.170184001287666, 8.755633434249873, 6.18156755394449, 2.9437959606478614, 5.879125209225023, 3.0466859290864363, 4.666700390995331, 0.25973719087993175, 6.586825740972145, 10.27982808908083, 0.43446389766417326, 0.4570053902679234, 4.266543244161653, 0.09544869009216939, 13.294457565986967, 8.223206524413001, 0.9159378488693903, 28.143346112438067, 29.0407596791416, 4.62869716748834, 6.143653682857163, 1.1989740389785766, 21.966663550793783, 9.963224387283802, 4.258498025258689, 8.487684068656904, 11.051853747994144, 3.009768859608144, 1.7146885685125852, 1.990775124946481, 19.023287932082198, 7.742107976939707, 9.95238214690174, 2.9569208206947195, 0.6439562419076736, 3.984574932654567, 10.123588142640656, 0.0030908858109341963, 0.21327911596841886, 7.6699568286314275, 2.512290220702468, 5.150240847766241, 15.551192427646129, 3.016398259079878, 10.414561268109196, 2.478173164881582, 11.653811976771522, 6.91117151068655, 0.7791506177460832, 4.293744814082132, 1.794796910932149, 8.33750931272678, 0.20335966832449515, 0.6105668275612999, 0.7791506177460832, 4.275251177729388, 10.056668746798024, 0.449016288086014, 2.8106473012291673, 4.212904847737505, 14.391922441481242, 1.5285937058898469, 0.07008605515115107, 0.835866147397212, 1.0727613040440644, 9.121580501617217, 1.4075500163015846, 5.90769584185864, 1.1166112916957474, 13.011599559006001, 15.599783971752188, 3.7986417546262885, 9.197367306391115, 6.694986348864319, 3.153724434237546, 0.5772789965918292, 5.9329214781265875, 4.044118445138423, 7.931392293135119, 5.722519516352875, 1.8683383946478784, 11.678070360118497, 4.952510435964697, 11.8316528084611, 20.025242856823112, 9.838431833445782, 2.935935688197361, 0.7008237254650282, 13.572341490727794, 3.5142354360755377, 8.04328056452789, 2.1421589149297375, 0.2504826923952663, 2.6672132735871514, 1.2411805728103502, 0.8098320016835885, 10.943923362810523, 0.804440983655553, 2.3656881992412195, 4.037424912399231, 10.610563022970524, 0.7458468347501395, 4.4778065611328515, 17.342341761164462, 0.49443801960217115, 1.3336895948282006, 0.8409151754720199, 13.35020895239947, 2.565676083872077, 3.48487552896232, 5.835888940678615, 0.3053768650675721, 6.169577321670317, 13.325963142457212, 10.70001311704931, 1.723625730248358, 19.135174038912936, 4.330319728249705, 1.4308634388482062, 4.814130403700844, 1.8105321527181957, 5.123747688446656, 8.64078890223314, 15.679945923599462, 7.196894764213226, 0.7684398731432651, 1.443125119582831, 4.063386554788835, 2.977097131850826, 8.667654501969736, 0.34506580353406946, 3.740809239763621, 5.352894876040978, 3.864888590657486, 13.176624985167194, 9.550151493169167, 2.848245464476477, 8.80720880029537, 2.8789748680035103, 0.5192919292456395, 2.5749744288860086, 2.184029251185521, 14.472202971852795, 3.434301540434433, 10.547657424915556, 7.91608162889645, 0.13108017595443755, 28.679180530559705, 13.648283757238843, 2.3006549984883122, 1.3630589596354667, 0.7600375745927153, 12.026353167633076, 0.2517630518731977, 1.0813577303811286, 2.07929152608898, 3.5465045459655045, 2.8293863697703756, 3.451342732112792, 11.531993364502185, 2.7016718785642695, 2.9866735271158973, 11.29111542449823, 5.973653246076466, 9.343469660416858, 20.31289109897743, 2.266828923797874, 0.5530780340248782, 1.5545885834942226, 4.108197281023806, 24.604834346722242, 2.7070533905452265, 0.10427111113084209, 2.2580110639894144, 1.758559247513702, 1.3221200308354355, 2.18429873292402, 3.099210687697166, 6.638494875947906, 6.8064528521223195, 1.5861668578267825, 1.017491041248931, 2.648269625949112, 8.148719268851693, 6.916836125710006, 7.280297420244835, 4.19808831185712, 15.842582710671635, 2.344608587971223, 3.9791840303098667, 6.872531670470792, 0.9259616568558187, 14.391922441481242, 3.7802997895939257, 6.359709665050218, 12.603726752873456, 1.5846982056132362, 15.74165796231911, 6.393280837943756, 2.3487816805454145, 1.7552673227093152, 5.458594483113828, 0.03735805395817238, 16.7571119826696, 3.825888350160298, 5.645430249758604, 3.7878468111394312, 1.4285411712729825, 8.329540463885806, 8.756815586073344, 15.806827600791237, 15.085444032331566, 3.5533707802276524, 18.69828562303769, 0.08197878838967654, 1.8418805885251999, 1.397784704414505, 9.699303429020489, 0.8332289222194191, 9.839006940619825, 4.1775370197176045, 0.5873641264382741, 4.042446116528097, 0.898136219674555, 26.879784455839967, 1.7538241688856573, 9.823156450359207, 16.10501302411184, 4.896863962225154, 9.97802089466974, 9.82652524264428, 7.185823712830384, 7.601223480322162, 0.13248098091446536, 40.079647959190055, 6.8103010974359535, 1.0455670813350082, 24.248855499657566, 1.1033944525206891, 0.1590224763080812, 0.09544869009216939, 0.0204138323490497, 3.105535992561471, 3.3621111868180407, 1.70481812957658, 1.6678503076454092, 12.03843552274364, 10.882455927503997, 12.436870484039304, 1.208185354375999, 6.443309968654347, 42.955670982423044, 21.48635355737393, 4.3287594441244295, 0.7748688114030386, 5.004179165546589, 2.540141614743277, 1.1807797952499897, 1.6705218080972344, 0.2979802114978156, 2.8626564084469734, 8.6573906817503, 7.251037147803132, 2.983666886630104, 10.347801175232163, 2.0381810235071214, 1.3943069147144342, 1.3716302303034098, 14.49867160145078, 3.098021306896044, 9.957640830517832, 6.628600006143316, 2.334235170059827, 4.880761834409157, 0.7520153684889976, 3.8502524585345554, 9.057176888027712, 3.6586389571929487, 12.516205604122064, 29.736271326673148, 25.52952313136143, 7.146193446606105, 5.4313321110407315, 5.541434232597354, 12.202482275880655, 4.707957597788777, 4.979380428703379, 17.03769025625823, 8.050691572906771, 0.5097016801142189, 2.016232243083479, 8.935139618872455, 6.892036405931719, 8.99764028980593, 5.048560535782926, 1.114390117927499, 3.3864578046995177, 7.97200379685286, 2.854513774037443, 2.3250015230976686, 0.578662250607394, 29.58923863398303, 2.4674365387702863, 12.453294289843456, 6.24808073938901, 1.3271288124338696, 1.2050896764798438, 17.461433852858278, 4.158936092901115, 6.074520456522762, 2.890919042489571, 14.370265550085563, 4.263287689104644, 4.906221184381209, 0.12629322320240988, 1.7128285699647847, 7.007508025848935, 7.0211757602383855, 54.85758723915151, 5.2248218813746234, 16.2985601607065, 8.923273061485332, 21.86617248482058, 0.6980526655228942, 0.436487396704635, 9.812120867158868, 8.92368729734332, 5.926890676134363, 8.564698764546762, 13.650527405960375, 3.0228001396011877, 5.3541297799761765, 4.896984670930593, 9.38661025478737, 8.232174539343248, 4.851902808622043, 1.1604037984816387, 7.448722028919448, 0.4373954345165174, 1.2775001038440195, 4.605706182839114, 14.195934964380456, 9.64925486637597, 1.499994494347817, 6.915858984451319, 7.250932392625771, 1.8262638297451872, 6.86959291442993, 0.5356442815488123, 2.8748855817224808, 2.4085990039239675, 8.94093425748153, 8.284079120739776, 4.023564746310512, 5.73708235182842, 0.9154272439600675, 0.6922619506427736, 4.521497672681264, 4.199192894528039, 11.109643066490634, 3.7051807608550575, 1.6892530604817608, 9.246915034442381, 3.433970842314551, 0.7989093124195961, 13.72707753346883, 4.409140406192606, 43.16383525813005, 10.60501919236232, 1.8141897692449618, 21.120423089311288, 5.527729702799782, 19.62976337285619, 1.731303217874867, 12.689706751677193, 9.167387503624788, 0.19650398818041026, 5.192499305808539, 2.200183072753518, 5.7939627673901555, 1.621361219319752, 6.931342481558289, 4.258498025258689, 0.49303060902764173, 16.080409022007018, 4.904936391040849, 4.004173151603581, 0.9115480306407306, 0.7684443448453493, 2.908476985469889, 2.6318883141383367, 9.693244734998487, 2.3567291148074645, 2.317605375608821, 9.584089628704035, 0.38385509712806254, 5.424964412878168, 7.642967634737033, 0.4450209321825298, 16.30060469656533, 4.223435055396266, 1.2538445073859634, 1.950038586728283, 2.0470957705434643, 8.813664382085037, 10.17823918514489, 4.38440464325474, 3.550117936048423, 4.275075998860174, 3.2832052993538667, 1.129905962375659, 48.904958805886494, 1.9696170578528722, 2.503678822537964, 14.721557830710267, 14.683536638833303, 3.7335311127119297, 2.238859452106496, 10.5104112734454, 4.3394097321390745, 2.3315753437813953, 0.41425750508096826, 1.1668462563057744, 2.3580496429711815, 8.30914173101641, 2.1010948842803714, 4.970061467013622, 10.393598481325355, 0.1657010767878404, 8.568653305026658, 2.602500664422095, 2.66879971650809, 2.397713993721979, 0.5383861707168336, 0.988778786330748, 2.265807953216438, 0.251629539852928, 0.29126348404847135, 3.892315188387286, 8.469478307133471, 16.168391357885028, 4.547369648018247, 7.168896188802009, 0.3453720028956866, 4.967789095993055, 0.9314849250423407, 4.187234545603999, 5.162553815721594, 7.243067268489526, 10.230371536432315, 1.8993297252449963, 12.520593339475269, 26.714853656570597, 3.6001845574166254, 0.8288736930089016, 2.653705148799091, 1.7210439401649325, 3.030139939453761, 2.8595745588046384, 13.826825311141405, 2.785614967158395, 8.50716567630819, 2.306965612877235, 2.470029609919834, 4.096614053190478, 8.476255499607788, 10.433380112391191, 5.897328011002131, 11.574538579172696, 1.6318538655490755, 8.888743072243244, 11.739145674377312, 1.8481497067665913, 4.679910612048587, 5.4881315454750705, 3.8778988685084954, 3.914929656402606, 19.174215517997297, 13.357002680325806, 9.47394458637946, 4.4820277639124555, 7.605552769160198, 9.38943490792596, 9.49275362109097, 7.425673399471776, 7.011318770004635, 2.640238361420714, 9.138062368329471, 5.994568153120397, 1.5293959700945408, 1.3077134097606944, 8.9002294837328, 6.094584259167249, 1.8103066070508476, 1.4694239413984462, 5.87501145797259, 28.582255266582436, 4.485393318658879, 8.32718788488691, 7.814440618663838, 0.04230528298634795, 3.733448910569635, 2.861849075197682, 11.5658295772553, 5.004583557891181, 4.980917766724431, 4.1393595885131305, 2.5775097189171987, 9.81546578684933, 2.021540314323701, 3.7897194432485577, 5.98511559974726, 17.95023555379126, 4.11882455234462, 4.557211637566709, 5.405858219716137, 2.490966394500539, 5.773502414454171, 1.9090704295249168, 3.6356514737202588, 3.173234073317431, 0.27134357988420277, 11.834503560388058, 7.216854228212725, 1.6682420334577586, 3.313031803972194, 4.600753966027057, 3.2937194291223477, 11.027749351016851, 0.7774363168404506, 1.432819565389181, 3.807250477762551, 6.060550184851209, 29.307174136943573, 2.6888863102270903, 4.992905976722412, 15.040518952122653, 5.634149445596634, 1.3736025481948246, 5.527758008679726, 5.080557093137045, 15.593316402131961, 1.4251143732030869, 3.580470092660434, 1.244861689907836, 12.197088042512746, 0.10706574436130722, 0.6639470566235843, 2.687768583817892, 11.305669678410752, 0.20771578842048655, 3.6662521976261218, 4.092922011461539, 0.9238025659305436, 2.319281018744732, 0.12533240672811985, 4.905283158488189, 5.95778718027934, 0.3091553545143455, 2.488425352606953, 3.0881123341994225, 1.90294953085845, 2.4677782483627624, 15.335203910117329, 14.183009799537928, 6.599129269713032, 6.992756038838195, 7.143171109645763, 10.84154086868071, 23.08225233925586, 2.4183270334305575, 1.4308634388482062, 0.3296639694557655, 1.0932378349414478, 19.640129346439352, 15.575160076869011, 3.2454091133512435, 8.053621566241475, 1.8088571620673122, 7.636456710467442, 0.9047886163881458, 5.812783116960858, 19.220513143495523, 1.8020618574881269, 5.411702300719256, 0.9010912800761162, 2.891867244692321, 9.49182483932968, 11.32496387000414, 4.339452643204723, 38.15947869297091, 33.851035790536685, 1.7176102966680986, 14.739860838472666, 4.559502919605251, 0.9566828556068572, 0.7370713745845876, 0.8569166881947297, 10.242732438967295, 12.678513793654588, 5.820151489199517, 1.7532961186155092, 0.9128873504670878, 2.5530160107111044, 4.250342762119118, 9.517158036746252, 2.838729929067398, 5.4486320456791475, 20.701746569369064, 15.775612800894848, 6.767262919874227, 15.27835571104616, 3.1591035104246146, 13.219517174838671, 11.83471598590053, 6.243740234529199, 2.1694797651536533, 9.203325075034826, 5.304351075608395, 26.409606780848165, 9.140900229658001, 5.8721089208248, 3.2726405916186962, 21.828157792110417, 19.86330584200448, 13.404995463721562, 9.006580120000919, 1.8688183762054251, 8.45568916407197, 2.363629995024752, 0.8879781697395828, 1.760416881960765, 7.5329022382993145, 4.02290494672387, 1.4102963094251493, 8.451334397640522, 2.443777189380924, 8.11867428893136, 24.503066968411943, 14.065989860350854, 1.966609499258439, 3.299683531714635, 3.36547888070605, 20.464207142284636, 3.482134167827482, 6.261115660276545, 11.834503560388058, 4.544389563426407, 1.980388407486899, 18.793038512849975, 9.928876074519792, 12.830992907069032, 14.708206921314854, 14.431522070754227, 7.3037404908898225, 3.6201138107882116, 5.954099948983895, 10.258121155498984, 16.821577670216456, 4.579531804833536, 2.5727956757548895, 8.581327400401193, 5.826263380737771, 0.46672620755191474, 1.8861373073137742, 22.559528487161586, 9.221048240444915, 10.934231796669948, 13.933595241748321, 20.52932892262635, 7.166710186827643, 1.1796517562407445, 1.1164197094096693, 10.198303722288856, 8.666718199966866, 2.34329869653007, 3.1874692582011335, 22.8035924921151, 0.9780725951846421, 7.192283558283293, 7.411646082113073, 1.9399037515419195, 18.33704151609693, 0.7651504541215957, 1.763503928007978, 2.3403337571786835, 0.4422408043595851, 4.409815125084322, 8.794224400389353, 2.7811519755284024, 2.71168302134258, 5.070242852113253, 5.720061330748948, 7.003720610070445, 7.638344320345013, 11.932024305213076, 0.42945102546266867, 4.348845238571059, 23.048525324799897, 4.648714706500485, 3.0449108571279453, 2.720769971276326, 7.506300075521537, 6.6115226174975135, 0.7736193077563343, 16.095374341714177, 3.4702722540244366, 9.813839507191943, 3.1787585267327074, 11.930364777217648, 4.902254325581595, 7.109444676667451, 0.6024753870815687, 3.8766328745493723, 6.267883687410077, 5.682390015429789, 5.718435061842911, 0.145155167168582, 5.096509786123164, 0.721665861249221, 6.531635497470049, 7.504146878643411, 1.7593881157812774, 2.4546871496610123, 7.311725219683623, 1.9482380571402445, 1.016788188553491, 0.04743201889057502, 8.21573136220573, 1.136123418648097, 7.703779496124874, 8.187464693738491, 5.100971996460714, 23.679195793061155, 1.705337511016296, 14.664437845555241, 1.0093985342809049, 3.9508123575422047, 6.706159777314866, 1.5515935255779567, 6.275347998078104, 7.972251580286581, 6.150634792272882, 10.526995071111116, 6.021477623682472, 4.9979908548387755, 8.437518801155235, 7.9360905211001125, 5.579949305836806, 0.2825085850817288, 5.515025934628852, 0.2160529539338114, 7.410986329415579, 5.202732417730942, 2.3478354624420774, 4.723266746735135, 8.488434723357571, 0.8275674296078819, 0.9217031458263705, 14.464539240383928, 10.932661271287941, 3.9501940113645873, 0.19682478401678338, 0.22686092685115966, 10.61840757250694, 5.640216531284565, 3.752029657264032, 0.6896350637979083, 18.46538219088724, 6.370424197731424, 2.7642519773497867, 0.6029391275478133, 8.67240101399699, 6.46954144822007, 5.012263620286603, 6.050862945027404, 2.9829463122337927, 3.6459560110857154, 0.2818334220124328, 2.147799149691167, 6.504425445244465, 9.701836686582539, 0.26580866084569266, 2.416842210046312, 1.0349508789950193, 0.8703060798220577, 8.123034861553279, 0.20546722151289032, 4.303537377352427, 0.2270894026370769, 8.086574746232642, 0.5575798075670093, 2.6952288900463657, 8.499010045615336, 5.928004610926891, 0.3779414281816713, 0.5519315311300943, 6.031045237599386, 17.468009699562558, 13.35874669212923, 0.6382115009101739, 22.63274113619042, 4.33627180376989, 1.7227554788099866, 7.251481764631222, 24.31513547322494, 13.911815484704832, 2.9610503427138326, 20.552086178445478, 1.0328858398825815, 5.985926167951121, 7.693719775979247, 7.3072796570705805, 8.212864507674652, 13.674998919968033, 2.413224003400032, 6.741841079312451, 4.79992623948209, 2.5569864380653655, 5.919990274996032, 3.9741220949369875, 7.190555431748507, 7.8686278401902285, 2.9376973460990428, 6.152390797873991, 4.037799333552329, 1.9709151785438195, 0.07386770572237822, 11.188188508741746, 9.305863137811599, 0.26690473798605296, 11.981573119782084, 4.13988476234185, 11.87424101329404, 3.0626284061394813, 6.5954206873951335, 9.148316710321595, 6.178708574303542, 1.777965046019986, 1.794796910932149, 3.5096088905421965, 5.918104949997087, 5.683179062481443, 18.191537081594447, 0.9959447679132154, 6.263012073531868, 5.351133230017155, 0.6790428836742848, 2.2661727110708796, 2.3860926309184265, 0.42776908666192526, 2.7922741860068783, 6.598348456851724, 2.9337567005186145, 10.011229372048547, 4.589251255962793, 16.050866256728014, 7.307113566346558, 8.592634031733823, 3.441670442236841, 2.14377693302211, 0.25306261275754593, 5.420321049493026, 0.3981122150437244, 2.5929862381368025, 1.7319007227680743, 1.42558486388279, 8.32718788488691, 0.24569423117387146, 6.03386116082774, 0.2654674581268921, 0.7452640297672133, 8.09381363015838, 23.002686661577414, 16.600029131474727, 0.270450167517879, 6.370304602028128, 11.952504292313634, 0.5385362293754952, 9.134330025901534, 0.3324644284397493, 11.045879970409061, 13.012213066356452, 4.817501909175988, 6.029902361752988, 16.9123432903084, 2.066981784403154, 14.851879418483804, 5.805377140924591, 3.644433398395165, 5.23461617750786, 1.9594356432449587, 0.6493091094210399, 24.42822964962265, 2.1943684198619806, 7.590402120540389, 2.1010948842803714, 6.793531741246795, 4.581825023482708, 7.260341815246306, 3.375139089254688, 2.664555587960706, 1.9009325788125437, 9.880246475939586, 0.1989849882717749, 22.39815114061288, 8.091639541051094, 7.067879446762433, 1.5285937058898469, 0.08419087923853397, 3.500686800488438, 10.451559045113939, 2.5916715088979174, 10.452003577855663, 1.0196986553768, 2.574749255879766, 12.866033143439566, 2.619560295030027, 9.922773379364273, 8.82518058957774, 3.509667453531762, 1.7193684462978638, 13.061605781664014, 1.8001726204662987, 0.6102007218379555, 12.842343248749074, 2.582307504316944, 0.9661788003663077, 5.152209896086674, 8.921890599301022, 2.4838807715482147, 2.3593628121424075, 0.8699934400664802, 3.8376671844676764, 3.590343525322183, 6.583577508788537, 6.677778064836801, 12.909731679659373, 3.57411200565231, 3.4791005912969837, 13.349558786612462, 2.522646758402857, 8.02799688086516, 2.204097365484486, 5.346474272989624, 26.153824321797373, 14.228912335063967, 4.773148985659589, 0.466564623393694, 0.22866153031587544, 11.542610255584592, 4.870627469017535, 18.954748846345296, 0.27780187423377356, 14.640151531163482, 19.613306052793533, 12.405481002591628, 11.767446573662937, 4.823745489900595, 12.460945443484619, 1.8470092518811185, 22.862547283419303, 3.312343657772729, 0.07210473564423125, 4.578441355886568, 19.804950880241677, 6.347436791525098, 6.31803365941505, 3.834552371731789, 1.0319187570360953, 4.250342762119118, 2.348179639775225, 11.958156563831338, 16.11011245337349, 2.1881539045356106, 4.275650174842724, 7.184527075663823, 9.854753710255537, 19.375693460643106, 1.0533475929430978, 4.108197281023806, 3.9981693585155824, 9.864158758831039, 1.2795835237488973, 8.80720880029537, 16.624558461055045, 5.327953363201573, 5.524381639053573, 8.317732052715982, 4.200604299326557, 3.8825312592713033, 10.414561268109196, 13.077263490717996, 8.01671498634046, 2.691834751224704, 5.687744916407169, 6.179858741448406, 11.12036867379284, 3.2902601497687556, 4.40767767382016, 8.45855431689813, 21.800355713771538, 3.936809875922478, 5.63216670254441, 4.908290334172037, 9.20406168267563, 2.359054735747165, 2.8799161520758503, 4.38253658614299, 7.84490176241988, 3.547515048346483, 12.095600004254393, 3.0406917151820494, 0.2997311425813649, 3.8727667098697793, 8.332339350489988, 6.08773717727918, 12.040506834252, 6.508327851786703, 44.74568732199555, 1.2100317856736211, 2.9789894326684703, 0.05753755803664262, 5.364623110189797, 1.7448543784124269, 5.562840854305222, 0.047587118685967766, 8.7534860725533, 1.8005221138572978, 8.605113178611461, 11.055664278926074, 5.837839235457139, 7.862207063877875, 18.805698023325547, 5.737283197961872, 6.690450827076631, 3.5375252253104605, 2.7190586135601222, 0.29242371808291523, 2.437040994399055, 6.536736467789794, 2.894711109414075, 22.95169789215906, 1.2651201853869907, 7.04537421624795, 5.46788574336604, 7.018506642920784, 1.0857547467887088, 7.00963729526849, 4.1525997582253495, 0.16179617710473337, 3.2090535289093793, 2.152754511220536, 7.324601979741061, 7.178492228674372, 1.8277404982037808, 2.7282563343273267, 8.380775095816816, 2.7646929768086466, 7.78391517182682, 7.044827474096479, 3.6287146139097217, 0.9524549584575368, 0.4501846666758098, 6.141290091696916, 2.9532492946753024, 5.3968448631310935, 0.01669264943070914, 12.889376108943878, 3.8462032951649263, 29.315100230786825, 2.2634426468142976, 1.5282256740050286, 4.545559810413266, 9.38707090154675, 5.7039562244713125, 6.113549052881382, 5.027709533634734, 0.20472535443299478, 3.1796761882098394, 9.220660733256935, 14.137601517252861, 14.33960346965928, 0.26255363899113604, 15.3503064232169, 12.15232140408743, 7.746145923424052, 1.356897705424771, 7.661921598206945, 3.1101248025513257, 4.281897410105188, 5.0642138440000934, 1.4982007944102094, 11.722825912940507, 0.06508919019394455, 17.099771201255166, 5.55505556688658, 8.321472264569465, 0.9905185082588588, 6.5794389943496725, 0.670821481844726, 10.175912908165657, 3.739857684127003, 4.517825940517085, 6.078360972901379, 3.7739646463708283, 5.986313497736248, 2.686905827021249, 5.828981614006608, 3.5619840766372164, 19.33644102410554, 3.9042265751975833, 0.06057240229500849, 6.7508005912150395, 7.853284029702962, 0.8002895136725385, 8.844485662655401, 11.946110412454246, 3.804692806426658, 4.388885887897118, 8.914563488714222, 7.254187135616505, 4.683945886783292, 1.2017482118667147, 1.7775743688980583, 46.37113056635441, 0.8716726660379237, 1.1975819519210376, 3.5619840766372164, 3.2896617000184847, 0.8204293165961616, 2.728510918520854, 4.57732153723785, 10.939683163535989, 2.718708184506042, 0.4908225171281515, 3.5458171526962303, 2.2535005240665598, 7.314540309367771, 16.99528359329588, 8.651226079483251, 3.3614441515469764, 7.247758774421399, 6.03699691914089, 3.4374941580530987, 2.6755618811704807, 1.0129957264374099, 9.501491658868133, 7.515960400340973, 0.1949684318558269, 0.5186938491196997, 10.746651062070086, 0.04437132901990992, 6.694015957555905, 4.156975867150322, 10.989753451256817, 5.790798432392606, 4.342958535957848, 10.037447015181954, 11.813342661588322, 8.478194347425614, 5.738687683577339, 21.331551992298444, 1.9192378498702425, 8.539935773141536, 14.963186746243338, 15.771722285897212, 8.066670942167704, 3.232010363211046, 0.8217052991636251, 5.010593435701864, 3.2062078870243766, 3.622258959647155, 4.821557707709005, 3.848755130177068, 9.192572666783633, 2.6335809952320233, 5.634149445596634, 4.182766566917252, 3.209598932488319, 0.8652400567120964, 18.201048500922383, 11.373645487782369, 7.2450170812107615, 5.66853498026326, 7.279690174268808, 8.90300602385544, 1.8039307882791888, 2.8260347162297705, 1.9289145513224497, 3.9996045811211047, 7.489855520278866, 8.901909596876237, 1.9593883688637703, 2.70805670602833, 7.341798665268158, 8.96546481525175, 5.51002268408287, 4.890954829878538, 5.488539191724996, 2.0066538163944188, 12.272162827756247, 9.584407119722684, 1.224459331654117, 1.2770607155348301, 1.69216665320873, 8.962450222296809, 4.042446116528097, 5.811838053419862, 4.980710435129865, 3.195308824794722, 34.28617653590136, 3.0776419485706725, 0.11148905919741026, 0.5229594386604219, 5.856131309534902, 3.396584580473913, 2.79002977876911, 9.708759181017925, 8.477460202118095, 7.440548002495262, 4.6471916435689025, 3.9420063206172102, 1.5180356713342604, 9.9799494321841, 18.098410309473408, 1.8983553034288718, 3.619361376666461, 3.3882329061315986, 1.605773241509766, 14.158533824020608, 8.364923909893662, 1.6726113666908873, 5.56650855290502, 1.3899036586339264, 7.727803322497337, 4.796852737265311, 22.69197084259956, 0.6133410689201924, 0.5818299087769674, 6.044015513499659, 11.719971330411695, 3.46704816915819, 5.7951590871328875, 8.753335478750358, 6.874443918452059, 7.239971889686785, 3.599643437217921, 0.30984593862878285, 19.91750275548263, 4.518996456228976, 10.17708041615505, 4.1517630444960645, 2.581417277432408, 0.48416239510731884, 0.40103279743686926, 2.6617348742480686, 47.02904112432283, 0.8814246739983815, 8.396390240904921, 6.676324907202925, 0.03887052804334701, 5.166840309503698, 1.7106815512482711, 3.105812983988713, 2.4357823333696302, 9.804007322224166, 0.3782708094408491, 10.002501439801563, 2.497773219452199, 6.060813733306119, 0.282170600819839, 28.692582899065158, 1.9262677190237127, 0.5502994905023684, 3.1142116590428373, 10.403865166219006, 4.934410239027855, 19.640657999164404, 10.149916316634005, 4.321077289580515, 0.6120381805857704, 4.93735414082154, 1.2389904487235697, 2.6266435671296904, 17.806017420545103, 2.7308099081094497, 13.383703671167638, 7.554033064265312, 2.3908973945081016, 4.098278820522054, 3.261377064371814, 5.918574529408502, 2.3891406753889015, 19.27139097457642, 2.899510977842141, 13.15571133322399, 8.565406141414854, 3.514639056264059, 0.0061639240561532915, 2.2604077984372424, 4.756024993889383, 4.011918949389923, 4.752713382777074, 0.4322242662565401, 8.422630621459447, 16.080409022007018, 30.68459376500192, 3.4168857746648484, 0.4763332352640788, 5.399784147645178, 5.03284464518174, 5.606479196644614, 5.621602202761778, 1.577100941114253, 3.057741953080793, 0.1075713282424573, 3.715788068762919, 0.2894406265791929, 0.08902538052379327, 13.933595241748321, 22.590422190743155, 13.20474840502176, 21.341683654043933, 10.545807209196013, 4.702151724769941, 6.179402956391385, 2.91559508558845, 11.996060688605112, 16.9123432903084, 5.447903833629255, 12.592578583110964, 0.9192403313508208, 0.07630008440807501, 2.350631591760319, 6.273523122019071, 2.3257384400018566, 3.4725365406323334, 0.918133339694549, 4.865147017126773, 8.877324513786618, 12.184638663605071, 5.437895402904033, 1.2883285226042887, 4.505360207372092, 6.407893383243703, 0.30984593862878285, 1.6236567508736641, 2.2973732483789746, 12.487917641101419, 0.9387028742355739, 20.802930411497144, 1.1160632696027903, 7.428750729816414, 13.101111060249835, 19.7605722072321, 2.3756877306428663, 0.49029275241090886, 12.385609891982398, 4.2487779043929095, 4.752713382777074, 5.512096029710818, 16.779551029254606, 11.665621586692495, 3.5938399680942155, 0.08444215086431939, 8.122217166189145, 6.481079785668669, 15.353464068255633, 1.5372789175706032, 7.6699568286314275, 0.17655809197205485, 0.001300374751266986, 14.060275864611313, 5.054298515045545, 1.717281038695475, 3.101193954457372, 34.28617653590136, 3.3882329061315986, 8.813945327920834, 8.25171185263602, 3.7691261509283436, 10.388959733349921, 0.7367330622670067, 9.704030549019862, 7.504146878643411, 5.402821945214242, 0.17968754456038027, 7.601307117143335, 21.08875213788137, 2.2592321234250363, 0.49029275241090886, 1.1508424553564964, 5.656930983220079, 2.623856247720987, 1.0649395632953969, 4.17721314008044, 7.059462493742586, 9.989472283502309, 11.042021180116762, 5.304113217642623, 1.7255876807960107, 4.133640267025257, 6.522079226042337, 3.232010363211046, 1.0842112480176667, 16.696026370486223, 3.047732277702522, 8.213238094712736, 8.934819013276993, 1.4817463569722804, 1.1467649756319247, 11.112949863327515, 0.5413242073419013, 2.0780147539749976, 9.576308170230163, 2.769720636838528, 5.072787025383249, 26.52449742608283, 6.740573850819686, 0.78546114724152, 11.471550404602354, 3.0921449252177524, 1.6745124136875091, 6.330503284295844, 1.555830628507394, 6.056833049830729, 9.404602282443989, 10.648244960499369, 8.347242808990288, 12.738064000600843, 2.529314658041841, 15.05075912980952, 8.541015783991696, 8.217296169739257, 1.1913869873699383, 2.6403874261709035, 4.153108232735661, 13.72969137824984, 0.6519101263923375, 1.3555999097070526, 4.063386554788835, 1.1694728970607855, 12.109694465998444, 3.455977738010887, 7.085410875548533, 2.5177654810206462, 3.0233499433414437, 14.747013530239087, 10.394660476421322, 0.24511697682083877, 0.05491617414867278, 7.279690174268808, 15.283613685091396, 0.1516017313440301, 2.3404330205619885, 3.017479907680908, 5.041014141985514, 5.244461879181159, 4.381820105266513, 5.315600366962471, 9.651822461754932, 3.6910759952119605, 7.957060632288872, 14.039690995507442, 4.503647402213318, 11.857547330652594, 6.115917221391655, 3.9958087423018895, 0.49988924381321453, 13.174307571006395, 8.33750931272678, 4.06610213106039, 12.303857904321916, 0.009241458749386311, 31.22262454886524, 10.819053760701252, 4.446423106545751, 0.17299448118717464, 4.172764006709301, 6.410790639898153, 14.264582911339389, 3.956408106521863, 1.5855324679758382, 6.062542142040144, 4.646028430745858, 2.6751124275341454, 19.96712855602719, 0.7704637003007377, 2.218490743364171, 1.184181618229502, 5.013991899563365, 6.106221757760935, 3.5047188303285424, 6.535922467646401, 5.675268906938442, 6.189382035714573, 0.7005495098460415, 2.350911518642647, 3.32041322395402, 16.820793620240963, 8.241672174726403, 5.7988329696695615, 1.1331800740665798, 20.97663722814137, 4.99410625053612, 11.704189491430697, 1.0716855342110136, 10.41921476155882, 3.1228486924536867, 2.6416487678106364, 2.372086682800287, 0.4910101687110924, 15.997285842824184, 17.908209836973683, 1.3855610562411442, 18.823484274599707, 1.1279519140631038, 5.747022935521713, 1.1679054251830507, 5.074552893845748, 0.5561765159841201, 3.1066072159787836, 2.714548107802443, 2.792174547362336, 1.9787318561517964, 3.7933819112081437, 1.0742316184533536, 5.8574230749741645, 19.764158807052056, 1.5548098214991484, 5.024317799399185, 1.8297736286772448, 0.2640164180717477, 3.4168857746648484, 7.067320655304698, 0.9501569083747512, 6.8573527306001125, 0.3299202841936335, 13.226442135306797, 3.4747763327844012, 1.8370145562360989, 2.502201271249774, 2.2737231013858548, 8.3131115771141, 8.389825291306508, 1.7928144873965834, 9.171462435289598, 1.8088571620673122, 8.671265649673146, 1.4808724047882151, 0.17901822145090307, 3.0628806178985286, 18.709567738736766, 8.451334397640522, 1.678807317167653, 3.0692444777474037, 2.2064663362747154, 8.129929984749467, 6.573803807751418, 4.572011068917135, 6.857236038318916, 12.275876907650542, 1.189901149675535, 7.980673663913848, 4.052931867794825, 2.167558238564297, 24.932932616640855, 6.034383542871822, 1.740894355994115, 4.936226744438329, 7.4527427856064605, 0.037770030041471614, 16.002849117020645, 15.83508967081491, 9.420772149623005, 5.163039530090174, 5.175318046252798, 0.13903672849092175, 7.835354142697426, 3.8514175997483733, 2.9778139976710065, 1.0405116142572979, 3.7346578464209443, 17.246605869764807, 1.8830798911222217, 1.3564490384584644, 4.746132716814003, 7.718692244400373, 15.406259460364566, 6.4722041565925394, 1.2865822194164878, 4.4311636548045925, 0.11678358642653563, 8.339347221608094, 15.461467231903825, 15.471096086279417, 3.7986171647044955, 2.1217896334325843, 6.614998455990445, 3.0946733817736174, 3.487241023536697, 6.586154013379215, 4.1586144275647285, 4.439514516912297, 12.88383946274503, 6.349047889009, 11.406305398986632, 4.3207026052749455, 0.5971514248825969, 1.4058672086470736, 7.163426031282498, 18.445676283186643, 7.477622226080825, 4.000981060351805, 0.8002895136725243, 11.004830134064008, 8.582953164366785, 4.191900792230598, 6.051294720075845, 2.455591453119794, 6.700471604397777, 14.397921365242084, 23.527824749743417, 10.452360651322437, 3.7733364119374073, 5.015398552982049, 3.772091439325834, 4.3986354781250725, 9.17778814095702, 8.186122246155605, 2.711149255793927, 0.8916836220777213, 5.13768250636528, 1.7998248353038093, 7.362619412064703, 1.000915022334766, 4.584863904482802, 1.1220647490571167, 8.758977742289886, 1.862339294667862, 1.7921612972035632, 18.459836269443358, 0.9229121571252819, 9.503024598101575, 5.054670120176539, 3.475227804915562, 3.8030533290571213, 14.182122301728285, 11.491780876392411, 8.528585524313158, 7.766572717571776, 5.495760599031286, 9.087642423803132, 11.143736649227606, 1.2854691554395572, 7.2438289548530435, 13.551171998637045, 8.644114145284078, 8.63356216409143, 6.566389541107483, 10.928443247902422, 0.019086938322800506, 12.109738823669716, 7.067879446762433, 1.1590847701001934, 4.502413586275736, 1.9183224031029908, 2.2617221872788917, 11.89484616490111, 5.1548153598356805, 5.054670120176539, 10.96863560230048, 1.1164197094096693, 10.681406991382786, 0.8088101452673726, 15.349325968731705, 13.488756327803344, 16.97222324130746, 16.392579603766578, 0.976096264025287, 4.650127482446564, 6.1274272495109585, 9.214102903265868, 4.741335467054995, 9.524925393369301, 5.272085363260089, 9.85590193298006, 15.079048943732218, 0.8533716412204342, 5.798631046883116, 15.598285973870716, 2.789430661973725, 7.849058924343467, 22.278435882421604, 4.769913041359132, 4.360798650592798, 0.24244031944168398, 4.767940410551674, 10.962599203796628, 5.173089567555095, 14.81852692256102, 4.909802140839545, 0.7383839790294502, 4.156385902697821, 1.4343792450812316, 5.742371560577141, 6.824786602128043, 1.2863118997771323, 0.12442596042086507, 0.7791637195570758, 22.625953695346524, 10.440650836476024, 1.3966888760887883, 9.88814489157346, 6.68286648991139, 7.503117525931316, 10.028298955710127, 0.7006145371072137, 3.418465650586242, 5.3033556593114355, 10.604940103726861, 1.8447226946132673, 0.16220399007082165, 5.550484415664883, 1.4712778868048844, 3.535308205470116, 4.25547684596566, 8.71206686655151, 2.3971695797221173, 14.608161434591352, 8.2389391326939, 8.97281788976116, 1.2857552981222824, 6.648199785472869, 8.307853807724698, 9.949367831783164, 4.563060888891457, 20.50637192309634, 2.2619697994852146, 4.173106625222445, 9.851360896087172, 1.6936904556288397, 1.6983242381451191, 1.1665858605771575, 2.700976657173271, 10.531139273889451, 8.39544452112078, 21.624616659898223, 13.791643347876715, 0.05630695798666352, 12.915197208379453, 1.1309766443907279, 6.13399577589982, 5.927381630437296, 7.22351719470889, 13.968786377318153, 6.08055226649887, 1.707492008351423, 4.235811373965461, 0.4764793761317776, 12.425244252783543, 8.300806107430716, 6.056833049830729, 16.1264059941214, 13.380047773591244, 8.617298956387657, 3.362794755513633, 9.44790802844507, 0.7347944275311136, 1.9733124897750258, 5.746762263381356, 2.8574112873884374, 3.072641937809024, 7.700004455126802, 51.48550003182821, 1.754433875179572, 6.96176630694309, 0.02122665049816419, 1.1874786448319838, 0.5549433487256579, 0.21443974990782522, 2.3338378817600614, 1.3738265691556109, 6.79126590094036, 3.8873216692741437, 2.2448003433960793, 0.061903597452726444]

print(np.var(MICEResults))


41.8969222380384


7.7043589054054
6.565687295977342

In [ ]:
for random_index in tqdm.tqdm(adress): 
    asthma_df = pd.read_csv('asthma_tmp.csv')  
    tmp = asthma_df.loc[random_index[0], random_index[1]]
    asthma_df.loc[random_index[0], random_index[1]] = np.nan 
    imputerMiss = MissForest(criterion = 'squared_error', verbose=0)
    asthma_MISS = imputerMiss.fit_transform(asthma_df.loc[:, "HR00":"HR23"])
    asthma_MISS = pd.DataFrame(asthma_MISS, columns=["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])
    differenceMissForest.append(abs(tmp - asthma_MISS.loc[random_index[0], random_index[1]]))

In [ ]:

print(differenceMissForest)

[8.849500000000006, 8.525500000000065, 1.5380000000000251, 2.1424999999999272, 12.227999999999966, 19.800500000000028, 0.8365000000000293, 5.993000000000038, 6.760500000000064, 2.506500000000031, 10.60350000000004, 20.401499999999984, 17.742500000000007, 18.80450000000002, 24.747000000000014, 10.105500000000006, 0.9700000000000415, 3.221500000000006, 11.695100000000025, 13.90000000000002, 4.294499999999999, 22.733500000000006, 9.317500000000038, 6.351999999999961, 3.154500000000013, 4.5594999999999715, 5.7270000000000465, 15.317999999999984, 2.7725000000000364, 3.0135000000000076, 9.003500000000017, 21.52049999999997, 0.973500000000044, 5.0449999999999875, 14.147999999999968, 5.020999999999987, 2.4985000000000213, 1.6170000000000186, 1.89650000000006, 10.496000000000038, 7.934999999999945, 5.907999999999973, 3.4500000000000597, 2.1599999999999966, 1.9080000000000297, 20.7055, 1.0184999999999462, 0.9110000000000582, 7.376000000000047, 3.452000000000055, 0.5734999999999815, 6.99350000000

In [ ]:
MissforestResult = [8.849500000000006, 8.525500000000065, 1.5380000000000251, 2.1424999999999272, 12.227999999999966, 19.800500000000028, 0.8365000000000293, 5.993000000000038, 6.760500000000064, 2.506500000000031, 10.60350000000004, 20.401499999999984, 17.742500000000007, 18.80450000000002, 24.747000000000014, 10.105500000000006, 0.9700000000000415, 3.221500000000006, 11.695100000000025, 13.90000000000002, 4.294499999999999, 22.733500000000006, 9.317500000000038, 6.351999999999961, 3.154500000000013, 4.5594999999999715, 5.7270000000000465, 15.317999999999984, 2.7725000000000364, 3.0135000000000076, 9.003500000000017, 21.52049999999997, 0.973500000000044, 5.0449999999999875, 14.147999999999968, 5.020999999999987, 2.4985000000000213, 1.6170000000000186, 1.89650000000006, 10.496000000000038, 7.934999999999945, 5.907999999999973, 3.4500000000000597, 2.1599999999999966, 1.9080000000000297, 20.7055, 1.0184999999999462, 0.9110000000000582, 7.376000000000047, 3.452000000000055, 0.5734999999999815, 6.993500000000026, 0.9170000000000442, 5.46850000000002, 12.538399999999967, 9.792500000000004, 1.6625000000000085, 6.314000000000021, 15.41150000000006, 9.87949999999995, 2.2804999999999893, 3.8420000000000414, 6.862000000000009, 2.786000000000044, 2.242000000000033, 8.525000000000063, 7.6555000000000035, 0.27400000000002933, 7.187499999999972, 7.337000000000032, 10.239000000000033, 11.228499999999983, 3.226499999999973, 2.1125000000000256, 9.596499999999992, 3.4510000000000076, 7.389000000000024, 0.7134999999999394, 18.493499999999983, 1.767500000000041, 7.626999999999953, 2.442499999999953, 14.399999999999949, 0.7835000000000321, 9.005999999999986, 5.681499999999971, 1.5380000000000535, 19.037999999999997, 1.1520000000000579, 14.43049999999998, 10.819500000000005, 6.657299999999992, 2.2035000000000053, 5.368500000000068, 6.2880000000000535, 0.34600000000002495, 9.37049999999995, 1.7074999999999534, 1.9280000000000115, 11.136999999999972, 2.3460000000000036, 12.166999999999973, 9.36499999999998, 3.6110000000000184, 10.368499999999997, 0.24750000000001648, 18.619999999999933, 0.8035000000000423, 4.484500000000068, 3.7485000000000355, 3.8670000000000897, 3.0599999999999454, 3.6739999999999355, 0.41349999999997067, 1.3759999999999906, 1.612000000000009, 4.6490000000000435, 5.099499999999949, 0.6125000000000256, 14.36499999999998, 2.225999999999999, 3.6730000000000587, 2.083999999999989, 7.39549999999997, 2.2700000000000387, 4.4755000000000535, 5.993399999999966, 0.1599999999999966, 2.431500000000085, 3.666500000000056, 4.549500000000023, 6.100500000000011, 9.69000000000004, 29.36400000000002, 1.1970000000000312, 3.687499999999986, 19.119499999999988, 5.3160000000000025, 6.2475000000000165, 12.467500000000058, 10.276500000000027, 3.813499999999962, 0.7352000000000345, 4.105999999999938, 1.0844999999999487, 4.093499999999949, 5.833499999999987, 12.833499999999987, 11.292500000000032, 7.028500000000037, 9.115500000000011, 0.8340000000000742, 4.940500000000014, 3.146000000000001, 1.1945000000000476, 13.289499999999933, 18.191000000000017, 0.17200000000002547, 10.960500000000039, 16.077499999999958, 8.631, 9.26700000000001, 1.2519999999999527, 8.376499999999993, 3.282499999999999, 1.4245000000000374, 25.340000000000003, 13.144499999999965, 4.223999999999975, 2.8869999999999294, 1.3425000000000722, 7.610500000000016, 13.32250000000002, 6.032999999999987, 6.357500000000044, 1.4015000000000413, 8.829499999999996, 1.2135000000000318, 0.522500000000008, 8.418500000000009, 15.378500000000003, 9.004999999999981, 3.9830000000000396, 16.94900000000002, 5.084500000000048, 3.4080000000000723, 4.251500000000021, 33.51649999999994, 7.968000000000018, 11.940500000000014, 4.556000000000012, 18.12649999999998, 15.434499999999971, 3.016500000000036, 25.18649999999998, 3.6980000000000075, 10.32399999999997, 4.6910000000000025, 3.2944999999999283, 3.4135000000000133, 3.9969999999999857, 8.606500000000068, 0.19300000000001205, 0.8994999999999607, 4.487499999999969, 5.077500000000029, 17.335500000000025, 0.3985000000000696, 2.5699999999999505, 5.142000000000024, 0.4294999999999476, 2.2620000000000573, 10.302000000000078, 2.8474999999999824, 1.8724999999999739, 5.709500000000048, 1.073000000000036, 5.414500000000018, 0.5199999999999818, 9.189000000000036, 7.5530000000000115, 10.599000000000032, 11.68799999999996, 11.523499999999999, 2.8805000000000263, 4.724999999999994, 8.762999999999963, 14.629000000000048, 12.462399999999931, 10.080500000000029, 3.5710000000000264, 11.050500000000014, 4.679999999999964, 7.653000000000034, 4.614499999999964, 5.30199999999995, 10.154000000000025, 5.123500000000021, 7.10199999999999, 7.21700000000007, 1.5120000000000573, 3.850000000000037, 8.263499999999965, 17.135500000000064, 0.17749999999993804, 12.527500000000003, 3.9290000000000305, 1.0459999999999212, 2.103500000000011, 8.910499999999999, 1.7215000000000487, 24.730500000000035, 9.481499999999997, 5.808000000000035, 0.6609999999999587, 1.8705000000000638, 3.284499999999966, 3.3034999999999854, 0.3480000000000132, 22.933499999999967, 3.0620000000000545, 4.969999999999985, 2.0840000000000316, 6.564000000000092, 11.80850000000001, 2.824000000000069, 0.7484999999999502, 4.782500000000084, 8.220000000000027, 1.8140000000000498, 14.985499999999973, 6.367000000000033, 13.848500000000058, 2.2169999999999703, 6.069500000000019, 0.8600000000000421, 4.09699999999998, 8.82750000000005, 5.672500000000014, 9.84399999999998, 6.889999999999972, 6.320999999999998, 8.215999999999951, 10.665499999999966, 5.584500000000006, 4.886999999999944, 6.402499999999989, 7.911000000000001, 7.300499999999971, 4.815000000000055, 4.81399999999995, 12.436999999999912, 12.291500000000013, 0.32299999999995066, 14.336500000000015, 3.4079999999999444, 25.876499999999993, 6.737000000000037, 2.90799999999993, 0.7970000000000113, 1.61850000000004, 6.152000000000072, 10.954999999999998, 3.889499999999984, 6.6984999999999815, 4.754500000000064, 10.853500000000025, 2.8095, 1.8660000000000565, 1.664000000000044, 9.076500000000038, 0.8190000000000168, 9.421999999999954, 4.129999999999967, 9.282000000000039, 7.9140000000000015, 2.7784999999999656, 3.702000000000055, 0.06549999999994327, 1.0795000000000243, 4.860000000000028, 14.40449999999997, 1.2070000000000363, 3.398500000000027, 4.672999999999945, 1.7579999999999956, 4.902499999999904, 2.6759999999999735, 9.927499999999952, 6.8579999999999615, 2.213500000000039, 2.086999999999975, 11.341500000000053, 1.964500000000001, 4.3589999999999804, 0.14300000000004331, 8.131000000000043, 9.898999999999958, 0.5020000000000522, 3.0224999999999653, 6.201499999999982, 1.504499999999993, 3.3195000000000903, 2.7379999999999285, 3.990600000000029, 3.9320000000000306, 3.5704999999999814, 4.1860000000000355, 0.7894999999999897, 14.582499999999996, 5.633999999999958, 2.0564999999999287, 12.838499999999954, 5.903499999999994, 5.386500000000012, 12.623499999999964, 2.51400000000001, 1.202999999999932, 3.5989999999999753, 3.35750000000003, 2.9924999999999784, 2.018999999999977, 18.223000000000027, 5.082000000000065]

print(np.var(MissforestResult))

32.044052275547216


In [ ]:

print(sum(differenceMissForest)/len(differenceMissForest))

6.643018956043962


6.643018956043962

## Compute the kids with the fewest missing values

In [ ]:
def extractChild(df):

    # Create a new database per children and save it into a list of all the same subject numbers.
    df_children = []    # List of all the children
    
    # Loop through all the subject numbers
    for subject in df['SubjectNr'].unique():
        df_children.append(df[df['SubjectNr'] == subject])
    
    return df_children

In [ ]:
# Move up 2 directories
data_directory = '../..' 

# Load the CSV files
asthma_df = pd.read_csv(os.path.join(data_directory, 'Data', 'astma.csv'))

asthma_df_children = extractChild(asthma_df)

counter = []
#for every child compute the number of Nan in the HR columns
for i in range(len(asthma_df_children)):
    counter.append((i, asthma_df_children[i].loc[:, "HR00":"HR23"].isna().sum().sum()))

print(counter)

# sort the counter list by the number of Nan
counter.sort(key=lambda x: x[1])

print(counter)

[(0, 24), (1, 91), (2, 72), (3, 15), (4, 23), (5, 16), (6, 76), (7, 61), (8, 25), (9, 53), (10, 85), (11, 273), (12, 56), (13, 230), (14, 42), (15, 613), (16, 108), (17, 36), (18, 16), (19, 13), (20, 18), (21, 100), (22, 98), (23, 108), (24, 42), (25, 39), (26, 40), (27, 163), (28, 47), (29, 420), (30, 109), (31, 21), (32, 38), (33, 36), (34, 37), (35, 460), (36, 24), (37, 23), (38, 22), (39, 93), (40, 36), (41, 47), (42, 345), (43, 28), (44, 27), (45, 28), (46, 431), (47, 52), (48, 156), (49, 130), (50, 29), (51, 39), (52, 76), (53, 411), (54, 92), (55, 225), (56, 44), (57, 190), (58, 41), (59, 284), (60, 19), (61, 19), (62, 72), (63, 38), (64, 20), (65, 54), (66, 43), (67, 69), (68, 42), (69, 26), (70, 82), (71, 26), (72, 124), (73, 67), (74, 38), (75, 40), (76, 28), (77, 27), (78, 60), (79, 213), (80, 131), (81, 42), (82, 102), (83, 32), (84, 32), (85, 293), (86, 117), (87, 59), (88, 46), (89, 20)]
[(19, 13), (3, 15), (5, 16), (18, 16), (20, 18), (60, 19), (61, 19), (64, 20), (89, 2

In [ ]:
# get index of a random element in a dataframe
def get_random_index(df, columns):
    # select a random element that is not Nan
    # take a random column in columns
    tmp = df.index
    number = np.random.randint(0, len(tmp))
    random_index = tmp[number]
    random_column = columns[np.random.randint(0, len(columns))]

    while pd.isna(df.loc[random_index, random_column]):
        tmp = df.index
        random_index = tmp[np.random.randint(0, len(tmp))]
        random_column = columns[np.random.randint(0, len(columns))]

    return [random_index, random_column, number]

In [ ]:
interestingKid = [19,3,5,18,20,60,61,64,89,31,38,4,37,0,36,8,69,71,44,77,43,45,76,50]


randomKid = []
for i in range(2500):
    # get random element of interestingKid
    randomKid.append(interestingKid[np.random.randint(0, len(interestingKid))])

adress = []
for kid in randomKid:
    adress.append((kid, get_random_index(asthma_df_children[kid], ["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                                               "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])))

print(adress)

[(89, [2610, 'HR11', 27]), (89, [2586, 'HR10', 3]), (19, [562, 'HR16', 11]), (37, [1076, 'HR11', 3]), (69, [2011, 'HR10', 9]), (5, [145, 'HR21', 0]), (89, [2607, 'HR23', 24]), (69, [2013, 'HR20', 11]), (43, [1274, 'HR05', 27]), (8, [240, 'HR09', 8]), (76, [2208, 'HR00', 2]), (19, [564, 'HR12', 13]), (3, [114, 'HR21', 0]), (4, [122, 'HR08', 6]), (50, [1473, 'HR14', 23]), (44, [1284, 'HR09', 8]), (20, [594, 'HR10', 14]), (76, [2229, 'HR05', 23]), (43, [1266, 'HR04', 19]), (18, [543, 'HR07', 21]), (50, [1471, 'HR22', 21]), (19, [568, 'HR07', 17]), (8, [259, 'HR21', 27]), (61, [1792, 'HR19', 23]), (20, [603, 'HR20', 23]), (3, [89, 'HR06', 2]), (45, [1333, 'HR08', 28]), (76, [2230, 'HR15', 24]), (38, [1126, 'HR23', 24]), (37, [1092, 'HR19', 19]), (71, [2085, 'HR02', 25]), (44, [1286, 'HR01', 10]), (8, [254, 'HR10', 22]), (71, [2088, 'HR11', 28]), (50, [1478, 'HR15', 28]), (5, [154, 'HR05', 9]), (77, [2253, 'HR13', 18]), (45, [1310, 'HR04', 5]), (36, [1047, 'HR19', 3]), (38, [1109, 'HR12', 7

In [ ]:
# Play with the number of neighbors
# Look at the kids with the fewest missing data

# TODO: read the csv all the time.

differenceKNN = []
differenceMICE = []
for random_index in tqdm.tqdm(adress):
    asthma_df = pd.read_csv('asthma_tmp.csv')
    asthma_df_children = extractChild(asthma_df)
    tmp = asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]]
    asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]] = np.nan
    imputerKNN = KNNImputer(n_neighbors = 5)          # n_neighbors=2
    # asthma_df.loc[:, "HR00":"HR23"] = imputerKNN.fit_transform(asthma_df.loc[:, "HR00":"HR23"])
    asthma_KNN = (imputerKNN.fit_transform(asthma_df_children[random_index[0]].loc[:, "HR00":"HR23"]))
    asthma_KNN = pd.DataFrame(asthma_KNN, columns=["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])
    # get the absolute value of the difference between the original value and the imputed value
    differenceKNN.append(abs(tmp - asthma_KNN.loc[random_index[1][2], random_index[1][1]]))

    asthma_df = pd.read_csv('asthma_tmp.csv')
    asthma_df_children = extractChild(asthma_df)
    tmp = asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]]
    asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]] = np.nan
    mice_imputer = IterativeImputer(verbose=0)
    imputed_data = mice_imputer.fit_transform(asthma_df_children[random_index[0]].loc[:, "HR00":"HR23"])
    # Convert the imputed data array back to a DataFrame
    asthma_MICE = pd.DataFrame(imputed_data, columns=["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])
    differenceMICE.append(abs(tmp - asthma_MICE.loc[random_index[1][2], random_index[1][1]]))

  0%|          | 3/2500 [00:01<15:17,  2.72it/s]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
  0%|          | 10/2500 [00:03<14:36,  2.84it/s]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
  1%|          | 17/2500 [00:05<12:03,  3.43it/s]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
  1%|          | 22/2500 [00:07<11:41,  3.53it/s]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
  1%|          | 23/2500 [00:07<13:01,  3.17it/s]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:

In [ ]:
print(sum(differenceKNN)/len(differenceKNN))
print(sum(differenceMICE)/len(differenceMICE))

# the variance
print(np.var(differenceKNN))
print(np.var(differenceMICE))

7.153252000000006
6.897327812555437
41.669131304496
40.72816928317416


In [ ]:
for random_index in tqdm.tqdm(adress): 
    asthma_df = pd.read_csv('asthma_tmp.csv') 
    asthma_df_children = extractChild(asthma_df) 
    tmp = asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]]
    asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]] = np.nan 
    imputerMiss = MissForest(criterion = 'squared_error', verbose=0)
    asthma_MISS = imputerMiss.fit_transform(asthma_df_children[random_index[0]].loc[:, "HR00":"HR23"])
    asthma_MISS = pd.DataFrame(asthma_MISS, columns=["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])
    differenceMissForest.append(abs(tmp - asthma_MISS.loc[random_index[1][2], random_index[1][1]]))

# STEPS

Evaluation of the steps with child

In [14]:
def extractChild(df):

    # Create a new database per children and save it into a list of all the same subject numbers.
    df_children = []    # List of all the children
    
    # Loop through all the subject numbers
    for subject in df['SubjectNr'].unique():
        df_children.append(df[df['SubjectNr'] == subject])
    
    return df_children

In [15]:
def preprocess(asthma_df):
    asthma_df.loc[:, "steps00":"steps10"] = asthma_df.loc[:, "steps00":"steps10"].fillna(0.0)
    asthma_df.loc[:, "steps22":"steps23"] = asthma_df.loc[:, "steps22":"steps23"].fillna(0.0)
    return asthma_df


In [16]:
data_directory = '../..' 

asthma_tmp = preprocess(pd.read_csv(os.path.join(data_directory, 'Data', 'astma.csv')))
asthma_tmp.to_csv(('asthma_tmp_steps.csv'), index=False)

In [17]:
# get index of a random element in a dataframe
def get_random_index(df, columns):
    # select a random element that is not Nan
    # take a random column in columns
    tmp = df.index
    number = np.random.randint(0, len(tmp))
    random_index = tmp[number]
    random_column = columns[np.random.randint(0, len(columns))]

    while pd.isna(df.loc[random_index, random_column]):
        tmp = df.index
        random_index = tmp[np.random.randint(0, len(tmp))]
        random_column = columns[np.random.randint(0, len(columns))]

    return [random_index, random_column, number]

In [18]:
interestingKid = [19,3,5,18,20,60,61,64,89,31,38,4,37,0,36,8,69,71,44,77,43,45,76,50]

asthma_df = pd.read_csv('asthma_tmp_steps.csv')
asthma_df_children = extractChild(asthma_df)

randomKid = []
for i in range(2500):
    # get random element of interestingKid
    randomKid.append(interestingKid[np.random.randint(0, len(interestingKid))])

adress = []
for kid in randomKid:
    adress.append((kid, get_random_index(asthma_df_children[kid], ["steps11","steps12","steps13","steps14","steps15",
                                                   "steps16","steps17","steps18","steps19","steps20","steps21"])))

print(adress)

[(3, [108, 'steps13', 21]), (69, [2028, 'steps17', 26]), (20, [592, 'steps17', 12]), (36, [1048, 'steps13', 4]), (8, [249, 'steps15', 17]), (61, [1776, 'steps16', 7]), (76, [2206, 'steps14', 0]), (38, [1115, 'steps20', 13]), (4, [132, 'steps15', 16]), (36, [1050, 'steps17', 6]), (38, [1114, 'steps16', 12]), (18, [525, 'steps16', 3]), (4, [119, 'steps19', 3]), (19, [579, 'steps18', 28]), (69, [2007, 'steps18', 5]), (37, [1085, 'steps16', 12]), (36, [1068, 'steps17', 24]), (77, [2259, 'steps14', 24]), (45, [1331, 'steps18', 26]), (89, [2583, 'steps17', 0]), (60, [1752, 'steps15', 12]), (18, [549, 'steps14', 27]), (77, [2237, 'steps19', 2]), (60, [1763, 'steps18', 23]), (20, [593, 'steps12', 13]), (3, [100, 'steps16', 13]), (50, [1455, 'steps21', 5]), (50, [1463, 'steps20', 13]), (18, [538, 'steps15', 16]), (31, [914, 'steps20', 15]), (18, [537, 'steps21', 15]), (64, [1870, 'steps21', 2]), (8, [258, 'steps13', 26]), (36, [1063, 'steps17', 19]), (50, [1476, 'steps18', 26]), (38, [1125, 'st

In [19]:
# Play with the number of neighbors
# Look at the kids with the fewest missing data

# TODO: read the csv all the time.

differenceKNN = []
differenceMICE = []
for random_index in tqdm.tqdm(adress):
    asthma_df = pd.read_csv('asthma_tmp_steps.csv')
    asthma_df_children = extractChild(asthma_df)
    tmp = asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]]
    asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]] = np.nan
    imputerKNN = KNNImputer(n_neighbors = 5)          # n_neighbors=2
    asthma_KNN = imputerKNN.fit_transform(asthma_df_children[random_index[0]].loc[:, "steps11":"steps21"])
    asthma_KNN = pd.DataFrame(asthma_KNN, columns=[["steps11","steps12","steps13","steps14","steps15",
                                                   "steps16","steps17","steps18","steps19","steps20","steps21"]])
    # get the absolute value of the difference between the original value and the imputed value
    differenceKNN.append(abs(tmp - (asthma_KNN.loc[random_index[1][2], random_index[1][1]])[0]))

    asthma_df = pd.read_csv('asthma_tmp_steps.csv')
    asthma_df_children = extractChild(asthma_df)
    tmp = asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]]
    asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]] = np.nan
    mice_imputer = IterativeImputer(verbose=0)
    imputed_data = mice_imputer.fit_transform(asthma_df_children[random_index[0]].loc[:, "steps11":"steps21"])
    # Convert the imputed data array back to a DataFrame
    asthma_MICE = pd.DataFrame(imputed_data, columns=[["steps11","steps12","steps13","steps14","steps15",
                                                   "steps16","steps17","steps18","steps19","steps20","steps21"]])
    differenceMICE.append(abs(tmp - (asthma_MICE.loc[random_index[1][2], random_index[1][1]])[0]))

  0%|          | 2/2500 [00:00<08:05,  5.14it/s]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
  0%|          | 5/2500 [00:00<07:44,  5.37it/s]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
  1%|          | 18/2500 [00:03<07:41,  5.38it/s]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
  1%|          | 24/2500 [00:04<08:03,  5.12it/s]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
  2%|▏         | 38/2500 [00:07<07:05,  5.79it/s]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:6

In [ ]:
print(differenceMICE[0])

26.009351025580372


In [20]:
print(sum(differenceKNN)/len(differenceKNN))
print(sum(differenceMICE)/len(differenceMICE))

# the variance
print(np.var(differenceKNN))
print(np.var(differenceMICE))

352.661040000001
358.2866777002531
173628.5712021184
161053.59673862148


Evaluation of the steps with the whole dataframe

In [ ]:
asthma_df = pd.read_csv('asthma_tmp_steps.csv')

adress = []
for i in range(2500):
    adress.append((asthma_df, get_random_index(["steps00", "steps01","steps02","steps03","steps04","steps05","steps06","steps07",
                                                   "steps08","steps09","steps10","steps11","steps12","steps13","steps14","steps15",
                                                   "steps16","steps17","steps18","steps19","steps20","steps21","steps22","steps23"])))

print(adress)

In [ ]:
differenceKNN = []
differenceMICE = []
for random_index in tqdm.tqdm(adress):
    asthma_df = pd.read_csv('asthma_tmp_steps.csv')
    tmp = asthma_df.loc[random_index[1][0], random_index[1][1]]
    asthma_df.loc[random_index[1][0], random_index[1][1]] = np.nan
    imputerKNN = KNNImputer(n_neighbors = 5)          # n_neighbors=2
    asthma_KNN = imputerKNN.fit_transform(asthma_df_children[random_index[0]].loc[:, "steps00":"steps23"])
    asthma_KNN = pd.DataFrame(asthma_KNN, columns=["steps00", "steps01","steps02","steps03","steps04","steps05","steps06","steps07",
                                                   "steps08","steps09","steps10","steps11","steps12","steps13","steps14","steps15",
                                                   "steps16","steps17","steps18","steps19","steps20","steps21","steps22","steps23"])
    # get the absolute value of the difference between the original value and the imputed value
    differenceKNN.append(abs(tmp - asthma_KNN.loc[random_index[1][2], random_index[1][1]]))

    asthma_df = pd.read_csv('asthma_tmp_steps.csv')
    asthma_df_children = extractChild(asthma_df)
    tmp = asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]]
    asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]] = np.nan
    mice_imputer = IterativeImputer(verbose=0)
    imputed_data = mice_imputer.fit_transform(asthma_df_children[random_index[0]].loc[:, "steps00":"steps23"])
    # Convert the imputed data array back to a DataFrame
    asthma_MICE = pd.DataFrame(imputed_data, columns=["steps00", "steps01","steps02","steps03","steps04","steps05","steps06","steps07",
                                                   "steps08","steps09","steps10","steps11","steps12","steps13","steps14","steps15",
                                                   "steps16","steps17","steps18","steps19","steps20","steps21","steps22","steps23"])
    differenceMICE.append(abs(tmp - asthma_MICE.loc[random_index[1][2], random_index[1][1]]))